In [0]:
%sql
ALTER TABLE axiom_datalake.cl_snapshot.fact_axiomfund_investment
SET TBLPROPERTIES (
  'delta.columnMapping.mode' = 'name'
);

CREATE TABLE axiom_datalake.cl_snapshot.fact_axiomfund_investment_test AS
SELECT *
FROM axiom_datalake.cl_snapshot.fact_axiomfund_investment;

In [0]:
%sql
-- select * from axiom_datalake.cl_snapshot.fact_axiomfund_investment_test

truncate table axiom_datalake.cl_snapshot.fact_axiomfund_investment_test

In [0]:
%sql
with CTE_ef_ec as (SELECT 
   b.soi_fund_name                       as f_soi_fund_name                 
	,a.of_prior_manager_funds_committed_to as f_of_prior_manager_funds_committed_to 
	,b.qr_recorded_fund_name  as f_qr_recorded_fund_name
	,a.over_subscribed  as f_over_subscribed
	,a.next_gen  as f_next_gen
	,a.advisory_board_member  as f_advisory_board_member
	,a.investment_type  as f_investment_type
	,b.ips_definition_realization_status  as f_ips_definition_realization_status
	,a.deal_source  as f_deal_source
	,b.description           as f_description
	,a.commitment_date_portfolio_summary  as f_commitment_date
	--,a.exit_date  as f_exit_date
	,cast(from_unixtime(unix_timestamp(a.exit_date, 'dd/MM/yyyy'), 'yyyy-MM-dd') as date) as f_exit_date
	,a.co_investment_offered as f_co_investment_offered 
	,a.investoraccountiqid  as f_investoraccountiqid
	,a.axiom_as_sponsor  as f_axiom_as_sponsor
	,a.board_seat  as f_board_seat
	,a.vintage_year_internal     as f_vintage_year_internal
	,a.fund  as f_fund
	,b.dc_iqid  as f_dc_iqid 
	,a.investor_currency  as f_investor_currency
	,b.fundiqid as f_fundiqid
	,case when lower(a.investment_type)=lower('Primary mainline') or lower(a.investment_type)=lower('Primary toehold') then 'primary'         when lower(a.investment_type)='secondary direct' or lower(a.investment_type)='secondary' then 'secondary'                                 when lower(a.investment_type)='co-investment'  then 'co-investment' else 'others' end as deal_category
  ,NULL AS c_vintage_year_internal
	,NULL AS c_stage_at_investment
	,NULL AS c_soi_fund_name
	,b.qr_recorded_fund_name AS c_qr_name
	,NULL AS c_co_investment_pool
	,NULL AS c_entry_valuation
	,NULL AS c_listing_status
	,NULL AS c_fund_ownership_at_entry
	,NULL AS c_management_ownership_at_entry
	,b.fund_size AS f_fund_size_sec
	--,NULL AS c_drawn
	,NULL AS c_purchase_price
	,b.size_of_purchased_commitment c_size_of_purchased_commitment
	,NULL AS c_other_investors_at_entry
	,NULL AS c_investment_type
	,NULL AS c_ips_definition_realization_status
	,NULL AS c_company_description
	,NULL AS c_fund
	,NULL AS c_companiesiqid
	,NULL AS c_dcid,a.company
	,a.fund as investments
	,a.investorinfofundiqid as Axiom_fund_iqid
	,NULL as Axiom_fund_name
  ,b.fund_status as fund_type
FROM axiom_datalake.raw_snapshot.efront_fundinvacc_ol a
LEFT JOIN axiom_datalake.raw_snapshot.efront_fund_ol b ON a.fundiqid = b.fundiqid
UNION 
  SELECT 
   NULL as f_soi_fund_name
	,NULL as f_of_prior_manager_funds_committed_to
	,NULL as f_qr_recorded_fund_name
	,NULL as f_over_subscribed
	,NULL as f_next_gen
	,NULL as f_advisory_board_member
	,NULL as f_investment_type
	,NULL as f_ips_definition_realization_status
	,NULL as f_deal_source
	,NULL as f_description
	,coalesce(cast(axiom_commitment_date as date),cast(from_unixtime(unix_timestamp(a.axiom_commitment_date, 'dd/MM/yyyy'), 'yyyy-MM-dd') as date)) as f_commitment_date
	--,a.exit_date as f_exit_date
	,cast(from_unixtime(unix_timestamp(a.exit_date, 'dd/MM/yyyy'), 'yyyy-MM-dd') as date) as f_exit_date
	,NULL as f_co_investment_offered
	,a.investmentsiqid  as f_investoraccountiqid
	,NULL as f_axiom_as_sponsor
	,NULL as f_board_seat
	,NULL as f_vintage_year_internal
	,a.fund as f_fund
	,NULL as f_dc_iqid 
	,investor_currency as f_investor_currency
	,NULL as f_fundiqid
	,case when lower(a.investment_type)=lower('Primary mainline') or lower(a.investment_type)=lower('Primary toehold') then 'primary'         
	when lower(a.investment_type)='secondary direct' or lower(a.investment_type)='secondary' then 'secondary'                                 
	when lower(a.investment_type)='co-investment'  then 'co-investment' else 'others' end as deal_category
  ,a.vintage_year_internal               AS c_vintage_year_internal
	,a.stage_at_investment AS c_stage_at_investment
	,b.soi_fund_name AS c_soi_fund_name
	,b.qr_name AS c_qr_name
	,a.co_investment_pool AS c_co_investment_pool
	,a.entry_valuation AS c_entry_valuation
	,a.listing_status AS c_listing_status
	,a.fund_ownership_at_entry AS c_fund_ownership_at_entry
	,a.management_ownership_at_entry AS c_management_ownership_at_entry
	,NULL AS f_fund_size_sec
	--,a.drawn AS c_drawn
	,a.purchase_price AS c_purchase_price
	,a.size_of_purchased_commitment AS c_size_of_purchased_commitment
	,a.other_investors_at_entry AS c_other_investors_at_entry
	,a.investment_type AS c_investment_type
	,a.ips_definition_realization_status AS c_ips_definition_realization_status
	,b.company_description AS c_company_description
		,a.fund AS c_fund
	,b.companiesiqid AS c_companiesiqid
	,b.dcid AS c_dcid,b.company
  ,a.company as investments
   	,NULL as Axiom_fund_iqid
	,c_fund as Axiom_fund_name
  ,NULL as fund_type
FROM axiom_datalake.raw_snapshot.efront_coyinvest_ol a
LEFT JOIN axiom_datalake.raw_snapshot.efront_coy_ol b ON a.company = b.company)

,CTE_final as(
select 
f_soi_fund_name,
f_of_prior_manager_funds_committed_to,
f_qr_recorded_fund_name,
f_over_subscribed,
f_next_gen,
f_advisory_board_member,
f_investment_type,
f_ips_definition_realization_status,
f_deal_source,
f_description,
f_commitment_date,
f_exit_date,
f_co_investment_offered,
f_investoraccountiqid as f_investoraccountiqid,
f_axiom_as_sponsor,
f_board_seat,
f_vintage_year_internal,
f_fund,
f_dc_iqid,
f_investor_currency,
f_fundiqid,
deal_category,
c_vintage_year_internal,
c_stage_at_investment,
c_soi_fund_name,
c_qr_name,
c_co_investment_pool,
c_entry_valuation,
c_listing_status,
c_fund_ownership_at_entry,
c_management_ownership_at_entry,
f_fund_size_sec,
c_purchase_price,
c_size_of_purchased_commitment,
c_other_investors_at_entry,
c_investment_type,
c_ips_definition_realization_status,
c_company_description,
c_fund,
c_companiesiqid,
c_dcid,
company as company,
investments as investments,
Axiom_fund_iqid,
Axiom_fund_name,
fund_type
from CTE_ef_ec a --where (fund_type != 'Blocker Vehicle'or fund_type is null)
union
select 
f_soi_fund_name,
f_of_prior_manager_funds_committed_to,
f_qr_recorded_fund_name,
f_over_subscribed,
f_next_gen,
f_advisory_board_member,
coalesce(a.investment_type,f_investment_type) as f_investment_type,
f_ips_definition_realization_status,
f_deal_source,
f_description,
f_commitment_date,
f_exit_date,
f_co_investment_offered,
coalesce(investmentsiqid,f_investoraccountiqid) as f_investoraccountiqid,
f_axiom_as_sponsor,
f_board_seat,
f_vintage_year_internal,
NULL as f_fund,
NULL as f_dc_iqid,
investor_currency as f_investor_currency,
NULL as f_fundiqid,
case when lower(a.investment_type)=lower('Primary mainline') or lower(a.investment_type)=lower('Primary toehold') then 'primary'         
	when lower(a.investment_type)='secondary direct' or lower(a.investment_type)='secondary' then 'secondary'                                 
	when lower(a.investment_type)='co-investment'  then 'co-investment' else 'others' end as deal_category,
a.vintage_year_internal AS c_vintage_year_internal,
a.stage_at_investment AS c_stage_at_investment,
b.soi_fund_name AS c_soi_fund_name,
b.qr_name AS c_qr_name,
a.co_investment_pool AS c_co_investment_pool,
a.entry_valuation AS c_entry_valuation,
a.listing_status AS c_listing_status,
a.fund_ownership_at_entry AS c_fund_ownership_at_entry,
a.management_ownership_at_entry AS c_management_ownership_at_entry,
NULL AS f_fund_size_sec,
--,a.drawn AS c_drawn
a.purchase_price AS c_purchase_price,
a.size_of_purchased_commitment AS c_size_of_purchased_commitment,
a.other_investors_at_entry AS c_other_investors_at_entry,
a.investment_type AS c_investment_type,
a.ips_definition_realization_status AS c_ips_definition_realization_status,
b.company_description AS c_company_description,
b.dcid AS c_dcid,
NULL as c_fund,
coalesce(a.companyiqid,c_companiesiqid ) as c_companiesiqid,
coalesce(a.company,a.company) as company,
coalesce(a.company,investments) as investments,
c.axiomfund_iqid as Axiom_fund_iqid,
Axiom_fund_name,
fund_type
from CTE_ef_ec d
join axiom_datalake.cl_snapshot.dim_axiomfund c on Axiom_fund_iqid=c.axiomfund_iqid
left join axiom_datalake.raw_snapshot.efront_coyinvest_ol a on d.f_fund=a.fund
LEFT JOIN axiom_datalake.raw_snapshot.efront_coy_ol b ON a.company = b.company
where fund_type='Blocker Vehicle' and investmentsiqid is not null
union
select 
f_soi_fund_name,
f_of_prior_manager_funds_committed_to,
f_qr_recorded_fund_name,
f_over_subscribed,
f_next_gen,
f_advisory_board_member,
coalesce(a.investment_type,f_investment_type) as f_investment_type,
f_ips_definition_realization_status,
f_deal_source,
f_description,
f_commitment_date,
f_exit_date,
f_co_investment_offered,
coalesce(investmentsiqid,f_investoraccountiqid) as f_investoraccountiqid,
f_axiom_as_sponsor,
f_board_seat,
f_vintage_year_internal,
NULL as f_fund,
NULL as f_dc_iqid,
investor_currency as f_investor_currency,
NULL as f_fundiqid,
case when lower(a.investment_type)=lower('Primary mainline') or lower(a.investment_type)=lower('Primary toehold') then 'primary'         
	when lower(a.investment_type)='secondary direct' or lower(a.investment_type)='secondary' then 'secondary'                                 
	when lower(a.investment_type)='co-investment'  then 'co-investment' else 'others' end as deal_category,
a.vintage_year_internal AS c_vintage_year_internal,
a.stage_at_investment AS c_stage_at_investment,
b.soi_fund_name AS c_soi_fund_name,
b.qr_name AS c_qr_name,
a.co_investment_pool AS c_co_investment_pool,
a.entry_valuation AS c_entry_valuation,
a.listing_status AS c_listing_status,
a.fund_ownership_at_entry AS c_fund_ownership_at_entry,
a.management_ownership_at_entry AS c_management_ownership_at_entry,
NULL AS f_fund_size_sec,
--,a.drawn AS c_drawn
a.purchase_price AS c_purchase_price,
a.size_of_purchased_commitment AS c_size_of_purchased_commitment,
a.other_investors_at_entry AS c_other_investors_at_entry,
a.investment_type AS c_investment_type,
a.ips_definition_realization_status AS c_ips_definition_realization_status,
b.company_description AS c_company_description,
b.dcid AS c_dcid,
NULL as c_fund,
coalesce(a.companyiqid,c_companiesiqid ) as c_companiesiqid,
coalesce(a.company,a.company) as company,
coalesce(a.company,investments) as investments,
c.axiomfund_iqid as Axiom_fund_iqid,
Axiom_fund_name,
fund_type
from CTE_ef_ec d
join axiom_datalake.cl_snapshot.dim_axiomfund c on Axiom_fund_iqid=c.axiomfund_iqid
left join axiom_datalake.raw_snapshot.efront_coyinvest_ol a on d.f_fund=a.fund
LEFT JOIN axiom_datalake.raw_snapshot.efront_coy_ol b ON a.company = b.company
where fund_type='Blocker Vehicle' and investmentsiqid is not null
union
select 
f_soi_fund_name,
coalesce(b.f_of_prior_manager_funds_committed_to,a.f_of_prior_manager_funds_committed_to) as f_of_prior_manager_funds_committed_to,--akib
--f_of_prior_manager_funds_committed_to,
f_qr_recorded_fund_name,
--f_over_subscribed,
coalesce(b.f_over_subscribed,a.f_over_subscribed) as f_over_subscribed,
---f_next_gen,
coalesce(b.f_next_gen,a.f_next_gen) as f_next_gen, ---sunil
f_advisory_board_member,
f_investment_type,
f_ips_definition_realization_status,
f_deal_source,
f_description,
f_commitment_date,
f_exit_date,
f_co_investment_offered,
coalesce(b.investoraccountiqid,f_investoraccountiqid) as f_investoraccountiqid,
f_axiom_as_sponsor,
f_board_seat,
f_vintage_year_internal,
coalesce(fund,f_fund)  as f_fund,
coalesce(dim_fund_dcid,f_dc_iqid ) as f_dc_iqid,
f_investor_currency,
coalesce(fundiqid,f_fundiqid) as f_fundiqid,
deal_category,
c_vintage_year_internal,
c_stage_at_investment,
c_soi_fund_name,
c_qr_name,
c_co_investment_pool,
c_entry_valuation,
c_listing_status,
c_fund_ownership_at_entry,
c_management_ownership_at_entry,
f_fund_size_sec,
c_purchase_price,
c_size_of_purchased_commitment,
c_other_investors_at_entry,
c_investment_type,
c_ips_definition_realization_status,
c_company_description,
coalesce(output_str,c_fund) as c_fund,
c_companiesiqid,
c_dcid as c_dcid,
coalesce(b.company,a.company) as company,
coalesce(b.output_str,investments) as investments,
Axiom_fund_iqid,
Axiom_fund_name,
fund_type
from CTE_ef_ec  a join (SELECT 
    SUBSTRING(b.fund_full_name, CHARINDEX('(', b.fund_full_name) + 1, CHARINDEX(')', b.fund_full_name) - CHARINDEX('(', b.fund_full_name) - 1) as output_str,
    b.fund_full_name AS fund1,
    b.dim_fund_dcid,
    b.dim_fund_iqid,
    a.investorinfofundiqid,
    a.investoraccountiqid,
    a.company,a.fund,
    a.fundiqid,
	a.of_prior_manager_funds_committed_to as f_of_prior_manager_funds_committed_to,--akib
  a.next_gen  as f_next_gen, --sunil
	a.over_subscribed  as f_over_subscribed
 
FROM axiom_datalake.raw_snapshot.efront_fundinvacc_ol a
JOIN axiom_datalake.cl_snapshot.dim_fund b
    ON a.investorinfofundiqid = b.dim_fund_iqid
WHERE 
    SUBSTRING(b.fund_full_name, CHARINDEX('(', b.fund_full_name) + 1, CHARINDEX(')', b.fund_full_name) - CHARINDEX('(', b.fund_full_name) - 1) IN 
    (
        SELECT 
            SUBSTRING(fund, CHARINDEX('(', fund) + 1, CHARINDEX(')', fund) - CHARINDEX('(', fund) - 1) AS output_str 
        FROM 
            axiom_datalake.raw_snapshot.efront_fund_ol
        WHERE 
            fund_status = 'Blocker Vehicle'
    ) AND a.company IS NOT NULL) b  on a.f_fund=b.fund1 where Axiom_fund_iqid is not null)
,CTE_dcur as (select * from axiom_datalake.cl_snapshot.dim_currency)
,CTE_dc as  (select * from axiom_datalake.cl_snapshot.vw_dim_company)
,CTE_daf as (select * from axiom_datalake.cl_snapshot.dim_axiomfund)
,CTE_df as (select * from axiom_datalake.cl_snapshot.dim_fund)
,CTE_df_bv as (select * from axiom_datalake.cl_snapshot.dim_fund where fund_type='Blocker Vehicle' )
,CTE_affam as (SELECT d1.dim_axiomfund_id,d1.axiomfund_full_name FROM CTE_daf d1 WHERE d1.axiomfund_full_name IN (SELECT d2.axiomfundfamily_name  FROM CTE_daf d2 WHERE d2.axiomfundfamily_name IS NOT NULL ))

insert into axiom_datalake.cl_snapshot.fact_axiomfund_investment_test(fact_axiomfund_investment_id,
fact_axiomfund_investment_key,
dim_axiomfund_id,
dim_investment_currency_code ,
dim_investee_fund_id,
 dim_blocker_vehicle_id,
 dim_investee_company_id,
stage_at_investment,
advisory_board_member,
axiom_has_a_board_seat,
axiom_was_a_sponsor,
co_inv_offered,
deal_source,
entry_valuation,
exit_date,
fund_ownership_at_entry_percentage,
fund_size_sec,
initial_investment_date,
investment_description,
investment_type,
ips_definition_realization_status,
is_next_gen,
is_oversubscribed,
is_part_of_co_investment_pool,
listing_status_investment,
management_ownership_at_entry_percentage,
name_for_axiom_quarterly_reporting,
other_investors_at_entry_sec_direct,
--percentage_drawn_sec_direct,
prior_manager_funds_committed_to,
purchase_price_sec_direct,
size_of_purchased_commitment_sec_direct,
soi_name,
vintage_internal,
fact_axiomfund_investment_IQID,
deal_category,
deal_allocation_id,
investments)
select
ROW_NUMBER()OVER(ORDER BY coalesce(dim_axiomfund_id,dim_investee_fund_id))AS fact_axiomfund_investment_id,
coalesce(deal_allocation_id,MD5(CONCAT_WS('',dim_axiomfund_id,dim_investee_company_id,dim_investee_fund_id ,deal_category))) AS fact_axiomfund_investment_key,
*
from 
(select 
dim_axiomfund_id,
dim_investment_currency_code ,
dim_investee_fund_id,
 dim_blocker_vehicle_id,
 dim_investee_company_id,
stage_at_investment,
advisory_board_member,
axiom_has_a_board_seat,
axiom_was_a_sponsor,
co_inv_offered,
deal_source,
entry_valuation,
exit_date,
fund_ownership_at_entry_percentage,
fund_size_sec,
initial_investment_date,
investment_description,
investment_type,
ips_definition_realization_status,
is_next_gen,
is_oversubscribed,
is_part_of_co_investment_pool,
listing_status_investment,
management_ownership_at_entry_percentage,
name_for_axiom_quarterly_reporting,
other_investors_at_entry_sec_direct,
--percentage_drawn_sec_direct,
prior_manager_funds_committed_to,
purchase_price_sec_direct,
size_of_purchased_commitment_sec_direct,
soi_name,
vintage_internal,
fact_axiomfund_investment_IQID,
deal_category,
deal_allocation_id,
investments
From (
Select base.dim_axiomfund_id as dim_axiomfund_id,
dcur.dim_currency_id as dim_investment_currency_code ,
--NULL as dim_special_handling_masking,
df.dim_fund_id as dim_investee_fund_id,
df_bv.dim_fund_id as dim_blocker_vehicle_id,
base.dim_company_id as dim_investee_company_id,
c_stage_at_investment as stage_at_investment,
f_advisory_board_member as advisory_board_member,
f_board_seat as axiom_has_a_board_seat,
f_axiom_as_sponsor as axiom_was_a_sponsor,
coalesce(f_co_investment_offered,'N/A') as co_inv_offered,
f_deal_source as deal_source,
c_entry_valuation as entry_valuation,
f_exit_date as exit_date,
c_fund_ownership_at_entry as fund_ownership_at_entry_percentage,
f_fund_size_sec as fund_size_sec,
cast(f_commitment_date as date) as initial_investment_date,
coalesce(f_description,c_company_description) as investment_description,
coalesce(f_investment_type,c_investment_type) as investment_type,
coalesce(f_ips_definition_realization_status,c_ips_definition_realization_status,'N/A') as ips_definition_realization_status,
f_next_gen as is_next_gen,
f_over_subscribed as is_oversubscribed,
c_co_investment_pool as is_part_of_co_investment_pool,
c_listing_status as listing_status_investment,
c_management_ownership_at_entry as management_ownership_at_entry_percentage,
coalesce(NULL,c_qr_name) as name_for_axiom_quarterly_reporting,
c_other_investors_at_entry as other_investors_at_entry_sec_direct,
--c_drawn as percentage_drawn_sec_direct,
f_of_prior_manager_funds_committed_to  as prior_manager_funds_committed_to,
c_purchase_price as purchase_price_sec_direct,
c_size_of_purchased_commitment  as size_of_purchased_commitment_sec_direct,
coalesce(f_soi_fund_name  ,c_soi_fund_name  ) as soi_name,
coalesce(f_vintage_year_internal  ,c_vintage_year_internal  ) as  vintage_internal,
f_investoraccountiqid as fact_axiomfund_investment_IQID,
deal_category,
base.deal_dcid as deal_allocation_id,
investments
from (select * from CTE_final base 
left JOIN axiom_datalake.cl_snapshot.map_dealalloc_axiominvest m on m.investmentiqid=base.f_investoraccountiqid
left join CTE_dc dc on (dc.dim_company_iqid=base.c_companiesiqid or dc.company_name=base.company )
--left join CTE_daf daf on (daf.axiomfund_dcid=base.f_dc_iqid or daf.axiomfund_iqid=base.f_fundiqid) or daf.axiomfund_full_name=base.f_fund)  base
left join CTE_daf daf on (daf.axiomfund_iqid=base.Axiom_fund_iqid) or daf.axiomfund_full_name=base.Axiom_fund_name)  base
left join CTE_df df on ( df.dim_fund_iqid=base.f_fundiqid or df.fund_full_name=base.f_fund) 
left join CTE_df_bv df_bv on ( df_bv.dim_fund_iqid=base.f_fundiqid  or df_bv.fund_full_name=base.f_fund) 
left join CTE_dcur dcur on dcur.currency_short_desc =base.f_investor_currency
where dim_axiomfund_id is not null) a
union all
select 
dim_axiomfund_id,
dim_investment_currency_code ,
dim_investee_fund_id,
 dim_blocker_vehicle_id,
 dim_investee_company_id,
stage_at_investment,
advisory_board_member,
axiom_has_a_board_seat,
axiom_was_a_sponsor,
co_inv_offered,
deal_source,
entry_valuation,
exit_date,
fund_ownership_at_entry_percentage,
fund_size_sec,
initial_investment_date,
investment_description,
investment_type,
ips_definition_realization_status,
is_next_gen,
is_oversubscribed,
is_part_of_co_investment_pool,
listing_status_investment,
management_ownership_at_entry_percentage,
name_for_axiom_quarterly_reporting,
other_investors_at_entry_sec_direct,
--percentage_drawn_sec_direct,
prior_manager_funds_committed_to,
purchase_price_sec_direct,
size_of_purchased_commitment_sec_direct,
soi_name,
vintage_internal,
fact_axiomfund_investment_IQID,
deal_category,
deal_allocation_id,
investments
From(
select 
b.dim_axiomfund_id,
min(dim_investment_currency_code) as dim_investment_currency_code ,
--max(im_special_handling_masking) as im_special_handling_masking,
min(dim_investee_fund_id) as dim_investee_fund_id,
min(dim_blocker_vehicle_id) as dim_blocker_vehicle_id,
min(dim_investee_company_id) as dim_investee_company_id,
min(stage_at_investment) as stage_at_investment,
min(advisory_board_member) as advisory_board_member,
min(axiom_has_a_board_seat) as axiom_has_a_board_seat,
min(axiom_was_a_sponsor) as axiom_was_a_sponsor,
min(co_inv_offered) as co_inv_offered,
min(deal_source) as deal_source,
min(entry_valuation) as entry_valuation,
min(exit_date) as exit_date,
min(fund_ownership_at_entry_percentage) as fund_ownership_at_entry_percentage,
min(fund_size_sec) as fund_size_sec,
min(initial_investment_date) as initial_investment_date,
min(investment_description) as investment_description,
investment_type as investment_type,
min(ips_definition_realization_status) as ips_definition_realization_status,
min(is_next_gen) as is_next_gen,
min(is_oversubscribed) as is_oversubscribed,
min(is_part_of_co_investment_pool) as is_part_of_co_investment_pool,
min(listing_status_investment) as listing_status_investment,
min(management_ownership_at_entry_percentage) as management_ownership_at_entry_percentage,
min(name_for_axiom_quarterly_reporting) as name_for_axiom_quarterly_reporting,
min(other_investors_at_entry_sec_direct) as other_investors_at_entry_sec_direct,
--max(percentage_drawn_sec_direct) as percentage_drawn_sec_direct,
min(prior_manager_funds_committed_to) as prior_manager_funds_committed_to,
min(purchase_price_sec_direct) as purchase_price_sec_direct,
min(size_of_purchased_commitment_sec_direct) as size_of_purchased_commitment_sec_direct,
min(soi_name) as soi_name,
min(vintage_internal) as vintage_internal,
min(fact_axiomfund_investment_IQID) as fact_axiomfund_investment_IQID,
min(deal_category) as deal_category,
min(deal_allocation_id) as deal_allocation_id,
investments
From (
Select 
base.dim_axiomfund_id as dim_axiomfund_id,
dcur.dim_currency_id as dim_investment_currency_code ,
--NULL as dim_special_handling_masking,
df.dim_fund_id as dim_investee_fund_id,
df_bv.dim_fund_id as dim_blocker_vehicle_id,
base.dim_company_id as dim_investee_company_id,
c_stage_at_investment as stage_at_investment,
f_advisory_board_member as advisory_board_member,
f_board_seat as axiom_has_a_board_seat,
f_axiom_as_sponsor as axiom_was_a_sponsor,
coalesce(f_co_investment_offered,'N/A') as co_inv_offered,
f_deal_source as deal_source,
c_entry_valuation as entry_valuation,
f_exit_date as exit_date,
c_fund_ownership_at_entry as fund_ownership_at_entry_percentage,
f_fund_size_sec as fund_size_sec,
cast(f_commitment_date as date) as initial_investment_date,
coalesce(f_description,c_company_description) as investment_description,
coalesce(f_investment_type,c_investment_type) as investment_type,
coalesce(f_ips_definition_realization_status,c_ips_definition_realization_status,'N/A') as ips_definition_realization_status,
f_next_gen as is_next_gen,
f_over_subscribed as is_oversubscribed,
coalesce(c_co_investment_pool,'FALSE') as is_part_of_co_investment_pool,
c_listing_status as listing_status_investment,
c_management_ownership_at_entry as management_ownership_at_entry_percentage,
coalesce(NULL,c_qr_name) as name_for_axiom_quarterly_reporting,
c_other_investors_at_entry as other_investors_at_entry_sec_direct,
--c_drawn as percentage_drawn_sec_direct,
f_of_prior_manager_funds_committed_to  as prior_manager_funds_committed_to,
c_purchase_price as purchase_price_sec_direct,
c_size_of_purchased_commitment  as size_of_purchased_commitment_sec_direct,
coalesce(f_soi_fund_name  ,c_soi_fund_name  ) as soi_name,
coalesce(f_vintage_year_internal  ,c_vintage_year_internal  ) as  vintage_internal,
f_investoraccountiqid as fact_axiomfund_investment_IQID,
deal_category,
base.deal_dcid as deal_allocation_id,
investments,base.axiomfundfamily_name
from (select * from CTE_final base 
left JOIN axiom_datalake.cl_snapshot.map_dealalloc_axiominvest m on m.investmentiqid=base.f_investoraccountiqid
left join CTE_dc dc on (dc.dim_company_iqid=base.c_companiesiqid or dc.company_name=base.company )
left join CTE_daf daf on (daf.axiomfund_iqid=base.Axiom_fund_iqid or daf.axiomfund_full_name=base.Axiom_fund_name))  base
left join CTE_df df on ( df.dim_fund_iqid=base.f_fundiqid or df.fund_full_name=base.f_fund) 
left join CTE_df_bv df_bv on (df_bv.dim_fund_iqid=base.f_fundiqid  or df_bv.fund_full_name=base.f_fund) 
left join CTE_dcur dcur on dcur.currency_short_desc =base.f_investor_currency
where dim_axiomfund_id is not null 
and base.dim_axiomfund_id in (Select dim_axiomfund_id from CTE_daf where axiomfundfamily_name is not null)) a
join CTE_affam b on a.axiomfundfamily_name=b.axiomfund_full_name
group by b.dim_axiomfund_id,
investment_type,
investments)a )b 


In [0]:
%sql
with CTE_ef_ec as (SELECT 
   b.soi_fund_name                       as f_soi_fund_name                 
	,a.of_prior_manager_funds_committed_to as f_of_prior_manager_funds_committed_to 
	,b.qr_recorded_fund_name  as f_qr_recorded_fund_name
	,a.over_subscribed  as f_over_subscribed
	,a.next_gen  as f_next_gen
	,a.advisory_board_member  as f_advisory_board_member
	,a.investment_type  as f_investment_type
	,b.ips_definition_realization_status  as f_ips_definition_realization_status
	,a.deal_source  as f_deal_source
	,b.description           as f_description
	,a.commitment_date_portfolio_summary  as f_commitment_date
	--,a.exit_date  as f_exit_date
	,cast(from_unixtime(unix_timestamp(a.exit_date, 'dd/MM/yyyy'), 'yyyy-MM-dd') as date) as f_exit_date
	,a.co_investment_offered as f_co_investment_offered 
	,a.investoraccountiqid  as f_investoraccountiqid
	,a.axiom_as_sponsor  as f_axiom_as_sponsor
	,a.board_seat  as f_board_seat
	,a.vintage_year_internal     as f_vintage_year_internal
	,a.fund  as f_fund
	,b.dc_iqid  as f_dc_iqid 
	,a.investor_currency  as f_investor_currency
	,b.fundiqid as f_fundiqid
	,case when lower(a.investment_type)=lower('Primary mainline') or lower(a.investment_type)=lower('Primary toehold') then 'primary'         when lower(a.investment_type)='secondary direct' or lower(a.investment_type)='secondary' then 'secondary'                                 when lower(a.investment_type)='co-investment'  then 'co-investment' else 'others' end as deal_category
  ,NULL AS c_vintage_year_internal
	,NULL AS c_stage_at_investment
	,NULL AS c_soi_fund_name
	,b.qr_recorded_fund_name AS c_qr_name
	,NULL AS c_co_investment_pool
	,NULL AS c_entry_valuation
	,NULL AS c_listing_status
	,NULL AS c_fund_ownership_at_entry
	,NULL AS c_management_ownership_at_entry
	,b.fund_size AS f_fund_size_sec
	--,NULL AS c_drawn
	,NULL AS c_purchase_price
	,b.size_of_purchased_commitment c_size_of_purchased_commitment
	,NULL AS c_other_investors_at_entry
	,NULL AS c_investment_type
	,NULL AS c_ips_definition_realization_status
	,NULL AS c_company_description
	,NULL AS c_fund
	,NULL AS c_companiesiqid
	,NULL AS c_dcid,a.company
	,a.fund as investments
	,a.investorinfofundiqid as Axiom_fund_iqid
	,NULL as Axiom_fund_name
  ,b.fund_status as fund_type
FROM axiom_datalake.raw_snapshot.efront_fundinvacc_ol a
LEFT JOIN axiom_datalake.raw_snapshot.efront_fund_ol b ON a.fundiqid = b.fundiqid
UNION 
  SELECT 
   NULL as f_soi_fund_name
	,NULL as f_of_prior_manager_funds_committed_to
	,NULL as f_qr_recorded_fund_name
	,NULL as f_over_subscribed
	,NULL as f_next_gen
	,NULL as f_advisory_board_member
	,NULL as f_investment_type
	,NULL as f_ips_definition_realization_status
	,NULL as f_deal_source
	,NULL as f_description
	,coalesce(cast(axiom_commitment_date as date),cast(from_unixtime(unix_timestamp(a.axiom_commitment_date, 'dd/MM/yyyy'), 'yyyy-MM-dd') as date)) as f_commitment_date
	--,a.exit_date as f_exit_date
	,cast(from_unixtime(unix_timestamp(a.exit_date, 'dd/MM/yyyy'), 'yyyy-MM-dd') as date) as f_exit_date
	,NULL as f_co_investment_offered
	,a.investmentsiqid  as f_investoraccountiqid
	,NULL as f_axiom_as_sponsor
	,NULL as f_board_seat
	,NULL as f_vintage_year_internal
	,a.fund as f_fund
	,NULL as f_dc_iqid 
	,investor_currency as f_investor_currency
	,NULL as f_fundiqid
	,case when lower(a.investment_type)=lower('Primary mainline') or lower(a.investment_type)=lower('Primary toehold') then 'primary'         
	when lower(a.investment_type)='secondary direct' or lower(a.investment_type)='secondary' then 'secondary'                                 
	when lower(a.investment_type)='co-investment'  then 'co-investment' else 'others' end as deal_category
  ,a.vintage_year_internal               AS c_vintage_year_internal
	,a.stage_at_investment AS c_stage_at_investment
	,b.soi_fund_name AS c_soi_fund_name
	,b.qr_name AS c_qr_name
	,a.co_investment_pool AS c_co_investment_pool
	,a.entry_valuation AS c_entry_valuation
	,a.listing_status AS c_listing_status
	,a.fund_ownership_at_entry AS c_fund_ownership_at_entry
	,a.management_ownership_at_entry AS c_management_ownership_at_entry
	,NULL AS f_fund_size_sec
	--,a.drawn AS c_drawn
	,a.purchase_price AS c_purchase_price
	,a.size_of_purchased_commitment AS c_size_of_purchased_commitment
	,a.other_investors_at_entry AS c_other_investors_at_entry
	,a.investment_type AS c_investment_type
	,a.ips_definition_realization_status AS c_ips_definition_realization_status
	,b.company_description AS c_company_description
		,a.fund AS c_fund
	,b.companiesiqid AS c_companiesiqid
	,b.dcid AS c_dcid,b.company
  ,a.company as investments
   	,NULL as Axiom_fund_iqid
	,c_fund as Axiom_fund_name
  ,NULL as fund_type
FROM axiom_datalake.raw_snapshot.efront_coyinvest_ol a
LEFT JOIN axiom_datalake.raw_snapshot.efront_coy_ol b ON a.company = b.company)

,CTE_final as(
select 
f_soi_fund_name,
f_of_prior_manager_funds_committed_to,
f_qr_recorded_fund_name,
f_over_subscribed,
f_next_gen,
f_advisory_board_member,
f_investment_type,
f_ips_definition_realization_status,
f_deal_source,
f_description,
f_commitment_date,
f_exit_date,
f_co_investment_offered,
f_investoraccountiqid as f_investoraccountiqid,
f_axiom_as_sponsor,
f_board_seat,
f_vintage_year_internal,
f_fund,
f_dc_iqid,
f_investor_currency,
f_fundiqid,
deal_category,
c_vintage_year_internal,
c_stage_at_investment,
c_soi_fund_name,
c_qr_name,
c_co_investment_pool,
c_entry_valuation,
c_listing_status,
c_fund_ownership_at_entry,
c_management_ownership_at_entry,
f_fund_size_sec,
c_purchase_price,
c_size_of_purchased_commitment,
c_other_investors_at_entry,
c_investment_type,
c_ips_definition_realization_status,
c_company_description,
c_fund,
c_companiesiqid,
c_dcid,
company as company,
investments as investments,
Axiom_fund_iqid,
Axiom_fund_name,
fund_type
from CTE_ef_ec a --where (fund_type != 'Blocker Vehicle'or fund_type is null)
union
select 
f_soi_fund_name,
f_of_prior_manager_funds_committed_to,
f_qr_recorded_fund_name,
f_over_subscribed,
f_next_gen,
f_advisory_board_member,
coalesce(a.investment_type,f_investment_type) as f_investment_type,
f_ips_definition_realization_status,
f_deal_source,
f_description,
f_commitment_date,
f_exit_date,
f_co_investment_offered,
coalesce(investmentsiqid,f_investoraccountiqid) as f_investoraccountiqid,
f_axiom_as_sponsor,
f_board_seat,
f_vintage_year_internal,
NULL as f_fund,
NULL as f_dc_iqid,
investor_currency as f_investor_currency,
NULL as f_fundiqid,
case when lower(a.investment_type)=lower('Primary mainline') or lower(a.investment_type)=lower('Primary toehold') then 'primary'         
	when lower(a.investment_type)='secondary direct' or lower(a.investment_type)='secondary' then 'secondary'                                 
	when lower(a.investment_type)='co-investment'  then 'co-investment' else 'others' end as deal_category,
a.vintage_year_internal AS c_vintage_year_internal,
a.stage_at_investment AS c_stage_at_investment,
b.soi_fund_name AS c_soi_fund_name,
b.qr_name AS c_qr_name,
a.co_investment_pool AS c_co_investment_pool,
a.entry_valuation AS c_entry_valuation,
a.listing_status AS c_listing_status,
a.fund_ownership_at_entry AS c_fund_ownership_at_entry,
a.management_ownership_at_entry AS c_management_ownership_at_entry,
NULL AS f_fund_size_sec,
--,a.drawn AS c_drawn
a.purchase_price AS c_purchase_price,
a.size_of_purchased_commitment AS c_size_of_purchased_commitment,
a.other_investors_at_entry AS c_other_investors_at_entry,
a.investment_type AS c_investment_type,
a.ips_definition_realization_status AS c_ips_definition_realization_status,
b.company_description AS c_company_description,
b.dcid AS c_dcid,
NULL as c_fund,
coalesce(a.companyiqid,c_companiesiqid ) as c_companiesiqid,
coalesce(a.company,a.company) as company,
coalesce(a.company,investments) as investments,
c.axiomfund_iqid as Axiom_fund_iqid,
Axiom_fund_name,
fund_type
from CTE_ef_ec d
join axiom_datalake.cl_snapshot.dim_axiomfund c on Axiom_fund_iqid=c.axiomfund_iqid
left join axiom_datalake.raw_snapshot.efront_coyinvest_ol a on d.f_fund=a.fund
LEFT JOIN axiom_datalake.raw_snapshot.efront_coy_ol b ON a.company = b.company
where fund_type='Blocker Vehicle' and investmentsiqid is not null
union
select 
f_soi_fund_name,
f_of_prior_manager_funds_committed_to,
f_qr_recorded_fund_name,
f_over_subscribed,
f_next_gen,
f_advisory_board_member,
coalesce(a.investment_type,f_investment_type) as f_investment_type,
f_ips_definition_realization_status,
f_deal_source,
f_description,
f_commitment_date,
f_exit_date,
f_co_investment_offered,
coalesce(investmentsiqid,f_investoraccountiqid) as f_investoraccountiqid,
f_axiom_as_sponsor,
f_board_seat,
f_vintage_year_internal,
NULL as f_fund,
NULL as f_dc_iqid,
investor_currency as f_investor_currency,
NULL as f_fundiqid,
case when lower(a.investment_type)=lower('Primary mainline') or lower(a.investment_type)=lower('Primary toehold') then 'primary'         
	when lower(a.investment_type)='secondary direct' or lower(a.investment_type)='secondary' then 'secondary'                                 
	when lower(a.investment_type)='co-investment'  then 'co-investment' else 'others' end as deal_category,
a.vintage_year_internal AS c_vintage_year_internal,
a.stage_at_investment AS c_stage_at_investment,
b.soi_fund_name AS c_soi_fund_name,
b.qr_name AS c_qr_name,
a.co_investment_pool AS c_co_investment_pool,
a.entry_valuation AS c_entry_valuation,
a.listing_status AS c_listing_status,
a.fund_ownership_at_entry AS c_fund_ownership_at_entry,
a.management_ownership_at_entry AS c_management_ownership_at_entry,
NULL AS f_fund_size_sec,
--,a.drawn AS c_drawn
a.purchase_price AS c_purchase_price,
a.size_of_purchased_commitment AS c_size_of_purchased_commitment,
a.other_investors_at_entry AS c_other_investors_at_entry,
a.investment_type AS c_investment_type,
a.ips_definition_realization_status AS c_ips_definition_realization_status,
b.company_description AS c_company_description,
b.dcid AS c_dcid,
NULL as c_fund,
coalesce(a.companyiqid,c_companiesiqid ) as c_companiesiqid,
coalesce(a.company,a.company) as company,
coalesce(a.company,investments) as investments,
c.axiomfund_iqid as Axiom_fund_iqid,
Axiom_fund_name,
fund_type
from CTE_ef_ec d
join axiom_datalake.cl_snapshot.dim_axiomfund c on Axiom_fund_iqid=c.axiomfund_iqid
left join axiom_datalake.raw_snapshot.efront_coyinvest_ol a on d.f_fund=a.fund
LEFT JOIN axiom_datalake.raw_snapshot.efront_coy_ol b ON a.company = b.company
where fund_type='Blocker Vehicle' and investmentsiqid is not null
union
select 
f_soi_fund_name,
f_of_prior_manager_funds_committed_to,
f_qr_recorded_fund_name,
f_over_subscribed,
f_next_gen,
f_advisory_board_member,
f_investment_type,
f_ips_definition_realization_status,
f_deal_source,
f_description,
f_commitment_date,
f_exit_date,
f_co_investment_offered,
coalesce(b.investoraccountiqid,f_investoraccountiqid) as f_investoraccountiqid,
f_axiom_as_sponsor,
f_board_seat,
f_vintage_year_internal,
coalesce(fund,f_fund)  as f_fund,
coalesce(dim_fund_dcid,f_dc_iqid ) as f_dc_iqid,
f_investor_currency,
coalesce(fundiqid,f_fundiqid) as f_fundiqid,
deal_category,
c_vintage_year_internal,
c_stage_at_investment,
c_soi_fund_name,
c_qr_name,
c_co_investment_pool,
c_entry_valuation,
c_listing_status,
c_fund_ownership_at_entry,
c_management_ownership_at_entry,
f_fund_size_sec,
c_purchase_price,
c_size_of_purchased_commitment,
c_other_investors_at_entry,
c_investment_type,
c_ips_definition_realization_status,
c_company_description,
coalesce(output_str,c_fund) as c_fund,
c_companiesiqid,
c_dcid as c_dcid,
coalesce(b.company,a.company) as company,
coalesce(b.output_str,investments) as investments,
Axiom_fund_iqid,
Axiom_fund_name,
fund_type
from CTE_ef_ec  a join (SELECT 
    SUBSTRING(b.fund_full_name, CHARINDEX('(', b.fund_full_name) + 1, CHARINDEX(')', b.fund_full_name) - CHARINDEX('(', b.fund_full_name) - 1) as output_str,
    b.fund_full_name AS fund1,
    b.dim_fund_dcid,
    b.dim_fund_iqid,
    a.investorinfofundiqid,
    a.investoraccountiqid,
    a.company,a.fund,
    a.fundiqid
FROM axiom_datalake.raw_snapshot.efront_fundinvacc_ol a
JOIN axiom_datalake.cl_snapshot.dim_fund b
    ON a.investorinfofundiqid = b.dim_fund_iqid
WHERE 
    SUBSTRING(b.fund_full_name, CHARINDEX('(', b.fund_full_name) + 1, CHARINDEX(')', b.fund_full_name) - CHARINDEX('(', b.fund_full_name) - 1) IN 
    (
        SELECT 
            SUBSTRING(fund, CHARINDEX('(', fund) + 1, CHARINDEX(')', fund) - CHARINDEX('(', fund) - 1) AS output_str 
        FROM 
            axiom_datalake.raw_snapshot.efront_fund_ol
        WHERE 
            fund_status = 'Blocker Vehicle'
    ) AND a.company IS NOT NULL) b  on a.f_fund=b.fund1 where Axiom_fund_iqid is not null)
,CTE_dcur as (select * from axiom_datalake.cl_snapshot.dim_currency)
,CTE_dc as  (select * from axiom_datalake.cl_snapshot.vw_dim_company)
,CTE_daf as (select * from axiom_datalake.cl_snapshot.dim_axiomfund)
,CTE_df as (select * from axiom_datalake.cl_snapshot.dim_fund)
,CTE_df_bv as (select * from axiom_datalake.cl_snapshot.dim_fund where fund_type='Blocker Vehicle' )
,CTE_affam as (SELECT d1.dim_axiomfund_id,d1.axiomfund_full_name FROM CTE_daf d1 WHERE d1.axiomfund_full_name IN (SELECT d2.axiomfundfamily_name  FROM CTE_daf d2 WHERE d2.axiomfundfamily_name IS NOT NULL ))

insert into axiom_datalake.cl_snapshot.fact_axiomfund_investment(fact_axiomfund_investment_id,
fact_axiomfund_investment_key,
dim_axiomfund_id,
dim_investment_currency_code ,
dim_investee_fund_id,
 dim_blocker_vehicle_id,
 dim_investee_company_id,
stage_at_investment,
advisory_board_member,
axiom_has_a_board_seat,
axiom_was_a_sponsor,
co_inv_offered,
deal_source,
entry_valuation,
exit_date,
fund_ownership_at_entry_percentage,
fund_size_sec,
initial_investment_date,
investment_description,
investment_type,
ips_definition_realization_status,
is_next_gen,
is_oversubscribed,
is_part_of_co_investment_pool,
listing_status_investment,
management_ownership_at_entry_percentage,
name_for_axiom_quarterly_reporting,
other_investors_at_entry_sec_direct,
--percentage_drawn_sec_direct,
prior_manager_funds_committed_to,
purchase_price_sec_direct,
size_of_purchased_commitment_sec_direct,
soi_name,
vintage_internal,
fact_axiomfund_investment_IQID,
deal_category,
deal_allocation_id,
investments)
select
ROW_NUMBER()OVER(ORDER BY coalesce(dim_axiomfund_id,dim_investee_fund_id))AS fact_axiomfund_investment_id,
coalesce(deal_allocation_id,MD5(CONCAT_WS('',dim_axiomfund_id,dim_investee_company_id,dim_investee_fund_id ,deal_category))) AS fact_axiomfund_investment_key,
*
from 
(select 
dim_axiomfund_id,
dim_investment_currency_code ,
dim_investee_fund_id,
 dim_blocker_vehicle_id,
 dim_investee_company_id,
stage_at_investment,
advisory_board_member,
axiom_has_a_board_seat,
axiom_was_a_sponsor,
co_inv_offered,
deal_source,
entry_valuation,
exit_date,
fund_ownership_at_entry_percentage,
fund_size_sec,
initial_investment_date,
investment_description,
investment_type,
ips_definition_realization_status,
is_next_gen,
is_oversubscribed,
is_part_of_co_investment_pool,
listing_status_investment,
management_ownership_at_entry_percentage,
name_for_axiom_quarterly_reporting,
other_investors_at_entry_sec_direct,
--percentage_drawn_sec_direct,
prior_manager_funds_committed_to,
purchase_price_sec_direct,
size_of_purchased_commitment_sec_direct,
soi_name,
vintage_internal,
fact_axiomfund_investment_IQID,
deal_category,
deal_allocation_id,
investments
From (
Select base.dim_axiomfund_id as dim_axiomfund_id,
dcur.dim_currency_id as dim_investment_currency_code ,
--NULL as dim_special_handling_masking,
df.dim_fund_id as dim_investee_fund_id,
df_bv.dim_fund_id as dim_blocker_vehicle_id,
base.dim_company_id as dim_investee_company_id,
c_stage_at_investment as stage_at_investment,
f_advisory_board_member as advisory_board_member,
f_board_seat as axiom_has_a_board_seat,
f_axiom_as_sponsor as axiom_was_a_sponsor,
coalesce(f_co_investment_offered,'N/A') as co_inv_offered,
f_deal_source as deal_source,
c_entry_valuation as entry_valuation,
f_exit_date as exit_date,
c_fund_ownership_at_entry as fund_ownership_at_entry_percentage,
f_fund_size_sec as fund_size_sec,
cast(f_commitment_date as date) as initial_investment_date,
coalesce(f_description,c_company_description) as investment_description,
coalesce(f_investment_type,c_investment_type) as investment_type,
coalesce(f_ips_definition_realization_status,c_ips_definition_realization_status,'N/A') as ips_definition_realization_status,
f_next_gen as is_next_gen,
f_over_subscribed as is_oversubscribed,
c_co_investment_pool as is_part_of_co_investment_pool,
c_listing_status as listing_status_investment,
c_management_ownership_at_entry as management_ownership_at_entry_percentage,
coalesce(NULL,c_qr_name) as name_for_axiom_quarterly_reporting,
c_other_investors_at_entry as other_investors_at_entry_sec_direct,
--c_drawn as percentage_drawn_sec_direct,
f_of_prior_manager_funds_committed_to  as prior_manager_funds_committed_to,
c_purchase_price as purchase_price_sec_direct,
c_size_of_purchased_commitment  as size_of_purchased_commitment_sec_direct,
coalesce(f_soi_fund_name  ,c_soi_fund_name  ) as soi_name,
coalesce(f_vintage_year_internal  ,c_vintage_year_internal  ) as  vintage_internal,
f_investoraccountiqid as fact_axiomfund_investment_IQID,
deal_category,
base.deal_dcid as deal_allocation_id,
investments
from (select * from CTE_final base 
left JOIN axiom_datalake.cl_snapshot.map_dealalloc_axiominvest m on m.investmentiqid=base.f_investoraccountiqid
left join CTE_dc dc on (dc.dim_company_iqid=base.c_companiesiqid or dc.company_name=base.company )
--left join CTE_daf daf on (daf.axiomfund_dcid=base.f_dc_iqid or daf.axiomfund_iqid=base.f_fundiqid) or daf.axiomfund_full_name=base.f_fund)  base
left join CTE_daf daf on (daf.axiomfund_iqid=base.Axiom_fund_iqid) or daf.axiomfund_full_name=base.Axiom_fund_name)  base
left join CTE_df df on ( df.dim_fund_iqid=base.f_fundiqid or df.fund_full_name=base.f_fund) 
left join CTE_df_bv df_bv on ( df_bv.dim_fund_iqid=base.f_fundiqid  or df_bv.fund_full_name=base.f_fund) 
left join CTE_dcur dcur on dcur.currency_short_desc =base.f_investor_currency
where dim_axiomfund_id is not null) a
union all
select 
dim_axiomfund_id,
dim_investment_currency_code ,
dim_investee_fund_id,
 dim_blocker_vehicle_id,
 dim_investee_company_id,
stage_at_investment,
advisory_board_member,
axiom_has_a_board_seat,
axiom_was_a_sponsor,
co_inv_offered,
deal_source,
entry_valuation,
exit_date,
fund_ownership_at_entry_percentage,
fund_size_sec,
initial_investment_date,
investment_description,
investment_type,
ips_definition_realization_status,
is_next_gen,
is_oversubscribed,
is_part_of_co_investment_pool,
listing_status_investment,
management_ownership_at_entry_percentage,
name_for_axiom_quarterly_reporting,
other_investors_at_entry_sec_direct,
--percentage_drawn_sec_direct,
prior_manager_funds_committed_to,
purchase_price_sec_direct,
size_of_purchased_commitment_sec_direct,
soi_name,
vintage_internal,
fact_axiomfund_investment_IQID,
deal_category,
deal_allocation_id,
investments
From(
select 
b.dim_axiomfund_id,
min(dim_investment_currency_code) as dim_investment_currency_code ,
--max(im_special_handling_masking) as im_special_handling_masking,
min(dim_investee_fund_id) as dim_investee_fund_id,
min(dim_blocker_vehicle_id) as dim_blocker_vehicle_id,
min(dim_investee_company_id) as dim_investee_company_id,
min(stage_at_investment) as stage_at_investment,
min(advisory_board_member) as advisory_board_member,
min(axiom_has_a_board_seat) as axiom_has_a_board_seat,
min(axiom_was_a_sponsor) as axiom_was_a_sponsor,
min(co_inv_offered) as co_inv_offered,
min(deal_source) as deal_source,
min(entry_valuation) as entry_valuation,
min(exit_date) as exit_date,
min(fund_ownership_at_entry_percentage) as fund_ownership_at_entry_percentage,
min(fund_size_sec) as fund_size_sec,
min(initial_investment_date) as initial_investment_date,
min(investment_description) as investment_description,
investment_type as investment_type,
min(ips_definition_realization_status) as ips_definition_realization_status,
min(is_next_gen) as is_next_gen,
min(is_oversubscribed) as is_oversubscribed,
min(is_part_of_co_investment_pool) as is_part_of_co_investment_pool,
min(listing_status_investment) as listing_status_investment,
min(management_ownership_at_entry_percentage) as management_ownership_at_entry_percentage,
min(name_for_axiom_quarterly_reporting) as name_for_axiom_quarterly_reporting,
min(other_investors_at_entry_sec_direct) as other_investors_at_entry_sec_direct,
--max(percentage_drawn_sec_direct) as percentage_drawn_sec_direct,
min(prior_manager_funds_committed_to) as prior_manager_funds_committed_to,
min(purchase_price_sec_direct) as purchase_price_sec_direct,
min(size_of_purchased_commitment_sec_direct) as size_of_purchased_commitment_sec_direct,
min(soi_name) as soi_name,
min(vintage_internal) as vintage_internal,
min(fact_axiomfund_investment_IQID) as fact_axiomfund_investment_IQID,
min(deal_category) as deal_category,
min(deal_allocation_id) as deal_allocation_id,
investments
From (
Select 
base.dim_axiomfund_id as dim_axiomfund_id,
dcur.dim_currency_id as dim_investment_currency_code ,
--NULL as dim_special_handling_masking,
df.dim_fund_id as dim_investee_fund_id,
df_bv.dim_fund_id as dim_blocker_vehicle_id,
base.dim_company_id as dim_investee_company_id,
c_stage_at_investment as stage_at_investment,
f_advisory_board_member as advisory_board_member,
f_board_seat as axiom_has_a_board_seat,
f_axiom_as_sponsor as axiom_was_a_sponsor,
coalesce(f_co_investment_offered,'N/A') as co_inv_offered,
f_deal_source as deal_source,
c_entry_valuation as entry_valuation,
f_exit_date as exit_date,
c_fund_ownership_at_entry as fund_ownership_at_entry_percentage,
f_fund_size_sec as fund_size_sec,
cast(f_commitment_date as date) as initial_investment_date,
coalesce(f_description,c_company_description) as investment_description,
coalesce(f_investment_type,c_investment_type) as investment_type,
coalesce(f_ips_definition_realization_status,c_ips_definition_realization_status,'N/A') as ips_definition_realization_status,
f_next_gen as is_next_gen,
f_over_subscribed as is_oversubscribed,
coalesce(c_co_investment_pool,'FALSE') as is_part_of_co_investment_pool,
c_listing_status as listing_status_investment,
c_management_ownership_at_entry as management_ownership_at_entry_percentage,
coalesce(NULL,c_qr_name) as name_for_axiom_quarterly_reporting,
c_other_investors_at_entry as other_investors_at_entry_sec_direct,
--c_drawn as percentage_drawn_sec_direct,
f_of_prior_manager_funds_committed_to  as prior_manager_funds_committed_to,
c_purchase_price as purchase_price_sec_direct,
c_size_of_purchased_commitment  as size_of_purchased_commitment_sec_direct,
coalesce(f_soi_fund_name  ,c_soi_fund_name  ) as soi_name,
coalesce(f_vintage_year_internal  ,c_vintage_year_internal  ) as  vintage_internal,
f_investoraccountiqid as fact_axiomfund_investment_IQID,
deal_category,
base.deal_dcid as deal_allocation_id,
investments,base.axiomfundfamily_name
from (select * from CTE_final base 
left JOIN axiom_datalake.cl_snapshot.map_dealalloc_axiominvest m on m.investmentiqid=base.f_investoraccountiqid
left join CTE_dc dc on (dc.dim_company_iqid=base.c_companiesiqid or dc.company_name=base.company )
left join CTE_daf daf on (daf.axiomfund_iqid=base.Axiom_fund_iqid or daf.axiomfund_full_name=base.Axiom_fund_name))  base
left join CTE_df df on ( df.dim_fund_iqid=base.f_fundiqid or df.fund_full_name=base.f_fund) 
left join CTE_df_bv df_bv on (df_bv.dim_fund_iqid=base.f_fundiqid  or df_bv.fund_full_name=base.f_fund) 
left join CTE_dcur dcur on dcur.currency_short_desc =base.f_investor_currency
where dim_axiomfund_id is not null 
and base.dim_axiomfund_id in (Select dim_axiomfund_id from CTE_daf where axiomfundfamily_name is not null)) a
join CTE_affam b on a.axiomfundfamily_name=b.axiomfund_full_name
group by b.dim_axiomfund_id,
investment_type,
investments)a )b 


In [0]:
%sql
WITH CTE_da AS (
		SELECT *
		FROM ( SELECT fact_deal_allocation_key 	,currency_of_commitment_amount 	,date_of_exit 	,fact_deal_id 	,date_of_commitment 	,CASE  		WHEN CHARINDEX('JPY', original_commitment_sec_split) = 0 			THEN original_commitment_sec_split 		ELSE SUBSTR(original_commitment_sec_split, 1, LENGTH(original_commitment_sec_split) - 3) 		END AS original_commitment_sec_split 	,axiom_commitment 	,axiom_commitment_lcy 	,CASE  		WHEN CHARINDEX('JPY', purchase_price_sec_split) = 0 			THEN purchase_price_sec_split 		ELSE SUBSTR(purchase_price_sec_split, 1, LENGTH(purchase_price_sec_split) - 3) 		END AS purchase_price_sec_split 	,discount_perc_to_nav 	,ROW_NUMBER() OVER ( 		PARTITION BY fact_deal_allocation_key ORDER BY date_of_commitment 		) rn FROM axiom_datalake.cl_snapshot.fact_deal_allocation ) a
		WHERE rn = 1
		)
	,CTE_af_inv AS (
		SELECT DISTINCT investment_iqid ,c.currency_short_desc
		FROM axiom_datalake.cl_snapshot.fact_axiomfund_investment_test i
		LEFT JOIN axiom_datalake.cl_snapshot.fact_axiomfund_investment_transaction_gn t ON t.axiom_fund_investment_id = i.fact_axiomfund_investment_id
		LEFT JOIN axiom_datalake.cl_snapshot.dim_currency c ON t.investment_currency_code = c.dim_currency_id
		WHERE t.transaction_date <= '2024-03-31' --2024-03-31,2023-09-30,2023-12-31
		)
	,CTE_affam AS (
		SELECT *
		FROM axiom_datalake.cl_snapshot.dim_axiomfund d1
		WHERE d1.axiomfund_full_name IN ( 	SELECT d2.axiomfundfamily_name 	FROM axiom_datalake.cl_snapshot.dim_axiomfund d2 	WHERE d2.axiomfundfamily_name IS NOT NULL 	)
		)
	,CTE_sp_combine_inv_type AS (
		SELECT condition_investments AS investments_sp ,revised_investment_type AS investment_type_sp
		FROM axiom_datalake.cl_snapshot.master_ddq_sh
		WHERE ( 	revised_investments 	,revised_investment_type 	) IN ( 	SELECT revised_investments 		,revised_investment_type 	FROM axiom_datalake.cl_snapshot.master_ddq_sh 	WHERE revised_investment_type IS NOT NULL 	GROUP BY revised_investments 		,revised_investment_type 	HAVING count(1) > 1 	)
		)
	,CTE_deal AS (
		SELECT deal.* ,gp.gp_name
		FROM ( SELECT a.fact_deal_id 	,comp.company_name AS company_name_deal 	,fnd.fund_full_name AS fund_full_name_deal 	,CASE  		WHEN deal.co_investment_spv_portfolio_company_name_legal_name IS NULL 			THEN lead(deal.co_investment_spv_portfolio_company_name_legal_name) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.co_investment_spv_portfolio_company_name_legal_name 		END AS co_investment_spv_portfolio_company_name_legal_name 	,CASE  		WHEN deal.deal_type IS NULL 			THEN lead(deal.deal_type) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.deal_type 		END AS deal_type 	,CASE  		WHEN deal.name_of_board_member IS NULL 			THEN lead(deal.name_of_board_member) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.name_of_board_member 		END AS name_of_board_member 	,CASE  		WHEN deal.axiom_deal_lead IS NULL 			THEN lead(deal.axiom_deal_lead) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.axiom_deal_lead 		END AS axiom_deal_lead 	,CASE  		WHEN deal.is_axiom_a_sponsor IS NULL 			THEN lead(deal.is_axiom_a_sponsor) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.is_axiom_a_sponsor 		END AS is_axiom_a_sponsor 	,CASE  		WHEN deal.buyer_transaction_price_discount_to_nav IS NULL 			THEN lead(deal.buyer_transaction_price_discount_to_nav) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.buyer_transaction_price_discount_to_nav 		END AS buyer_transaction_price_discount_to_nav 	,deal.axioms_net_purchase_price_on_funding_date AS axioms_net_purchase_price_on_funding_date 	,CASE  		WHEN deal.name_for_axiom_quarterly_reporting IS NULL 			THEN lead(deal.name_for_axiom_quarterly_reporting) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.name_for_axiom_quarterly_reporting 		END AS name_for_axiom_quarterly_reporting,CASE  		WHEN deal.is_the_fund_over_subscribed IS NULL 			THEN lead(deal.is_the_fund_over_subscribed) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.is_the_fund_over_subscribed 		END AS is_the_fund_over_subscribed ,CASE  		WHEN deal.coinvestment_offered IS NULL 			THEN lead(deal.coinvestment_offered) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.coinvestment_offered 		END AS coinvestment_offered 	,CASE  		WHEN deal.next_gen_manager IS NULL 			THEN lead(deal.next_gen_manager) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.next_gen_manager 		END AS next_gen_manager 	,CASE  		WHEN deal.does_axiom_have_board_seat IS NULL 			THEN lead(deal.does_axiom_have_board_seat) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.does_axiom_have_board_seat 		END AS does_axiom_have_board_seat 	,CASE  		WHEN deal.num_of_prior_funds_axiom_committed_in_gp IS NULL 			THEN lead(deal.num_of_prior_funds_axiom_committed_in_gp) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.num_of_prior_funds_axiom_committed_in_gp 		END AS num_of_prior_funds_axiom_committed_in_gp 	,CASE  		WHEN deal.general_partner IS NULL 			THEN lead(deal.general_partner) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.general_partner 		END general_partner 	,CASE  		WHEN deal.soi_name_finance IS NULL 			THEN lead(deal.soi_name_finance) OVER ( 					PARTITION BY a.fact_deal_allocation_key ORDER BY date_of_commitment ASC 					) 		ELSE deal.soi_name_finance 		END soi_name_finance 	,fact_deal_allocation_key 	,rn FROM ( 	SELECT fact_deal_allocation_key 		,fact_deal_id 		,date_of_commitment 		,ROW_NUMBER() OVER ( 			PARTITION BY fact_deal_allocation_key ORDER BY date_of_commitment ASC 			) AS rn 	FROM axiom_datalake.cl_snapshot.fact_deal_allocation 	) a LEFT JOIN axiom_datalake.cl_snapshot.fact_deal deal ON a.fact_deal_id = deal.fact_deal_id LEFT JOIN axiom_datalake.cl_snapshot.dim_company comp ON deal.company_name = comp.dim_company_id LEFT JOIN axiom_datalake.cl_snapshot.dim_fund fnd ON deal.legal_fund_name = fnd.dim_fund_id ) deal
		LEFT JOIN axiom_datalake.cl_snapshot.dim_gp gp ON deal.general_partner = gp.dim_company_id
		WHERE rn = 1
		)
	,CTE_af_inv_per AS (
		SELECT a.* ,b.gn_contribution_net_of_recallable_dyn AS prior_year_contribution_net_of_recallable_usd_dyn ,b.gn_total_value_dyn AS prior_year_total_value_usd_dyn ,b.gn_total_value AS prior_year_total_value_usd ,b.gn_contribution_net_of_recallable AS prior_year_contribution_net_of_recallable_usd
		FROM axiom_datalake.cl_snapshot.fact_axiom_fund_and_investment_performance a
		LEFT JOIN axiom_datalake.cl_snapshot.fact_axiom_fund_and_investment_performance b ON a.dim_axiomfund_id = b.dim_axiomfund_id AND a.report_type_id = b.report_type_id AND a.investment_iqid = b.investment_iqid AND b.as_of_date = DATEADD(year, - 1, a.as_of_date)
		ORDER BY a.dim_axiomfund_id ,a.report_type_id ,a.investment_iqid ,a.as_of_date DESC
		)
	,CTE_comma_separated AS (
		SELECT investments
		FROM ( SELECT investments 	,investment_type 	,dim_axiomfund_id 	,COUNT(1) FROM axiom_datalake.cl_snapshot.fact_axiomfund_investment_test WHERE dim_blocker_vehicle_id IS NULL GROUP BY investments 	,investment_type 	,dim_axiomfund_id HAVING COUNT(1) > 1 )
		)
	,CTE_blocker_vehicle AS (
		SELECT SUBSTRING(investments, CHARINDEX('(', investments) + 1, CHARINDEX(')', investments) - CHARINDEX('(', investments) - 1) AS blocker_vehicle_investments
		FROM axiom_datalake.cl_snapshot.fact_axiomfund_investment_test
		WHERE dim_blocker_vehicle_id IS NOT NULL
		GROUP BY investments
		)
	,CTE_af_inv_base AS (
		SELECT af.axiomfund_full_name AS axiom_fund_name ,af.axiomfund_short_name AS fof ,af_inv.fact_axiomfund_investment_iqid AS axiomfund_investment_iqid ,COALESCE(CAST(af.axiomfund_vintage AS INT), 'NULL') AS vintage_year_of_fund ,CASE  	WHEN LOWER(af_inv.investment_type) IN ( 			'secondary direct' 			,'co-investment' 			) 		THEN comp.geography_gn_analysis 	ELSE COALESCE(fnd.geography_gn_analysis, comp.geography_gn_analysis) 	END AS geography_gn_analysis ,CASE  	WHEN LOWER(af_inv.investment_type) IN ( 			'secondary direct' 			,'co-investment' 			) 		AND af_inv.dim_investee_company_id IS NOT NULL 		THEN comp.geography_one_pager 	ELSE COALESCE(fnd.geography_one_pager, comp.geography_one_pager, 'NULL') 	END AS geography_one_pager ,coalesce(cast(cast(original_vehicle_size_usd AS DECIMAL(18, 6)) AS string), original_vehicle_size_usd) AS original_vehicle_size_usd ,original_vehicle_size_lcy AS original_vehicle_size_lcy ,COALESCE(fnd.strategy_for_gn_analysis, 'NULL') AS strategy_for_gn_analysis ,COALESCE(af_inv.stage_at_investment, fnd.strategy_one_pager, 'NULL') AS strategy_one_pager ,fnd.one_pager_fund_size_lcy AS total_fund_size_lcy ,fnd.one_pager_fund_size_usd AS total_fund_size_usd ,COALESCE(CAST(vintage_cambridge AS INT), 'NULL') AS vintage_cambridge ,COALESCE(af_inv.axiom_was_a_sponsor, fact_dc.is_axiom_a_sponsor, 'NULL') AS axiom_was_a_sponsor ,COALESCE(fact_dc.discount_perc_to_nav, 'NULL') AS discount_perc_to_Nav ,COALESCE(fact_dc.axiom_deal_lead, 'NULL') AS names_of_investment_lead ,CASE  	WHEN af_inv.advisory_board_member IS NOT NULL 		THEN af_inv.advisory_board_member 	WHEN LOWER(fact_dc.does_axiom_have_board_seat) IN ( 			'yes' 			,'observer' 			) 		AND fact_dc.name_of_board_member IS NOT NULL 		THEN fact_dc.name_of_board_member 	WHEN LOWER(fact_dc.does_axiom_have_board_seat) IN ( 			'yes' 			,'observer' 			) 		AND fact_dc.name_of_board_member IS NULL 		THEN 'No fixed representative' 	WHEN LOWER(af_inv.investment_type) IN ( 			'primary mainline' 			,'primary toehold' 			,'secondary' 			) 		AND LOWER(fact_dc.does_axiom_have_board_seat) = 'no' 		THEN 'No' 	WHEN LOWER(af_inv.investment_type) LIKE 'primary%' 		OR LOWER(af_inv.investment_type) LIKE 'secondary%' 		OR LOWER(af_inv.investment_type) LIKE '%co-investment' 		THEN fact_dc.name_of_board_member 	ELSE 'NULL' 	END AS names_of_lp_advisory ,COALESCE(af_inv.initial_investment_date, fact_dc.date_of_commitment, 'NULL') AS initial_investment_date ,COALESCE(tr.currency_short_desc, fact_dc.currency_of_commitment_amount, 'NULL') AS investment_currency ,af_inv.investments AS investments ,COALESCE(af_inv.exit_date, fact_dc.date_of_exit, 'NULL') AS liquidation_exit_date 
		
		,cast(fact_dc.original_commitment_sec_split AS BIGINT) AS original_commitment_lcy 
		,fact_dc.original_commitment_sec_split
		
		,COALESCE(CASE  		WHEN fact_dc.purchase_price_sec_split IS NOT NULL 			THEN fact_dc.purchase_price_sec_split 		WHEN LOWER(af_inv.investment_type) = 'secondary' 			THEN fact_dc.axioms_net_purchase_price_on_funding_date 		ELSE purchase_price_sec_split 		END, 0) AS purchase_price_sec ,CASE  	WHEN af_inv.axiom_has_a_board_seat IS NOT NULL 		THEN af_inv.axiom_has_a_board_seat 	WHEN LOWER(af_inv.investment_type) LIKE '%primary%' 		OR LOWER(af_inv.investment_type) = 'secondary' 		THEN COALESCE(fact_dc.does_axiom_have_board_seat, 'NULL') 	ELSE 'NULL' 	END AS board_seat_fund_only 
		
		,CASE  	WHEN af_inv.fund_size_sec IS NOT NULL 		THEN af_inv.fund_size_sec 	WHEN LOWER(af_inv.investment_type) LIKE '%secondary%' 		THEN COALESCE(fnd.total_original_fund_size_lcy, fnd.fund_size_hardcap, fnd.fund_size_target) 	ELSE af_inv.fund_size_sec 	END AS fund_size_sec ,COALESCE(af_inv.IPS_Definition_Realization_Status, 'NULL') AS investment_status ,COALESCE(af_inv.investment_type, 'NULL') AS investment_type ,COALESCE(af_inv.is_next_gen, fact_dc.next_gen_manager, 'NULL') AS next_gen 	
		,CASE  	WHEN LOWER(af_inv.investment_type) LIKE '%primary%' 		THEN COALESCE(fact_dc.is_the_fund_over_subscribed, af_inv.is_oversubscribed, 'NULL') 	ELSE COALESCE(af_inv.is_oversubscribed, 'N/A') 	END AS over_subscribed ,CASE  	WHEN LOWER(af_inv.investment_type) = 'co-investment' 		THEN COALESCE(af_inv.is_part_of_co_investment_pool, 'NULL') 	END AS pool ,COALESCE(af_inv.name_for_axiom_quarterly_reporting, fact_dc.name_for_axiom_quarterly_reporting, 'NULL') AS qr_name ,COALESCE(soi_name_finance, af_inv.soi_name) AS soi_name ,CASE  	WHEN vintage_internal IS NOT NULL 		THEN vintage_internal 	WHEN LOWER(af_inv.investment_type) IN ( 			'secondary direct' 			,'co-investment' 			) 		THEN COALESCE(YEAR(fact_dc.date_of_commitment), 'NULL') 	ELSE 'NULL' 	END AS vintage_year ,COALESCE(fact_dc.gp_name, 'NULL') AS gp ,COALESCE(af_inv.prior_manager_funds_committed_to, fact_dc.num_of_prior_funds_axiom_committed_in_gp, 'NULL') AS no_of_prior_manager_funds_committed_to ,CASE  	WHEN af_inv.co_inv_offered IS NOT NULL 		THEN af_inv.co_inv_offered 	WHEN LOWER(af_inv.investment_type) LIKE '%primary%' 		THEN COALESCE(fact_dc.coinvestment_offered, 'NULL') 	ELSE 'NULL' 	END AS co_investment_offered ,af_inv.dim_axiomfund_id ,af_inv.fact_axiomfund_investment_iqid ,af_inv.exit_date ,fact_dc.date_of_exit ,fact_dc.axiom_commitment ,fact_dc.axiom_commitment_lcy ,af.axiomfundfamily_name AS axiomfundfamily_name_derived ,bv.blocker_vehicle_investments AS dim_blocker_vehicle_id ,CASE  	WHEN af_inv.deal_source IS NOT NULL 		AND LOWER(af_inv.deal_source) = 're_up' 		THEN 'Yes' 	WHEN ( 			af_inv.deal_source IS NULL 			OR LOWER(af_inv.deal_source) <> 're_up' 			) 		AND ( 			COALESCE(af_inv.prior_manager_funds_committed_to, fact_dc.num_of_prior_funds_axiom_committed_in_gp, 'NULL') IS NULL 			OR COALESCE(af_inv.prior_manager_funds_committed_to, fact_dc.num_of_prior_funds_axiom_committed_in_gp, 'NULL') = 'NULL' 			OR COALESCE(af_inv.prior_manager_funds_committed_to, fact_dc.num_of_prior_funds_axiom_committed_in_gp, 'NULL') = 0 			) 		THEN 'No' 	WHEN ( 			af_inv.deal_source IS NULL 			OR LOWER(af_inv.deal_source) <> 're_up' 			) 		AND COALESCE(af_inv.prior_manager_funds_committed_to, fact_dc.num_of_prior_funds_axiom_committed_in_gp, 'NULL') >= 1 		THEN 'Yes' 	ELSE af_inv.deal_source 	END AS re_up
		FROM ( SELECT af.* FROM axiom_datalake.cl_snapshot.fact_axiomfund_investment_test af WHERE af.fact_axiomfund_investment_iqid IN ( 		SELECT investment_iqid 		FROM CTE_af_inv 		) ) af_inv
		LEFT JOIN CTE_af_inv tr ON af_inv.fact_axiomfund_investment_iqid = tr.investment_iqid
		LEFT JOIN CTE_blocker_vehicle bv ON af_inv.investments = bv.blocker_vehicle_investments
		LEFT JOIN axiom_datalake.cl_snapshot.dim_axiomfund af ON af_inv.dim_axiomfund_id = af.dim_axiomfund_id
		LEFT JOIN axiom_datalake.cl_snapshot.dim_fund fnd ON af_inv.dim_investee_fund_id = fnd.dim_fund_id
		LEFT JOIN axiom_datalake.cl_snapshot.dim_company comp ON af_inv.dim_investee_company_id = comp.dim_company_id
		LEFT JOIN axiom_datalake.cl_snapshot.dim_currency c ON af_inv.dim_investment_currency_code = c.dim_currency_id
		LEFT JOIN ( SELECT DISTINCT dc.* 	,company_name_deal 	,fund_full_name_deal 	,co_investment_spv_portfolio_company_name_legal_name 	,deal.deal_type 	,deal.name_of_board_member 	,deal.axiom_deal_lead 	,deal.is_axiom_a_sponsor 	,deal.buyer_transaction_price_discount_to_nav 	,deal.axioms_net_purchase_price_on_funding_date 	,name_for_axiom_quarterly_reporting 	,is_the_fund_over_subscribed 	,coinvestment_offered 	,next_gen_manager 	,does_axiom_have_board_seat 	,num_of_prior_funds_axiom_committed_in_gp 	,gp_name 	,soi_name_finance FROM CTE_da dc LEFT JOIN CTE_deal deal ON dc.fact_deal_id = deal.fact_deal_id ) fact_dc ON af_inv.fact_axiomfund_investment_key = fact_dc.fact_deal_allocation_key
		)
	,CTE_af_inv_base_excluded_family AS (
		SELECT investments ,axiom_fund_name ,fof ,axiomfund_investment_iqid ,vintage_year_of_fund ,geography_gn_analysis ,geography_one_pager ,CASE  	WHEN investments = investments_sp 		THEN COALESCE(original_vehicle_size_usd, axiom_commitment) 	ELSE original_vehicle_size_usd 	END AS original_vehicle_size_usd ,original_vehicle_size_lcy ,strategy_for_gn_analysis ,strategy_one_pager ,CASE  	WHEN investments = investments_sp 		THEN COALESCE(total_fund_size_lcy, axiom_commitment_lcy) 	ELSE total_fund_size_lcy 	END AS total_fund_size_lcy ,CASE  	WHEN investments = investments_sp 		THEN COALESCE(total_fund_size_usd, axiom_commitment) 	ELSE total_fund_size_usd 	END AS total_fund_size_usd 
		
		--,vintage_cambridge 
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE vintage_cambridge 	END AS vintage_cambridge
		
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE axiom_was_a_sponsor 	END AS axiom_was_a_sponsor ,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE discount_perc_to_Nav 	END AS discount_perc_to_Nav 
		
		--,names_of_investment_lead 
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE names_of_investment_lead 	END AS names_of_investment_lead
		--,names_of_lp_advisory 
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE names_of_lp_advisory 	END AS names_of_lp_advisory
		
		,initial_investment_date ,investment_currency ,liquidation_exit_date ,CASE  	WHEN investments = investments_sp 		THEN COALESCE(original_commitment_lcy, axiom_commitment_lcy) 	ELSE original_commitment_lcy 	END AS original_commitment_lcy 
		
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN 0 	ELSE purchase_price_sec 	END AS purchase_price_sec 
		
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE board_seat_fund_only 	END AS board_seat_fund_only 
		
		--,CASE  	WHEN fund_size_sec = '80B' 		THEN '80000000000' 	WHEN investments = investments_sp 		THEN cast((COALESCE(fund_size_sec, axiom_commitment_lcy)) AS string) 	ELSE cast(fund_size_sec AS string) 	END AS fund_size_sec 
				
		,CAST(REPLACE(CASE WHEN fund_size_sec = '80B'  THEN '80000000000'  WHEN investments = investments_sp  THEN COALESCE(fund_size_sec , axiom_commitment_lcy )   ELSE fund_size_sec   END  , ',', '') AS DOUBLE  ) AS fund_size_sec
		
		
		,investment_status ,investment_type 
		
		--,next_gen 
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE next_gen 	END AS next_gen
		
		--,over_subscribed 
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE over_subscribed 	END AS over_subscribed
		,pool ,qr_name ,soi_name 
		--,vintage_year 
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE vintage_year 	END AS vintage_year
		--,gp 
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE gp 	END AS gp
		
		--,no_of_prior_manager_funds_committed_to 
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE no_of_prior_manager_funds_committed_to 	END AS no_of_prior_manager_funds_committed_to
		
		,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE co_investment_offered 	END AS co_investment_offered ,axiom_commitment ,axiom_commitment_lcy ,axiomfundfamily_name_derived ,dim_blocker_vehicle_id ,COALESCE(round((perc_drawn_lcy * 100), 6), perc_drawn_lcy) AS perc_drawn_lcy ,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE CAST(perc_drawn_sec AS DECIMAL(18, 6)) 	END AS perc_drawn_sec ,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE COALESCE(round((perc_of_fund_lcy_alloc_rule * 100), 6), perc_of_fund_lcy_alloc_rule) 	END AS perc_of_fund_lcy_alloc_rule ,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE COALESCE(round((perc_of_fund_lcy_one_pager * 100), 6), perc_of_fund_lcy_one_pager) 	END AS perc_of_fund_lcy_one_pager ,contribution_usd ,contribution_net_of_recallable_usd AS contribution_net_of_recallable_usd ,distribution_usd ,distribution_net_of_recallable_usd AS distribution_net_of_recallable_usd ,COALESCE(round((gn_irr_lcy * 100), 2), gn_irr_lcy) AS gn_irr_lcy ,COALESCE(round((gn_irr_usd * 100), 2), gn_irr_usd) AS gn_irr_usd ,gn_moc_lcy AS gn_moc_lcy ,nav_usd ,nav_lcy ,COALESCE((prior_year_total_value_usd / prior_year_contribution_net_of_recallable_usd), 'NULL') AS prior_year_gn_moc_usd ,COALESCE(((prior_year_total_value_usd_dyn / prior_year_contribution_net_of_recallable_usd_dyn)), 'NULL') AS prior_year_gn_moc_usd_dyn ,recallable_usd ,COALESCE(remaining_commitment_usd, 'NULL') AS remaining_commitment_usd ,total_value_usd ,total_remaining_commitment ,af_inv_per_as_of_date ,af_inv_per_report_type ,investment_status_fund_derived ,perc_of_fund_lcy ,commitment_usd AS commitment_usd ,COALESCE(round((gn_irr_usd_dyn * 100), 2), gn_irr_usd_dyn, 'NULL') AS gn_irr_usd_dyn ,COALESCE(round((gn_irr_lcy_dyn * 100), 2), gn_irr_lcy_dyn, 'NULL') AS gn_irr_lcy_dyn ,COALESCE(commitment_usd_dyn, 'NULL') AS commitment_usd_dyn ,COALESCE(remaining_commitment_usd_dyn, 'NULL') AS remaining_commitment_usd_dyn ,COALESCE(contribution_usd_dyn, 'NULL') AS contribution_usd_dyn ,COALESCE(distribution_usd_dyn, 'NULL') AS distribution_usd_dyn ,COALESCE(recallable_usd_dyn, 'NULL') AS recallable_usd_dyn ,COALESCE(contribution_net_of_recallable_usd_dyn, 'NULL') AS contribution_net_of_recallable_usd_dyn ,COALESCE(distribution_net_of_recallable_usd_dyn, 'NULL') AS distribution_net_of_recallable_usd_dyn ,COALESCE(nav_usd_dyn, 'NULL') AS nav_usd_dyn ,COALESCE(nav_lcy_dyn, 'NULL') AS nav_lcy_dyn ,COALESCE(total_value_usd_dyn, 'NULL') AS total_value_usd_dyn ,COALESCE(total_remaining_commitment_dyn, 'NULL') AS total_remaining_commitment_dyn ,(((contribution_net_of_recallable_usd) / (commitment_usd))) * 100 AS perc_drawn_usd ,((total_value_usd) / (contribution_net_of_recallable_usd)) AS gn_moc_usd ,((distribution_net_of_recallable_usd) / (contribution_net_of_recallable_usd)) AS dpi_usd ,CASE  	WHEN investments = investments_sp 		AND lower(investment_type) <> lower(investment_type_sp) 		THEN NULL 	ELSE COALESCE((perc_drawn_sec_dyn), ((((contribution_net_of_recallable_lcy_dyn) / (commitment_lcy_dyn))) * 100)) 	END AS perc_drawn_sec_dyn ,((((contribution_net_of_recallable_usd_dyn) / (commitment_usd_dyn))) * 100) AS perc_drawn_usd_dyn ,((((contribution_net_of_recallable_lcy_dyn) / (commitment_lcy_dyn))) * 100) AS perc_drawn_lcy_dyn ,((distribution_net_of_recallable_usd_dyn) / (contribution_net_of_recallable_usd_dyn)) AS dpi_usd_dyn ,((total_value_usd_dyn) / (contribution_net_of_recallable_usd_dyn)) AS gn_moc_usd_dyn 
		--,((total_value_lcy_dyn) / (contribution_net_of_recallable_lcy_dyn)) AS gn_moc_lcy_dyn 
		--,(gn_distribution_lcy_dyn - gn_recallable_lcy_dyn + nav_lcy_dyn)/(gn_contribution_lcy_dyn - gn_recallable_lcy_dyn) AS gn_moc_lcy_dyn 		
				
		,remaining_commitment_lcy_dyn ,commitment_lcy_dyn ,total_value_lcy_dyn ,contribution_net_of_recallable_lcy_dyn ,prior_year_total_value_usd_dyn ,prior_year_contribution_net_of_recallable_usd_dyn ,gn_investee_fund_size_lcy ,gn_investee_fund_size ,valuation_type_gg ,valuation_type_gn ,gg_nav_dyn ,gg_cash_returned_dyn ,gg_cash_invested_dyn ,((gg_nav_dyn + gg_cash_returned_dyn) / gg_cash_invested_dyn) AS gg_moc_dyn ,nn_distribution_dyn ,nn_nav_dyn ,nn_contribution_dyn ,((nn_distribution_dyn + nn_nav_dyn) / nn_contribution_dyn) AS nn_moc_dyn ,(nn_distribution_dyn / nn_contribution_dyn) AS nn_dpi_dyn ,re_up ,investments_sp ,investment_type_sp,gn_distribution_lcy_dyn,gn_recallable_lcy_dyn,gn_contribution_lcy_dyn
		
		FROM ( SELECT af_inv.* 	,coalesce(cast(cast(af_inv_per.gn_percentage_drawn_lcy AS DOUBLE) AS string), 'NULL') AS perc_drawn_lcy 	,coalesce(replace(af_inv_per.percentage_drawn_sec, '%', ''), af_inv_per.gn_percentage_drawn_lcy) AS perc_drawn_sec 	,coalesce(af_inv_per.perc_of_fund_lcy_alloc_rule, 'NULL') AS perc_of_fund_lcy_alloc_rule 	,coalesce(af_inv_per.perc_of_fund_lcy_one_pager, 'NULL') AS perc_of_fund_lcy_one_pager 	,af_inv_per.gn_contribution AS contribution_usd 	,af_inv_per.gn_contribution_dyn AS contribution_usd_dyn 	,cast(cast(af_inv_per.gn_contribution_net_of_recallable AS DOUBLE) AS string) AS contribution_net_of_recallable_usd 	,cast(cast(af_inv_per.gn_contribution_net_of_recallable_dyn AS DOUBLE) AS string) AS contribution_net_of_recallable_usd_dyn 	,af_inv_per.gn_distribution AS distribution_usd 	,af_inv_per.gn_distribution_dyn AS distribution_usd_dyn 	,cast(cast(af_inv_per.gn_distribution_net_of_recallable AS DOUBLE) AS string) AS distribution_net_of_recallable_usd 	,cast(cast(af_inv_per.gn_distribution_net_of_recallable_dyn AS DOUBLE) AS string) AS distribution_net_of_recallable_usd_dyn 	,CASE  		WHEN af_inv.Investments = "Advanced Power Electronics Corp" 			THEN coalesce(cast(cast(af_inv_per.gn_irr AS DOUBLE) AS string), 'NULL') 		ELSE coalesce(cast(cast(af_inv_per.gn_irr_lcy AS DOUBLE) AS string), 'NULL') 		END gn_irr_lcy 	,CASE  		WHEN af_inv.Investments = "Advanced Power Electronics Corp" 			THEN coalesce(cast(cast(af_inv_per.gn_irr_dyn AS DOUBLE) AS string), 'NULL') 		ELSE coalesce(cast(cast(af_inv_per.gn_irr_lcy_dyn AS DOUBLE) AS string), 'NULL') 		END gn_irr_lcy_dyn 	,coalesce(cast(cast(af_inv_per.gn_irr AS DOUBLE) AS string), 'NULL') AS gn_irr_usd 	,coalesce(cast(cast(af_inv_per.gn_irr_dyn AS DOUBLE) AS string), 'NULL') AS gn_irr_usd_dyn 	,CASE  		WHEN af_inv.Investments = "Advanced Power Electronics Corp" 			THEN coalesce(cast(cast(af_inv_per.gn_moc AS DOUBLE) AS string), 'NULL') 		ELSE coalesce(cast(cast(af_inv_per.gn_moc_lcy AS DOUBLE) AS string), 'NULL') 		END gn_moc_lcy 	,coalesce(af_inv_per.gn_nav, 'NULL') AS nav_usd 	,coalesce(af_inv_per.gn_nav_dyn, 'NULL') AS nav_usd_dyn 	,coalesce(af_inv_per.gn_nav_lcy, 'NULL') AS nav_lcy 	,coalesce(af_inv_per.gn_nav_lcy_dyn, 'NULL') AS nav_lcy_dyn 	,coalesce(af_inv_per.gn_recallable, 'NULL') AS recallable_usd 	,coalesce(af_inv_per.gn_recallable_dyn, 'NULL') AS recallable_usd_dyn 	,coalesce(af_inv_per.gn_remaining_commitment, 'NULL') AS remaining_commitment_usd 	,coalesce(af_inv_per.gn_remaining_commitment_dyn, 'NULL') AS remaining_commitment_usd_dyn 	,coalesce(af_inv_per.gn_total_value, 'NULL') AS total_value_usd 	,coalesce(af_inv_per.gn_total_value_dyn, 'NULL') AS total_value_usd_dyn 	,coalesce(cast(af_inv_per.gn_remaining_commitment AS DOUBLE), 'NULL') AS total_remaining_commitment 	,coalesce(cast(af_inv_per.gn_remaining_commitment_dyn AS DOUBLE), 'NULL') AS total_remaining_commitment_dyn 	,coalesce(cast(af_inv_per.as_of_date AS DATE), 'NULL') AS af_inv_per_as_of_date 	,coalesce(af_inv_per.report_type_id, 'NULL') AS af_inv_per_report_type 	,coalesce(af_inv_per.realization_status_derived, 'NULL') AS investment_status_fund_derived 	,CASE  		WHEN lower(af_inv.investment_type) LIKE '%primary%' 			THEN cast(cast(af_inv_per.perc_of_fund_lcy_one_pager AS DOUBLE) AS string) 		ELSE 'N/A' 		END AS perc_of_fund_lcy 	,coalesce(cast(cast(coalesce(af_inv_per.gn_commitment, af_inv.axiom_commitment) AS DOUBLE) AS string), 'NULL') AS commitment_usd 	,coalesce(cast(cast(af_inv_per.gn_commitment_dyn AS DOUBLE) AS string), 'NULL') AS commitment_usd_dyn 	,af_inv_per.percentage_drawn_sec AS perc_drawn_sec_dyn 	,gn_remaining_commitment_lcy_dyn AS remaining_commitment_lcy_dyn 	,gn_commitment_lcy_dyn AS commitment_lcy_dyn 	,gn_total_value_lcy_dyn AS total_value_lcy_dyn 	,gn_contribution_net_of_recallable_lcy_dyn AS contribution_net_of_recallable_lcy_dyn 	,prior_year_total_value_usd_dyn 	,prior_year_contribution_net_of_recallable_usd_dyn 	,gn_investee_fund_size_lcy 	,gn_investee_fund_size 	,prior_year_total_value_usd 	,prior_year_contribution_net_of_recallable_usd 	,valuation_type_gg AS valuation_type_gg 	,valuation_type_gn as valuation_type_gn 	,cast(cast(af_inv_per.gg_nav_dyn AS DOUBLE) AS string) AS gg_nav_dyn 	,cast(cast(af_inv_per.gg_cash_returned_dyn AS DOUBLE) AS string) AS gg_cash_returned_dyn 	,cast(cast(af_inv_per.gg_cash_invested_dyn AS DOUBLE) AS string) AS gg_cash_invested_dyn 	,cast(cast(af_inv_per.nn_distribution_dyn AS DOUBLE) AS string) AS nn_distribution_dyn 	,cast(cast(af_inv_per.nn_nav_dyn AS DOUBLE) AS string) AS nn_nav_dyn 	,cast(cast(af_inv_per.nn_contribution_dyn AS DOUBLE) AS string) AS nn_contribution_dyn 	,cast(cast(af_inv_per.nn_dpi_dyn AS DOUBLE) AS string) AS nn_dpi_dyn 	,sp_comb.investments_sp AS investments_sp 	,sp_comb.investment_type_sp AS investment_type_sp,coalesce(af_inv_per.gn_distribution_lcy_dyn, 'NULL') AS gn_distribution_lcy_dyn ,coalesce(af_inv_per.gn_recallable_lcy_dyn, 'NULL') AS gn_recallable_lcy_dyn ,coalesce(af_inv_per.gn_contribution_lcy_dyn, 'NULL') AS gn_contribution_lcy_dyn FROM ( 	SELECT * 	FROM CTE_af_inv_base af_inv 	WHERE af_inv.dim_axiomfund_id NOT IN ( 			SELECT dim_axiomfund_id 			FROM CTE_affam 			) 	) af_inv LEFT JOIN CTE_sp_combine_inv_type sp_comb ON af_inv.investments = sp_comb.investments_sp LEFT JOIN CTE_af_inv_per af_inv_per ON af_inv.dim_axiomfund_id = af_inv_per.dim_axiomfund_id 	AND af_inv.fact_axiomfund_investment_iqid = af_inv_per.investment_iqid 	AND cast(af_inv_per.as_of_date AS DATE) = '2024-03-31' 	AND af_inv_per.report_type_id IN ('R3') )
		)
		
		
	,CTE_comma_separated_records AS (
		SELECT *
		FROM ( SELECT investments 	,MIN(axiom_fund_name) AS axiom_fund_name 	,fof AS fof 	,MIN(axiomfund_investment_iqid) AS axiomfund_investment_iqid 	,MIN(vintage_year_of_fund) AS vintage_year_of_fund 	,MIN(geography_gn_analysis) AS geography_gn_analysis 	,MIN(geography_one_pager) AS geography_one_pager 	,MIN(original_vehicle_size_usd) AS original_vehicle_size_usd 	,MIN(original_vehicle_size_lcy) AS original_vehicle_size_lcy 	,MIN(strategy_for_gn_analysis) AS strategy_for_gn_analysis 	,MIN(strategy_one_pager) AS strategy_one_pager 	,MIN(total_fund_size_lcy) AS total_fund_size_lcy 	,MIN(total_fund_size_usd) AS total_fund_size_usd 	,MIN(vintage_cambridge) AS vintage_cambridge 	,MIN(axiom_was_a_sponsor) AS axiom_was_a_sponsor 	,MIN(discount_perc_to_Nav) AS discount_perc_to_Nav 	,MIN(names_of_investment_lead) AS names_of_investment_lead 	,MIN(names_of_lp_advisory) AS names_of_lp_advisory 	,MIN(initial_investment_date) AS initial_investment_date 	,CASE  		WHEN MAX(investment_currency) = 'USD' 			THEN 'USD' 		ELSE MIN(investment_currency) 		END AS investment_currency 	,MIN(liquidation_exit_date) AS liquidation_exit_date 	,SUM(original_commitment_lcy) AS original_commitment_lcy 	,SUM(purchase_price_sec) AS purchase_price_sec 	,MIN(board_seat_fund_only) AS board_seat_fund_only 	,MIN(fund_size_sec) AS fund_size_sec 	,MIN(investment_status) AS investment_status 	,investment_type AS investment_type 	,MIN(next_gen) AS next_gen 	,MIN(over_subscribed) AS over_subscribed 	,MIN(pool) AS pool 	,MIN(qr_name) AS qr_name 	,MIN(soi_name) AS soi_name 	,MIN(vintage_year) AS vintage_year 	,MIN(gp) AS gp 	,MIN(no_of_prior_manager_funds_committed_to) AS no_of_prior_manager_funds_committed_to 	,MIN(co_investment_offered) AS co_investment_offered 	,MIN(axiom_commitment) AS axiom_commitment 	,MIN(axiom_commitment_lcy) AS axiom_commitment_lcy 	,MIN(axiomfundfamily_name_derived) AS axiomfundfamily_name_derived 	,MIN(dim_blocker_vehicle_id) AS dim_blocker_vehicle_id 	,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 				perc_drawn_lcy 				,'%' 				))) AS perc_drawn_lcy 	,CASE  		WHEN ARRAY_CONTAINS(COLLECT_LIST(CAST(perc_drawn_sec AS STRING)), 'N/A') 			THEN 'N/A' 		ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 						perc_drawn_sec 						,'%' 						))) 		END AS perc_drawn_sec 	,COALESCE(SUM(perc_of_fund_lcy_alloc_rule), MIN(perc_of_fund_lcy_alloc_rule)) AS perc_of_fund_lcy_alloc_rule 	,COALESCE(SUM(perc_of_fund_lcy_one_pager), MIN(perc_of_fund_lcy_one_pager)) AS perc_of_fund_lcy_one_pager 	,SUM(contribution_usd) AS contribution_usd 	,SUM(contribution_usd_dyn) AS contribution_usd_dyn 	,SUM(contribution_net_of_recallable_usd) AS contribution_net_of_recallable_usd 	,SUM(contribution_net_of_recallable_usd_dyn) AS contribution_net_of_recallable_usd_dyn 	,SUM(distribution_usd) AS distribution_usd 	,SUM(distribution_usd_dyn) AS distribution_usd_dyn 	,SUM(distribution_net_of_recallable_usd) AS distribution_net_of_recallable_usd 	,SUM(distribution_net_of_recallable_usd_dyn) AS distribution_net_of_recallable_usd_dyn 	,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 				gn_irr_lcy 				,'%' 				))) AS gn_irr_lcy 	,MIN(gn_irr_lcy_dyn) AS gn_irr_lcy_dyn 	,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 				gn_irr_usd 				,'%' 				))) AS gn_irr_usd 	,MIN(gn_irr_usd_dyn) AS gn_irr_usd_dyn 	,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 				gn_moc_lcy 				,'x' 				))) AS gn_moc_lcy 	,SUM(nav_usd_dyn) AS nav_usd_dyn 	,SUM(nav_usd) AS nav_usd 	,SUM(nav_lcy) AS nav_lcy 	,SUM(nav_lcy_dyn) AS nav_lcy_dyn 	,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 				prior_year_gn_moc_usd 				,'x' 				))) AS prior_year_gn_moc_usd 	,SUM(recallable_usd) AS recallable_usd 	,SUM(recallable_usd_dyn) AS recallable_usd_dyn 	,SUM(remaining_commitment_usd) AS remaining_commitment_usd 	,SUM(remaining_commitment_usd_dyn) AS remaining_commitment_usd_dyn 	,SUM(total_value_usd) AS total_value_usd 	,SUM(total_value_usd_dyn) AS total_value_usd_dyn 	,MIN(total_remaining_commitment) AS total_remaining_commitment 	,MIN(total_remaining_commitment_dyn) AS total_remaining_commitment_dyn 	,MIN(af_inv_per_as_of_date) AS af_inv_per_as_of_date 	,MIN(af_inv_per_report_type) AS af_inv_per_report_type 	,CASE  		WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Fully Realized,Fully Realized' 			THEN 'Fully Realized' 		WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Unrealized,Unrealized' 			THEN 'Unrealized' 		WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Partially Realized,Partially Realized' 			THEN 'Partially Realized' 		ELSE 'Data Error' 		END AS investment_status_fund_derived 	,MIN(perc_of_fund_lcy) AS perc_of_fund_lcy 	,SUM(commitment_usd) AS commitment_usd 	,SUM(commitment_usd_dyn) AS commitment_usd_dyn 	,((SUM(contribution_net_of_recallable_usd) / SUM(commitment_usd))) * 100 AS perc_drawn_usd 	,(SUM(total_value_usd) / SUM(contribution_net_of_recallable_usd)) AS gn_moc_usd 	,(SUM(distribution_net_of_recallable_usd) / SUM(contribution_net_of_recallable_usd)) AS dpi_usd 	,COALESCE(MIN(perc_drawn_sec_dyn), (((SUM(contribution_net_of_recallable_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100)) AS perc_drawn_sec_dyn 	,sum(remaining_commitment_lcy_dyn) AS remaining_commitment_lcy_dyn 	,sum(commitment_lcy_dyn) AS commitment_lcy_dyn 	,SUM(total_value_lcy_dyn) AS total_value_lcy_dyn 	,SUM(contribution_net_of_recallable_lcy_dyn) AS contribution_net_of_recallable_lcy_dyn 	,SUM(prior_year_total_value_usd_dyn) AS prior_year_total_value_usd_dyn 	,SUM(prior_year_contribution_net_of_recallable_usd_dyn) AS prior_year_contribution_net_of_recallable_usd_dyn 	,SUM(gn_investee_fund_size_lcy) AS gn_investee_fund_size_lcy 	,SUM(gn_investee_fund_size) AS gn_investee_fund_size 	,((SUM(contribution_net_of_recallable_usd_dyn) / SUM(commitment_usd_dyn))) * 100 AS perc_drawn_usd_dyn 	,((SUM(contribution_net_of_recallable_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100 AS perc_drawn_lcy_dyn 	,(SUM(distribution_net_of_recallable_usd_dyn) / SUM(contribution_net_of_recallable_usd_dyn)) AS dpi_usd_dyn 	,(SUM(total_value_usd_dyn) / SUM(contribution_net_of_recallable_usd_dyn)) AS gn_moc_usd_dyn 	
		
		--,(SUM(total_value_lcy_dyn) / SUM(contribution_net_of_recallable_lcy_dyn)) AS gn_moc_lcy_dyn 
		--,(SUM(gn_distribution_lcy_dyn) - SUM(gn_recallable_lcy_dyn) + SUM(nav_usd_dyn) )/(SUM(gn_contribution_lcy_dyn) -  SUM(gn_recallable_lcy_dyn)) AS gn_moc_lcy_dyn

		,sum(prior_year_total_value_usd_dyn) / sum(prior_year_contribution_net_of_recallable_usd_dyn) AS prior_year_gn_moc_usd_dyn 	,min(valuation_type_gg) AS valuation_type_gg 	,min(valuation_type_gn) AS valuation_type_gn 	,SUM(gg_nav_dyn) AS gg_nav_dyn 	,SUM(gg_cash_returned_dyn) AS gg_cash_returned_dyn 	,SUM(gg_cash_invested_dyn) AS gg_cash_invested_dyn 	,((SUM(gg_nav_dyn) + SUM(gg_cash_returned_dyn)) / SUM(gg_cash_invested_dyn)) AS gg_moc_dyn 	,SUM(nn_distribution_dyn) AS nn_distribution_dyn 	,SUM(nn_nav_dyn) AS nn_nav_dyn 	,SUM(nn_contribution_dyn) AS nn_contribution_dyn 	,((SUM(nn_distribution_dyn) + SUM(nn_nav_dyn)) / SUM(nn_contribution_dyn)) AS nn_moc_dyn 	,(SUM(nn_distribution_dyn) / SUM(nn_contribution_dyn)) AS nn_dpi_dyn 	,min(re_up) AS re_up
		,SUM(gn_distribution_lcy_dyn) AS gn_distribution_lcy_dyn
		,SUM(gn_recallable_lcy_dyn) AS gn_recallable_lcy_dyn
		,SUM(gn_contribution_lcy_dyn) AS gn_contribution_lcy_dyn
			
		
		FROM CTE_af_inv_base_excluded_family WHERE investments IN ( 		SELECT investments 		FROM CTE_comma_separated 		) GROUP BY fof 	,investments 	,investment_type )
		)
		
		
		
	,CTE_af_inv_base_only_family AS (
		SELECT investments AS investments ,MIN(axiom_fund_name) AS axiom_fund_name ,fof AS fof ,MIN(axiomfund_investment_iqid) AS axiomfund_investment_iqid ,MIN(vintage_year_of_fund) AS vintage_year_of_fund ,MIN(geography_gn_analysis) AS geography_gn_analysis ,MIN(geography_one_pager) AS geography_one_pager ,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN sum(original_vehicle_size_usd) 	ELSE MIN(original_vehicle_size_usd) 	END AS original_vehicle_size_usd ,MIN(original_vehicle_size_lcy) AS original_vehicle_size_lcy ,MIN(strategy_for_gn_analysis) AS strategy_for_gn_analysis ,MIN(strategy_one_pager) AS strategy_one_pager

		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN sum(total_fund_size_lcy) 	ELSE MIN(total_fund_size_lcy) 	END AS total_fund_size_lcy

		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN sum(total_fund_size_usd) 	ELSE MIN(total_fund_size_usd) 	END AS total_fund_size_usd 
		
		--,MIN(vintage_cambridge) AS vintage_cambridge 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(vintage_cambridge) 	END AS vintage_cambridge
		--,MIN(axiom_was_a_sponsor) AS axiom_was_a_sponsor 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(axiom_was_a_sponsor) 	END AS axiom_was_a_sponsor
		
		,MIN(discount_perc_to_Nav) AS discount_perc_to_Nav 
		
		--,MIN(names_of_investment_lead) AS names_of_investment_lead 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(names_of_investment_lead) 	END AS names_of_investment_lead
		--,MIN(names_of_lp_advisory) AS names_of_lp_advisory 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(names_of_lp_advisory) 	END AS names_of_lp_advisory
		
		,MIN(initial_investment_date) AS initial_investment_date ,MIN(investment_currency) AS investment_currency ,MIN(liquidation_exit_date) AS liquidation_exit_date ,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN sum(original_commitment_lcy) 	ELSE SUM(original_commitment_lcy) 	END AS original_commitment_lcy ,
		
		SUM(purchase_price_sec) AS purchase_price_sec 
		
		
		--,MIN(board_seat_fund_only) AS board_seat_fund_only 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(board_seat_fund_only) 	END AS board_seat_fund_only
		
		
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN SUM(fund_size_sec) 	ELSE MIN(fund_size_sec) 	END AS fund_size_sec 
		
		,MIN(investment_status) AS investment_status ,investment_type AS investment_type 
		
		--,MIN(next_gen) AS next_gen 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(next_gen) 	END AS next_gen
		
		
		--,MIN(over_subscribed) AS over_subscribed 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(over_subscribed) 	END AS over_subscribed
		
		,MIN(pool) AS pool ,MIN(qr_name) AS qr_name ,MIN(soi_name) AS soi_name 
		
		--,MIN(vintage_year) AS vintage_year 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(vintage_year) 	END AS vintage_year
		--,MIN(gp) AS gp 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(gp) 	END AS gp
		--,MIN(no_of_prior_manager_funds_committed_to) AS no_of_prior_manager_funds_committed_to 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(no_of_prior_manager_funds_committed_to) 	END AS no_of_prior_manager_funds_committed_to
		
		
		--,MIN(co_investment_offered) AS co_investment_offered 
		,CASE  	WHEN min(investments) = min(investments_sp) 		AND lower(min(investment_type)) <> lower(min(investment_type_sp)) 		THEN NULL 	ELSE MIN(co_investment_offered) 	END AS co_investment_offered
		
		,MIN(axiom_commitment) AS axiom_commitment ,MIN(axiom_commitment_lcy) AS axiom_commitment_lcy ,MIN(axiomfundfamily_name_derived) AS axiomfundfamily_name_derived ,MIN(dim_blocker_vehicle_id) AS dim_blocker_vehicle_id ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL		THEN MIN(perc_drawn_lcy) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					perc_drawn_lcy 					,'%' 					))) 	END AS perc_drawn_lcy		,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL	THEN MIN(perc_drawn_sec) 	WHEN ARRAY_CONTAINS(COLLECT_LIST(CAST(perc_drawn_sec AS STRING)), 'N/A') 		THEN 'N/A' 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					perc_drawn_sec 					,'%' 					))) 	END AS perc_drawn_sec 	,COALESCE(SUM(perc_of_fund_lcy_alloc_rule), MIN(perc_of_fund_lcy_alloc_rule)) AS perc_of_fund_lcy_alloc_rule ,COALESCE(SUM(perc_of_fund_lcy_one_pager), MIN(perc_of_fund_lcy_one_pager)) AS perc_of_fund_lcy_one_pager ,SUM(contribution_usd) AS contribution_usd ,SUM(contribution_net_of_recallable_usd) AS contribution_net_of_recallable_usd ,SUM(distribution_usd) AS distribution_usd ,SUM(distribution_net_of_recallable_usd) AS distribution_net_of_recallable_usd 	,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL		THEN MIN(gn_irr_lcy) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					gn_irr_lcy 					,'%' 					))) 	END AS gn_irr_lcy	
		
		,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL		THEN MIN(gn_irr_usd) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( gn_irr_usd 	,'%' ))) 	END AS gn_irr_usd	
		
		,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL	THEN MIN(gn_moc_lcy) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					gn_moc_lcy 					,'x' 					))) 	END AS gn_moc_lcy ,SUM(nav_usd) AS nav_usd ,SUM(nav_lcy) AS nav_lcy ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL		THEN MIN(prior_year_gn_moc_usd) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					prior_year_gn_moc_usd ,'x' 	))) 	END AS prior_year_gn_moc_usd ,SUM(recallable_usd) AS recallable_usd ,SUM(remaining_commitment_usd) AS remaining_commitment_usd ,SUM(total_value_usd) AS total_value_usd ,MIN(total_remaining_commitment) AS total_remaining_commitment ,MIN(af_inv_per_as_of_date) AS af_inv_per_as_of_date ,MIN(af_inv_per_report_type) AS af_inv_per_report_type ,CASE  	WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Fully Realized,Fully Realized' 		THEN 'Fully Realized' 	WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Unrealized,Unrealized' 		THEN 'Unrealized' 	WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Partially Realized,Partially Realized' 		THEN 'Partially Realized' 	ELSE 'Data Error' 	END AS investment_status_fund_derived ,MIN(perc_of_fund_lcy) AS perc_of_fund_lcy ,SUM(commitment_usd) AS commitment_usd ,MIN(gn_irr_usd_dyn) AS gn_irr_usd_dyn ,MIN(gn_irr_lcy_dyn) AS gn_irr_lcy_dyn ,sum(commitment_usd_dyn) AS commitment_usd_dyn ,SUM(remaining_commitment_usd_dyn) AS remaining_commitment_usd_dyn ,SUM(contribution_usd_dyn) AS contribution_usd_dyn ,SUM(distribution_usd_dyn) AS distribution_usd_dyn ,SUM(recallable_usd_dyn) AS recallable_usd_dyn ,SUM(contribution_net_of_recallable_usd_dyn) AS contribution_net_of_recallable_usd_dyn ,SUM(distribution_net_of_recallable_usd_dyn) AS distribution_net_of_recallable_usd_dyn ,SUM(nav_usd_dyn) AS nav_usd_dyn ,SUM(nav_lcy_dyn) AS nav_lcy_dyn ,SUM(total_value_usd_dyn) AS total_value_usd_dyn ,MIN(total_remaining_commitment_dyn) AS total_remaining_commitment_dyn ,COALESCE(MIN(perc_drawn_sec_dyn), (((SUM(remaining_commitment_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100)) AS perc_drawn_sec_dyn ,SUM(remaining_commitment_lcy_dyn) AS remaining_commitment_lcy_dyn ,SUM(commitment_lcy_dyn) AS commitment_lcy_dyn ,SUM(total_value_lcy_dyn) AS total_value_lcy_dyn ,SUM(contribution_net_of_recallable_lcy_dyn) AS contribution_net_of_recallable_lcy_dyn ,SUM(prior_year_total_value_usd_dyn) AS prior_year_total_value_usd_dyn ,SUM(prior_year_contribution_net_of_recallable_usd_dyn) AS prior_year_contribution_net_of_recallable_usd_dyn ,SUM(gn_investee_fund_size_lcy) AS gn_investee_fund_size_lcy ,SUM(gn_investee_fund_size) AS gn_investee_fund_size ,((SUM(contribution_net_of_recallable_usd) / SUM(commitment_usd))) * 100 AS perc_drawn_usd ,(SUM(total_value_usd) / SUM(contribution_net_of_recallable_usd)) AS gn_moc_usd ,(SUM(distribution_net_of_recallable_usd) / SUM(contribution_net_of_recallable_usd)) AS dpi_usd ,(((SUM(contribution_net_of_recallable_usd_dyn) / SUM(commitment_usd_dyn))) * 100) AS perc_drawn_usd_dyn ,((SUM(contribution_net_of_recallable_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100 AS perc_drawn_lcy_dyn ,(SUM(distribution_net_of_recallable_usd_dyn) / SUM(contribution_net_of_recallable_usd_dyn)) AS dpi_usd_dyn ,(SUM(total_value_usd_dyn) / SUM(contribution_net_of_recallable_usd_dyn)) AS gn_moc_usd_dyn 
		
		--,(SUM(total_value_lcy_dyn) / SUM(contribution_net_of_recallable_lcy_dyn)) AS gn_moc_lcy_dyn 
		--,(SUM(gn_distribution_lcy_dyn) - SUM(gn_recallable_lcy_dyn) + SUM(nav_usd_dyn) )/(SUM(gn_contribution_lcy_dyn) -  SUM(gn_recallable_lcy_dyn)) AS gn_moc_lcy_dyn
		
		,sum(prior_year_total_value_usd_dyn) / sum(prior_year_contribution_net_of_recallable_usd_dyn) AS prior_year_gn_moc_usd_dyn ,min(valuation_type_gg) AS valuation_type_gg ,min(valuation_type_gn) AS valuation_type_gn ,SUM(gg_nav_dyn) AS gg_nav_dyn ,SUM(gg_cash_returned_dyn) AS gg_cash_returned_dyn ,SUM(gg_cash_invested_dyn) AS gg_cash_invested_dyn ,((SUM(gg_nav_dyn) + SUM(gg_cash_returned_dyn)) / SUM(gg_cash_invested_dyn)) AS gg_moc_dyn ,SUM(nn_distribution_dyn) AS nn_distribution_dyn ,SUM(nn_nav_dyn) AS nn_nav_dyn ,SUM(nn_contribution_dyn) AS nn_contribution_dyn ,((SUM(nn_distribution_dyn) + SUM(nn_nav_dyn)) / SUM(nn_contribution_dyn)) AS nn_moc_dyn ,(SUM(nn_distribution_dyn) / SUM(nn_contribution_dyn)) AS nn_dpi_dyn ,min(re_up) AS re_up ,min(investments_sp) AS investments_sp ,min(investment_type_sp) AS investment_type_sp
		,SUM(gn_distribution_lcy_dyn) AS gn_distribution_lcy_dyn
		,SUM(gn_recallable_lcy_dyn) AS gn_recallable_lcy_dyn
		,SUM(gn_contribution_lcy_dyn) AS gn_contribution_lcy_dyn
		
		
		FROM ( SELECT DISTINCT af_inv.investments 	,af_inv.axiom_fund_name 	,af_inv.fof 	,af_inv.axiomfund_investment_iqid 	,af_inv.vintage_year_of_fund 	,af_inv.geography_gn_analysis 	,af_inv.geography_one_pager 	,af_inv_per.original_vehicle_size_usd 	,af_inv.original_vehicle_size_lcy 	,af_inv.strategy_for_gn_analysis 	,af_inv.strategy_one_pager 	,af_inv_per.total_fund_size_lcy 	,af_inv_per.total_fund_size_usd 	,af_inv.vintage_cambridge 	,af_inv_per.axiom_was_a_sponsor 	,af_inv_per.discount_perc_to_Nav 	,af_inv_per.names_of_investment_lead 	,af_inv_per.names_of_lp_advisory 	,af_inv_per.initial_investment_date 	,af_inv.investment_currency 	,af_inv_per.liquidation_exit_date 	,af_inv_per.original_commitment_lcy 	,af_inv_per.purchase_price_sec 	,af_inv_per.board_seat_fund_only 	,af_inv_per.fund_size_sec 	,af_inv.investment_status 	,af_inv.investment_type 	,af_inv.next_gen 	,af_inv.over_subscribed 	,af_inv.pool 	,af_inv.qr_name 	,af_inv_per.soi_name 	,af_inv.vintage_year 	,af_inv_per.gp 	,af_inv.no_of_prior_manager_funds_committed_to 	,af_inv_per.co_investment_offered 	,af_inv.axiom_commitment 	,af_inv.axiom_commitment_lcy 	,af_inv.axiomfundfamily_name_derived 	,af_inv.dim_blocker_vehicle_id 	,perc_drawn_lcy 	,perc_drawn_sec 	,perc_of_fund_lcy_alloc_rule 	,perc_of_fund_lcy_one_pager 	,contribution_usd 	,contribution_net_of_recallable_usd 	,distribution_usd 	,distribution_net_of_recallable_usd 	,gn_irr_lcy 	,gn_irr_usd 	,gn_moc_lcy 	,nav_usd 	,nav_lcy 	,prior_year_gn_moc_usd 	,recallable_usd 	,remaining_commitment_usd 	,total_value_usd 	,total_remaining_commitment 	,af_inv_per_as_of_date 	,af_inv_per_report_type 	,investment_status_fund_derived 	,perc_of_fund_lcy 	,commitment_usd 	,gn_irr_usd_dyn 	,gn_irr_lcy_dyn 	,commitment_usd_dyn 	,remaining_commitment_usd_dyn 	,contribution_usd_dyn 	,distribution_usd_dyn 	,recallable_usd_dyn 	,contribution_net_of_recallable_usd_dyn 	,distribution_net_of_recallable_usd_dyn 	,nav_usd_dyn 	,nav_lcy_dyn 	,total_value_usd_dyn 	,total_remaining_commitment_dyn 	,perc_drawn_sec_dyn 	,remaining_commitment_lcy_dyn 	,commitment_lcy_dyn 	,total_value_lcy_dyn 	,contribution_net_of_recallable_lcy_dyn 	,prior_year_total_value_usd_dyn 	,prior_year_contribution_net_of_recallable_usd_dyn 	,gn_investee_fund_size_lcy 	,gn_investee_fund_size 	,valuation_type_gg 	,valuation_type_gn 	,gg_nav_dyn 	,gg_cash_returned_dyn 	,gg_cash_invested_dyn 	,nn_distribution_dyn 	,nn_nav_dyn 	,nn_contribution_dyn 	,nn_dpi_dyn 	,af_inv.re_up 	,investments_sp 	,investment_type_sp
		,gn_distribution_lcy_dyn
		,gn_recallable_lcy_dyn 
		,gn_contribution_lcy_dyn 
		FROM ( 	SELECT * 	FROM CTE_af_inv_base af_inv 	WHERE af_inv.dim_axiomfund_id IN ( 			SELECT dim_axiomfund_id 			FROM CTE_affam 			) 	) af_inv LEFT JOIN CTE_af_inv_base_excluded_family af_inv_per ON af_inv.axiom_fund_name = af_inv_per.axiomfundfamily_name_derived 	AND af_inv.investments = af_inv_per.investments 	AND af_inv.investment_type = af_inv_per.investment_type ) a
		GROUP BY fof ,investments ,investment_type
		)
		
	,CTE_final_fof AS (
		SELECT investments ,axiom_fund_name ,fof ,axiomfund_investment_iqid ,vintage_year_of_fund ,geography_gn_analysis ,geography_one_pager ,original_vehicle_size_usd ,original_vehicle_size_lcy ,strategy_for_gn_analysis ,strategy_one_pager ,total_fund_size_lcy ,total_fund_size_usd ,vintage_cambridge ,axiom_was_a_sponsor ,discount_perc_to_Nav ,names_of_investment_lead ,names_of_lp_advisory ,initial_investment_date ,investment_currency ,liquidation_exit_date ,original_commitment_lcy ,purchase_price_sec ,board_seat_fund_only ,fund_size_sec ,investment_status ,investment_type ,next_gen ,over_subscribed ,pool ,qr_name ,soi_name ,vintage_year ,gp ,no_of_prior_manager_funds_committed_to ,co_investment_offered ,axiom_commitment ,axiom_commitment_lcy ,axiomfundfamily_name_derived ,dim_blocker_vehicle_id ,perc_drawn_lcy ,perc_drawn_sec ,perc_of_fund_lcy_alloc_rule ,perc_of_fund_lcy_one_pager ,contribution_usd ,contribution_net_of_recallable_usd ,distribution_usd ,distribution_net_of_recallable_usd ,gn_irr_lcy ,gn_irr_usd ,gn_moc_lcy ,nav_usd ,nav_lcy ,prior_year_gn_moc_usd ,recallable_usd ,remaining_commitment_usd ,total_value_usd ,total_remaining_commitment ,af_inv_per_as_of_date ,af_inv_per_report_type ,investment_status_fund_derived ,perc_of_fund_lcy ,commitment_usd ,gn_irr_usd_dyn ,gn_irr_lcy_dyn ,commitment_usd_dyn ,remaining_commitment_usd_dyn ,contribution_usd_dyn ,distribution_usd_dyn ,recallable_usd_dyn ,contribution_net_of_recallable_usd_dyn ,distribution_net_of_recallable_usd_dyn ,nav_usd_dyn ,nav_lcy_dyn ,total_value_usd_dyn ,total_remaining_commitment_dyn ,perc_drawn_sec_dyn ,remaining_commitment_lcy_dyn ,commitment_lcy_dyn ,total_value_lcy_dyn ,contribution_net_of_recallable_lcy_dyn ,prior_year_total_value_usd_dyn ,prior_year_contribution_net_of_recallable_usd_dyn ,gn_investee_fund_size_lcy ,gn_investee_fund_size ,perc_drawn_usd ,gn_moc_usd ,dpi_usd ,perc_drawn_usd_dyn ,perc_drawn_lcy_dyn ,dpi_usd_dyn ,gn_moc_usd_dyn 
		--,gn_moc_lcy_dyn 
		,prior_year_gn_moc_usd_dyn ,valuation_type_gg ,valuation_type_gn ,gg_nav_dyn ,gg_cash_returned_dyn ,gg_cash_invested_dyn ,gg_moc_dyn ,nn_distribution_dyn ,nn_nav_dyn ,nn_contribution_dyn ,nn_moc_dyn ,nn_dpi_dyn ,re_up ,investments_sp ,investment_type_sp,gn_distribution_lcy_dyn
		,gn_recallable_lcy_dyn 
		,gn_contribution_lcy_dyn 
		FROM CTE_af_inv_base_excluded_family
		WHERE investments NOT IN ( 	SELECT investments 	FROM CTE_comma_separated 	)
		
		UNION
		
		SELECT investments ,axiom_fund_name ,fof ,axiomfund_investment_iqid ,vintage_year_of_fund ,geography_gn_analysis ,geography_one_pager ,original_vehicle_size_usd ,original_vehicle_size_lcy ,strategy_for_gn_analysis ,strategy_one_pager ,total_fund_size_lcy ,total_fund_size_usd ,vintage_cambridge ,axiom_was_a_sponsor ,discount_perc_to_Nav ,names_of_investment_lead ,names_of_lp_advisory ,initial_investment_date ,investment_currency ,liquidation_exit_date ,original_commitment_lcy ,purchase_price_sec ,board_seat_fund_only ,fund_size_sec ,investment_status ,investment_type ,next_gen ,over_subscribed ,pool ,qr_name ,soi_name ,vintage_year ,gp ,no_of_prior_manager_funds_committed_to ,co_investment_offered ,axiom_commitment ,axiom_commitment_lcy ,axiomfundfamily_name_derived ,dim_blocker_vehicle_id ,perc_drawn_lcy ,perc_drawn_sec ,perc_of_fund_lcy_alloc_rule ,perc_of_fund_lcy_one_pager ,contribution_usd ,contribution_net_of_recallable_usd ,distribution_usd ,distribution_net_of_recallable_usd ,gn_irr_lcy ,gn_irr_usd ,gn_moc_lcy ,nav_usd ,nav_lcy ,prior_year_gn_moc_usd ,recallable_usd ,remaining_commitment_usd ,total_value_usd ,total_remaining_commitment ,af_inv_per_as_of_date ,af_inv_per_report_type ,investment_status_fund_derived ,perc_of_fund_lcy ,commitment_usd ,gn_irr_usd_dyn ,gn_irr_lcy_dyn ,commitment_usd_dyn ,remaining_commitment_usd_dyn ,contribution_usd_dyn ,distribution_usd_dyn ,recallable_usd_dyn ,contribution_net_of_recallable_usd_dyn ,distribution_net_of_recallable_usd_dyn ,nav_usd_dyn ,nav_lcy_dyn ,total_value_usd_dyn ,total_remaining_commitment_dyn ,perc_drawn_sec_dyn ,remaining_commitment_lcy_dyn ,commitment_lcy_dyn ,total_value_lcy_dyn ,contribution_net_of_recallable_lcy_dyn ,prior_year_total_value_usd_dyn ,prior_year_contribution_net_of_recallable_usd_dyn ,gn_investee_fund_size_lcy ,gn_investee_fund_size ,perc_drawn_usd ,gn_moc_usd ,dpi_usd ,perc_drawn_usd_dyn ,perc_drawn_lcy_dyn ,dpi_usd_dyn ,gn_moc_usd_dyn 
		--,gn_moc_lcy_dyn 
		,prior_year_gn_moc_usd_dyn ,valuation_type_gg ,valuation_type_gn ,gg_nav_dyn ,gg_cash_returned_dyn ,gg_cash_invested_dyn ,gg_moc_dyn ,nn_distribution_dyn ,nn_nav_dyn ,nn_contribution_dyn ,nn_moc_dyn ,nn_dpi_dyn ,re_up ,NULL AS investments_sp ,NULL AS investment_type_sp,gn_distribution_lcy_dyn
		,gn_recallable_lcy_dyn 
		,gn_contribution_lcy_dyn 
		FROM CTE_comma_separated_records
		
		UNION
		
		SELECT *
		FROM CTE_af_inv_base_only_family
		)
	,CTE_final AS (
		SELECT fof ,min(axiom_fund_name) AS axiom_fund_name ,min(axiomfund_investment_iqid) AS axiomfund_investment_iqid ,MIN(vintage_year_of_fund) AS vintage_year_of_fund ,investments ,MIN(qr_name) AS qr_name ,MIN(soi_name) AS soi_name ,MIN(gp) AS gp ,investment_type ,MIN(pool) AS pool ,MIN(investment_currency) AS investment_currency ,min(total_fund_size_lcy) AS total_fund_size_lcy ,min(total_fund_size_usd) AS total_fund_size_usd ,COALESCE(SUM(perc_of_fund_lcy_alloc_rule), MIN(perc_of_fund_lcy_alloc_rule)) AS perc_of_fund_lcy_alloc_rule ,COALESCE(SUM(perc_of_fund_lcy_one_pager), MIN(perc_of_fund_lcy_one_pager)) AS perc_of_fund_lcy_one_pager ,MIN(geography_one_pager) AS geography_one_pager ,MIN(geography_gn_analysis) AS geography_gn_analysis ,MIN(strategy_one_pager) AS strategy_one_pager ,MIN(strategy_for_gn_analysis) AS strategy_for_gn_analysis ,min(vintage_year) AS vintage_year ,min(vintage_cambridge) AS vintage_cambridge ,min(initial_investment_date) AS initial_investment_date ,min(liquidation_exit_date) AS liquidation_exit_date ,MIN(investment_status) AS investment_status ,MIN(investment_status_fund_derived) AS investment_status_fund_derived ,MIN(no_of_prior_manager_funds_committed_to) AS no_of_prior_manager_funds_committed_to ,MIN(names_of_lp_advisory) AS names_of_lp_advisory ,MIN(names_of_investment_lead) AS names_of_investment_lead ,MIN(axiom_was_a_sponsor) AS axiom_was_a_sponsor ,MIN(board_seat_fund_only) AS board_seat_fund_only ,MIN(next_gen) AS next_gen ,MIN(over_subscribed) AS over_subscribed ,MIN(co_investment_offered) AS co_investment_offered ,MIN(discount_perc_to_nav) AS discount_perc_to_nav ,sum(commitment_usd) AS commitment_usd ,SUM(contribution_usd) AS contribution_usd ,SUM(distribution_usd) AS distribution_usd ,SUM(recallable_usd) AS recallable_usd ,sum(distribution_net_of_recallable_usd) AS distribution_net_of_recallable_usd ,sum(contribution_net_of_recallable_usd) AS contribution_net_of_recallable_usd ,sum(remaining_commitment_usd) AS remaining_commitment_usd ,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 			perc_drawn_lcy 			,'%' 			))) AS perc_drawn_lcy ,sum(cast(nav_usd AS DOUBLE)) AS nav_usd ,SUM(nav_lcy) AS nav_lcy ,sum(total_value_usd) AS total_value_usd ,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 			gn_irr_usd 			,'%' 			))) AS gn_irr_usd ,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 			gn_irr_lcy 			,'%' 			))) AS gn_irr_lcy ,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 			gn_moc_lcy 			,'x' 			))) AS gn_moc_lcy ,sum(original_commitment_lcy) AS original_commitment_lcy ,MIN(fund_size_sec) AS fund_size_sec ,CASE  	WHEN ARRAY_CONTAINS(COLLECT_LIST(CAST(perc_drawn_sec AS STRING)), 'N/A') 		THEN 'N/A' 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					perc_drawn_sec 					,'%' 					))) 	END AS perc_drawn_sec ,sum(purchase_price_sec) AS purchase_price_sec ,CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 			prior_year_gn_moc_usd 			,'x' 			))) AS prior_year_gn_moc_usd ,MIN(af_inv_per_as_of_date) AS af_inv_per_as_of_date ,MIN(af_inv_per_report_type) AS af_inv_per_report_type ,MIN(gn_irr_usd_dyn) AS gn_irr_usd_dyn ,MIN(gn_irr_lcy_dyn) AS gn_irr_lcy_dyn ,sum(commitment_usd_dyn) AS commitment_usd_dyn ,SUM(remaining_commitment_usd_dyn) AS remaining_commitment_usd_dyn ,SUM(contribution_usd_dyn) AS contribution_usd_dyn ,SUM(distribution_usd_dyn) AS distribution_usd_dyn ,SUM(recallable_usd_dyn) AS recallable_usd_dyn ,SUM(contribution_net_of_recallable_usd_dyn) AS contribution_net_of_recallable_usd_dyn ,SUM(distribution_net_of_recallable_usd_dyn) AS distribution_net_of_recallable_usd_dyn ,SUM(nav_usd_dyn) AS nav_usd_dyn ,SUM(nav_lcy_dyn) AS nav_lcy_dyn ,SUM(total_value_usd_dyn) AS total_value_usd_dyn ,MIN(total_remaining_commitment_dyn) AS total_remaining_commitment_dyn ,COALESCE(MIN(perc_drawn_sec_dyn), (((SUM(remaining_commitment_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100)) AS perc_drawn_sec_dyn ,SUM(remaining_commitment_lcy_dyn) AS remaining_commitment_lcy_dyn ,SUM(commitment_lcy_dyn) AS commitment_lcy_dyn ,SUM(total_value_lcy_dyn) AS total_value_lcy_dyn ,SUM(contribution_net_of_recallable_lcy_dyn) AS contribution_net_of_recallable_lcy_dyn ,SUM(prior_year_total_value_usd_dyn) AS prior_year_total_value_usd_dyn ,SUM(prior_year_contribution_net_of_recallable_usd_dyn) AS prior_year_contribution_net_of_recallable_usd_dyn ,SUM(gn_investee_fund_size_lcy) AS gn_investee_fund_size_lcy ,SUM(gn_investee_fund_size) AS gn_investee_fund_size ,MIN(dim_blocker_vehicle_id) AS dim_blocker_vehicle_id ,(((sum(contribution_net_of_recallable_usd) / sum(commitment_usd))) * 100) AS perc_drawn_usd ,(sum(total_value_usd) / sum(contribution_net_of_recallable_usd)) AS gn_moc_usd ,(sum(distribution_net_of_recallable_usd) / sum(contribution_net_of_recallable_usd)) AS dpi_usd ,(((SUM(contribution_net_of_recallable_usd_dyn) / SUM(commitment_usd_dyn))) * 100) AS perc_drawn_usd_dyn ,(((SUM(contribution_net_of_recallable_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100) AS perc_drawn_lcy_dyn ,(SUM(distribution_net_of_recallable_usd_dyn) / SUM(contribution_net_of_recallable_usd_dyn)) AS dpi_usd_dyn ,(SUM(total_value_usd_dyn) / SUM(contribution_net_of_recallable_usd_dyn)) AS gn_moc_usd_dyn 
		
		--,(SUM(total_value_lcy_dyn) / SUM(contribution_net_of_recallable_lcy_dyn)) AS gn_moc_lcy_dyn 
		,(SUM(gn_distribution_lcy_dyn) - SUM(gn_recallable_lcy_dyn) + SUM(nav_usd_dyn) )/(SUM(gn_contribution_lcy_dyn) -  SUM(gn_recallable_lcy_dyn)) AS gn_moc_lcy_dyn
		
		,sum(prior_year_total_value_usd_dyn) / sum(prior_year_contribution_net_of_recallable_usd_dyn) AS prior_year_gn_moc_usd_dyn ,MIN(original_vehicle_size_usd) AS original_vehicle_size_usd ,MIN(original_vehicle_size_lcy) AS original_vehicle_size_lcy ,min(valuation_type_gg) AS valuation_type_gg ,min(valuation_type_gn) AS valuation_type_gn ,SUM(gg_nav_dyn) AS gg_nav_dyn ,SUM(gg_cash_returned_dyn) AS gg_cash_returned_dyn ,SUM(gg_cash_invested_dyn) AS gg_cash_invested_dyn ,((SUM(gg_nav_dyn) + SUM(gg_cash_returned_dyn)) / SUM(gg_cash_invested_dyn)) AS gg_moc_dyn ,SUM(nn_distribution_dyn) AS nn_distribution_dyn ,SUM(nn_nav_dyn) AS nn_nav_dyn ,SUM(nn_contribution_dyn) AS nn_contribution_dyn ,((SUM(nn_distribution_dyn) + SUM(nn_nav_dyn)) / SUM(nn_contribution_dyn)) AS nn_moc_dyn ,(SUM(nn_distribution_dyn) / SUM(nn_contribution_dyn)) AS nn_dpi_dyn ,min(re_up) AS re_up ,min(investments_sp) AS investments_sp ,min(investment_type_sp) AS investment_type_sp
		,SUM(gn_distribution_lcy_dyn) AS gn_distribution_lcy_dyn
		,SUM(gn_recallable_lcy_dyn) AS gn_recallable_lcy_dyn
		,SUM(gn_contribution_lcy_dyn) AS gn_contribution_lcy_dyn
		
		FROM CTE_final_fof
		GROUP BY fof ,investments ,investment_type
		)
		
		
		
		
		
		
		
	,CTE_query_1 AS (
		SELECT fof ,axiom_fund_name ,axiomfund_investment_iqid ,vintage_year_of_fund ,coalesce(sh2.revised_investments, base.investments) AS investments ,coalesce(sh2.revised_qr_name, qr_name) AS qr_name ,coalesce(sh2.revised_Soi_name, soi_name) AS soi_name ,coalesce(sh2.revised_gp, gp) AS gp ,gp_um AS gp_um ,coalesce(sh2.revised_investment_type, investment_type) AS investment_type ,pool ,investment_currency ,coalesce(sh2.revised_total_fund_size_lcy, total_fund_size_lcy) AS total_fund_size_lcy ,total_fund_size_lcy_um AS total_fund_size_lcy_um ,coalesce(sh2.revised_total_fund_size_usd, total_fund_size_usd) AS total_fund_size_usd ,total_fund_size_usd_um AS total_fund_size_usd_um ,coalesce(sh2.revised_original_vehicle_size_usd, original_vehicle_size_usd) AS original_vehicle_size_usd ,original_vehicle_size_usd_um AS original_vehicle_size_usd_um ,coalesce(sh2.revised_original_vehicle_size_lcy, original_vehicle_size_lcy) AS original_vehicle_size_lcy ,original_vehicle_size_lcy_um AS original_vehicle_size_lcy_um ,coalesce(sh2.revised_perc_of_fund_lcy_one_pager, perc_of_fund_lcy_one_pager) AS perc_of_fund_lcy_one_pager ,coalesce(sh2.revised_perc_of_fund_lcy_alloc_rule, perc_of_fund_lcy_alloc_rule) AS perc_of_fund_lcy_alloc_rule ,coalesce(sh2.revised_geography_one_pager, geography_one_pager) AS geography_one_pager ,geography_gn_analysis ,strategy_one_pager ,strategy_for_gn_analysis ,vintage_year ,coalesce(sh2.revised_vintage_cambridge, vintage_cambridge) AS vintage_cambridge ,initial_investment_date ,liquidation_exit_date ,investment_status ,investment_status_fund_derived ,coalesce(sh2.revised_no_of_prior_manager_funds_committed_to, no_of_prior_manager_funds_committed_to) AS no_of_prior_manager_funds_committed_to ,no_of_prior_manager_funds_committed_to_um AS no_of_prior_manager_funds_committed_to_um ,coalesce(sh2.revised_names_of_lp_advisory, names_of_lp_advisory) AS names_of_lp_advisory ,names_of_investment_lead ,coalesce(sh2.revised_axiom_was_a_sponsor, axiom_was_a_sponsor) AS axiom_was_a_sponsor ,coalesce(sh2.revised_board_seat_fund_only, board_seat_fund_only) AS board_seat_fund_only ,board_seat_fund_only_um AS board_seat_fund_only_um ,coalesce(sh2.revised_next_gen, next_gen) AS next_gen ,next_gen_um AS next_gen_um ,coalesce(sh2.revised_over_subscribed, over_subscribed) AS over_subscribed ,coalesce(sh2.revised_co_investment_offered, co_investment_offered) AS co_investment_offered ,coalesce(sh2.revised_discount_perc_to_nav, discount_perc_to_nav) AS discount_perc_to_nav ,commitment_usd ,contribution_usd ,distribution_usd ,recallable_usd ,coalesce(sh2.revised_distribution_net_of_recallable_usd, distribution_net_of_recallable_usd) AS distribution_net_of_recallable_usd ,distribution_net_of_recallable_usd_um AS distribution_net_of_recallable_usd_um ,coalesce(sh2.revised_contribution_net_of_recallable_usd, contribution_net_of_recallable_usd) AS contribution_net_of_recallable_usd ,contribution_net_of_recallable_usd_um AS contribution_net_of_recallable_usd_um ,coalesce(sh2.revised_remaining_commitment_usd, remaining_commitment_usd) AS remaining_commitment_usd ,remaining_commitment_usd_um AS remaining_commitment_usd_um ,coalesce(sh2.revised_perc_drawn_usd, perc_drawn_usd) AS perc_drawn_usd ,perc_drawn_usd_um AS perc_drawn_usd_um ,coalesce(sh2.revised_perc_drawn_lcy, perc_drawn_lcy) AS perc_drawn_lcy ,perc_drawn_lcy_um AS perc_drawn_lcy_um ,coalesce(sh2.revised_nav_usd, nav_usd) AS nav_usd ,nav_usd_um AS nav_usd_um ,nav_lcy ,coalesce(sh2.revised_total_value_usd, total_value_usd) AS total_value_usd ,total_value_usd AS total_value_usd_um ,coalesce(sh2.revised_gn_irr_usd, gn_irr_usd) AS gn_irr_usd ,gn_irr_usd_um AS gn_irr_usd_um ,coalesce(sh2.revised_gn_moc_usd, gn_moc_usd) AS gn_moc_usd ,gn_moc_usd_um AS gn_moc_usd_um ,coalesce(sh2.revised_gn_irr_lcy, gn_irr_lcy) AS gn_irr_lcy ,gn_irr_lcy_um AS gn_irr_lcy_um ,coalesce(sh2.revised_gn_moc_lcy, gn_moc_lcy) AS gn_moc_lcy ,gn_moc_lcy_um ,coalesce(sh2.revised_dpi_usd, dpi_usd) AS dpi_usd ,dpi_usd_um AS dpi_usd_um ,coalesce(sh2.revised_original_commitment_lcy, original_commitment_lcy) AS original_commitment_lcy ,original_commitment_lcy_um AS original_commitment_lcy_um ,coalesce(sh2.revised_fund_size_sec, fund_size_sec) AS fund_size_sec ,coalesce(sh2.revised_perc_drawn_Sec, perc_drawn_sec) AS perc_drawn_sec ,purchase_price_sec ,coalesce(sh2.revised_prior_year_gn_moc_usd, prior_year_gn_moc_usd) AS prior_year_gn_moc_usd ,prior_year_gn_moc_usd_um AS prior_year_gn_moc_usd_um ,af_inv_per_as_of_date ,af_inv_per_report_type ,gn_irr_usd_dyn ,gn_irr_usd_dyn_um AS gn_irr_usd_dyn_um ,gn_irr_lcy_dyn ,gn_irr_lcy_dyn_um AS gn_irr_lcy_dyn_um ,commitment_usd_dyn ,remaining_commitment_usd_dyn ,remaining_commitment_usd_dyn_um AS remaining_commitment_usd_dyn_um ,contribution_usd_dyn ,distribution_usd_dyn ,recallable_usd_dyn ,contribution_net_of_recallable_usd_dyn ,contribution_net_of_recallable_usd_dyn_um AS contribution_net_of_recallable_usd_dyn_um ,distribution_net_of_recallable_usd_dyn ,distribution_net_of_recallable_usd_dyn_um AS distribution_net_of_recallable_usd_dyn_um ,perc_drawn_usd_dyn ,perc_drawn_usd_dyn_um AS perc_drawn_usd_dyn_um ,perc_drawn_lcy_dyn ,perc_drawn_lcy_dyn_um AS perc_drawn_lcy_dyn_um ,nav_usd_dyn ,nav_usd_dyn_um AS nav_usd_dyn_um ,nav_lcy_dyn ,total_value_usd_dyn ,total_value_usd_dyn_um AS total_value_usd_dyn_um ,dpi_usd_dyn ,dpi_usd_dyn_um AS dpi_usd_dyn_um ,gn_moc_usd_dyn ,gn_moc_usd_dyn_um AS gn_moc_usd_dyn_um ,prior_year_gn_moc_usd_dyn ,prior_year_gn_moc_usd_dyn_um AS prior_year_gn_moc_usd_dyn_um ,gn_moc_lcy_dyn ,gn_moc_lcy_dyn_um ,dim_blocker_vehicle_id ,perc_drawn_sec_dyn ,perc_drawn_sec_dyn_um AS perc_drawn_sec_dyn_um ,remaining_commitment_lcy_dyn ,commitment_lcy_dyn ,total_value_lcy_dyn ,contribution_net_of_recallable_lcy_dyn ,prior_year_total_value_usd_dyn ,prior_year_contribution_net_of_recallable_usd_dyn ,gn_investee_fund_size_lcy ,gn_investee_fund_size 
		,coalesce(sh2.investments_um, investments_um2) AS investments_um3,valuation_type_gg ,valuation_type_gn ,gg_nav_dyn ,gg_cash_returned_dyn ,gg_cash_invested_dyn ,gg_moc_dyn ,gg_moc_dyn_um ,nn_distribution_dyn ,nn_nav_dyn ,nn_contribution_dyn ,nn_moc_dyn ,nn_moc_dyn_um ,nn_dpi_dyn ,nn_dpi_dyn_um ,coalesce(sh2.revised_re_up, re_up) AS re_up ,re_up_um AS re_up_um ,investments_sp ,investment_type_sp
		,gn_distribution_lcy_dyn,gn_recallable_lcy_dyn,gn_contribution_lcy_dyn
		
		FROM ( SELECT fof 	,axiom_fund_name 	,axiomfund_investment_iqid 	,vintage_year_of_fund 	,coalesce(sh1.revised_investments, base.investments) AS investments 	,coalesce(sh1.revised_qr_name, qr_name) AS qr_name 	,coalesce(sh1.revised_Soi_name, soi_name) AS soi_name 	,coalesce(sh1.revised_gp, gp) AS gp 	,gp_um AS gp_um 	,coalesce(sh1.revised_investment_type, investment_type) AS investment_type 	,pool 	,investment_currency 	,coalesce(sh1.revised_total_fund_size_lcy, total_fund_size_lcy) AS total_fund_size_lcy 	,total_fund_size_lcy_um AS total_fund_size_lcy_um 	,coalesce(sh1.revised_total_fund_size_usd, total_fund_size_usd) AS total_fund_size_usd 	,total_fund_size_usd_um AS total_fund_size_usd_um 	,coalesce(sh1.revised_original_vehicle_size_usd, original_vehicle_size_usd) AS original_vehicle_size_usd 	,original_vehicle_size_usd_um AS original_vehicle_size_usd_um 	,coalesce(sh1.revised_original_vehicle_size_lcy, original_vehicle_size_lcy) AS original_vehicle_size_lcy 	,original_vehicle_size_lcy_um AS original_vehicle_size_lcy_um 	,coalesce(sh1.revised_perc_of_fund_lcy_one_pager, perc_of_fund_lcy_one_pager) AS perc_of_fund_lcy_one_pager 	,coalesce(sh1.revised_perc_of_fund_lcy_alloc_rule, perc_of_fund_lcy_alloc_rule) AS perc_of_fund_lcy_alloc_rule 	,coalesce(sh1.revised_geography_one_pager, geography_one_pager) AS geography_one_pager 	,geography_gn_analysis 	,strategy_one_pager 	,strategy_for_gn_analysis 	,vintage_year 	,coalesce(sh1.revised_vintage_cambridge, vintage_cambridge) AS vintage_cambridge 	,initial_investment_date 	,liquidation_exit_date 	,investment_status 	,investment_status_fund_derived 	,coalesce(sh1.revised_no_of_prior_manager_funds_committed_to, no_of_prior_manager_funds_committed_to) AS no_of_prior_manager_funds_committed_to 	,no_of_prior_manager_funds_committed_to_um AS no_of_prior_manager_funds_committed_to_um 	,coalesce(sh1.revised_names_of_lp_advisory, names_of_lp_advisory) AS names_of_lp_advisory 	,names_of_investment_lead 	,coalesce(sh1.revised_axiom_was_a_sponsor, axiom_was_a_sponsor) AS axiom_was_a_sponsor 	,coalesce(sh1.revised_board_seat_fund_only, board_seat_fund_only) AS board_seat_fund_only 	,board_seat_fund_only_um AS board_seat_fund_only_um 	,coalesce(sh1.revised_next_gen, next_gen) AS next_gen 	,next_gen_um AS next_gen_um 	,coalesce(sh1.revised_over_subscribed, over_subscribed) AS over_subscribed 	,coalesce(sh1.revised_co_investment_offered, co_investment_offered) AS co_investment_offered 	,coalesce(sh1.revised_discount_perc_to_nav, discount_perc_to_nav) AS discount_perc_to_nav 	,commitment_usd 	,contribution_usd 	,distribution_usd 	,recallable_usd 	,coalesce(sh1.revised_distribution_net_of_recallable_usd, distribution_net_of_recallable_usd) AS distribution_net_of_recallable_usd 	,distribution_net_of_recallable_usd_um AS distribution_net_of_recallable_usd_um 	,coalesce(sh1.revised_contribution_net_of_recallable_usd, contribution_net_of_recallable_usd) AS contribution_net_of_recallable_usd 	,contribution_net_of_recallable_usd_um AS contribution_net_of_recallable_usd_um 	,coalesce(sh1.revised_remaining_commitment_usd, remaining_commitment_usd) AS remaining_commitment_usd 	,remaining_commitment_usd_um AS remaining_commitment_usd_um 	,coalesce(sh1.revised_perc_drawn_usd, perc_drawn_usd) AS perc_drawn_usd 	,perc_drawn_usd_um AS perc_drawn_usd_um 	,coalesce(sh1.revised_perc_drawn_lcy, perc_drawn_lcy) AS perc_drawn_lcy 	,perc_drawn_lcy_um AS perc_drawn_lcy_um 	,coalesce(sh1.revised_nav_usd, nav_usd) AS nav_usd 	,nav_usd_um AS nav_usd_um 	,nav_lcy 	,coalesce(sh1.revised_total_value_usd, total_value_usd) AS total_value_usd 	,total_value_usd_um AS total_value_usd_um 	,coalesce(sh1.revised_gn_irr_usd, gn_irr_usd) AS gn_irr_usd 	,gn_irr_usd_um AS gn_irr_usd_um 	,coalesce(sh1.revised_gn_moc_usd, gn_moc_usd) AS gn_moc_usd 	,gn_moc_usd_um AS gn_moc_usd_um 	,coalesce(sh1.revised_gn_irr_lcy, gn_irr_lcy) AS gn_irr_lcy 	,gn_irr_lcy_um AS gn_irr_lcy_um 	,coalesce(sh1.revised_gn_moc_lcy, gn_moc_lcy) AS gn_moc_lcy 	,gn_moc_lcy_um 	,coalesce(sh1.revised_dpi_usd, dpi_usd) AS dpi_usd 	,dpi_usd_um AS dpi_usd_um 	,coalesce(sh1.revised_original_commitment_lcy, original_commitment_lcy) AS original_commitment_lcy 	,original_commitment_lcy_um AS original_commitment_lcy_um 	,coalesce(sh1.revised_fund_size_sec, fund_size_sec) AS fund_size_sec 	,coalesce(sh1.revised_perc_drawn_Sec, perc_drawn_sec) AS perc_drawn_sec 	,purchase_price_sec 	,coalesce(sh1.revised_prior_year_gn_moc_usd, prior_year_gn_moc_usd) AS prior_year_gn_moc_usd 	,prior_year_gn_moc_usd_um AS prior_year_gn_moc_usd_um 	,af_inv_per_as_of_date 	,af_inv_per_report_type 	,gn_irr_usd_dyn 	,gn_irr_usd_dyn_um AS gn_irr_usd_dyn_um 	,gn_irr_lcy_dyn 	,gn_irr_lcy_dyn_um AS gn_irr_lcy_dyn_um 	,commitment_usd_dyn 	,remaining_commitment_usd_dyn 	,remaining_commitment_usd_dyn_um AS remaining_commitment_usd_dyn_um 	,contribution_usd_dyn 	,distribution_usd_dyn 	,recallable_usd_dyn 	,contribution_net_of_recallable_usd_dyn 	,contribution_net_of_recallable_usd_dyn_um AS contribution_net_of_recallable_usd_dyn_um 	,distribution_net_of_recallable_usd_dyn 	,distribution_net_of_recallable_usd_dyn_um AS distribution_net_of_recallable_usd_dyn_um 	,perc_drawn_usd_dyn 	,perc_drawn_usd_dyn_um AS perc_drawn_usd_dyn_um 	,perc_drawn_lcy_dyn 	,perc_drawn_lcy_dyn_um AS perc_drawn_lcy_dyn_um 	,nav_usd_dyn 	,nav_usd_dyn_um AS nav_usd_dyn_um 	,nav_lcy_dyn 	,total_value_usd_dyn 	,total_value_usd_dyn_um AS total_value_usd_dyn_um 	,dpi_usd_dyn 	,dpi_usd_dyn_um AS dpi_usd_dyn_um 	,gn_moc_usd_dyn 	,gn_moc_usd_dyn_um AS gn_moc_usd_dyn_um 	,prior_year_gn_moc_usd_dyn 	,prior_year_gn_moc_usd_dyn_um AS prior_year_gn_moc_usd_dyn_um 	,gn_moc_lcy_dyn 	,gn_moc_lcy_dyn_um 	,dim_blocker_vehicle_id 	,perc_drawn_sec_dyn 	,perc_drawn_sec_dyn AS perc_drawn_sec_dyn_um 	,remaining_commitment_lcy_dyn 	,commitment_lcy_dyn 	,total_value_lcy_dyn 	,contribution_net_of_recallable_lcy_dyn 	,prior_year_total_value_usd_dyn 	,prior_year_contribution_net_of_recallable_usd_dyn 	,gn_investee_fund_size_lcy 	,gn_investee_fund_size 	,coalesce(sh1.investments_um, investments_um1) AS investments_um2 	,valuation_type_gg 	,valuation_type_gn 	,gg_nav_dyn 	,gg_cash_returned_dyn 	,gg_cash_invested_dyn 	,gg_moc_dyn 	,gg_moc_dyn_um 	,nn_distribution_dyn 	,nn_nav_dyn 	,nn_contribution_dyn 	,nn_moc_dyn 	,nn_moc_dyn_um 	,nn_dpi_dyn 	,nn_dpi_dyn_um 	,coalesce(sh1.revised_re_up, re_up) AS re_up 	,re_up_um AS re_up_um 	,investments_sp 	,investment_type_sp
		,gn_distribution_lcy_dyn,gn_recallable_lcy_dyn,gn_contribution_lcy_dyn
		FROM ( 	SELECT fof 		,axiom_fund_name 		,axiomfund_investment_iqid 		,vintage_year_of_fund 		,coalesce(revised_investments, base.investments) AS investments 		,coalesce(revised_qr_name, qr_name) AS qr_name 		,coalesce(revised_Soi_name, soi_name) AS soi_name 		,coalesce(revised_gp, gp) AS gp 		,gp AS gp_um 		,coalesce(revised_investment_type, investment_type) AS investment_type 		,pool 		,investment_currency 		,coalesce(revised_total_fund_size_lcy, total_fund_size_lcy) AS total_fund_size_lcy 		,total_fund_size_lcy AS total_fund_size_lcy_um 		,coalesce(revised_total_fund_size_usd, total_fund_size_usd) AS total_fund_size_usd 		,total_fund_size_usd AS total_fund_size_usd_um 		,coalesce(revised_original_vehicle_size_usd, original_vehicle_size_usd) AS original_vehicle_size_usd 		,original_vehicle_size_usd AS original_vehicle_size_usd_um 		,coalesce(revised_original_vehicle_size_lcy, original_vehicle_size_lcy) AS original_vehicle_size_lcy 		,original_vehicle_size_lcy AS original_vehicle_size_lcy_um 		,coalesce(revised_perc_of_fund_lcy_one_pager, perc_of_fund_lcy_one_pager) AS perc_of_fund_lcy_one_pager 		,coalesce(revised_perc_of_fund_lcy_alloc_rule, perc_of_fund_lcy_alloc_rule) AS perc_of_fund_lcy_alloc_rule 		,coalesce(revised_geography_one_pager, geography_one_pager) AS geography_one_pager 		,geography_gn_analysis 		,strategy_one_pager 		,strategy_for_gn_analysis 		,vintage_year 		,coalesce(revised_vintage_cambridge, vintage_cambridge) AS vintage_cambridge 		,initial_investment_date 		,liquidation_exit_date 		,investment_status 		,investment_status_fund_derived 		,coalesce(revised_no_of_prior_manager_funds_committed_to, no_of_prior_manager_funds_committed_to) AS no_of_prior_manager_funds_committed_to 		,no_of_prior_manager_funds_committed_to AS no_of_prior_manager_funds_committed_to_um 		,coalesce(revised_names_of_lp_advisory, names_of_lp_advisory) AS names_of_lp_advisory 		,names_of_investment_lead 		,coalesce(revised_axiom_was_a_sponsor, axiom_was_a_sponsor) AS axiom_was_a_sponsor 		,coalesce(revised_board_seat_fund_only, board_seat_fund_only) AS board_seat_fund_only 		,board_seat_fund_only AS board_seat_fund_only_um 		,coalesce(revised_next_gen, next_gen) AS next_gen 		,next_gen AS next_gen_um 		,coalesce(revised_over_subscribed, over_subscribed) AS over_subscribed 		,coalesce(revised_co_investment_offered, co_investment_offered) AS co_investment_offered 		,coalesce(revised_discount_perc_to_nav, discount_perc_to_nav) AS discount_perc_to_nav 		,commitment_usd 		,contribution_usd 		,distribution_usd 		,recallable_usd 		,coalesce(revised_distribution_net_of_recallable_usd, distribution_net_of_recallable_usd) AS distribution_net_of_recallable_usd 		,distribution_net_of_recallable_usd AS distribution_net_of_recallable_usd_um 		,coalesce(revised_contribution_net_of_recallable_usd, contribution_net_of_recallable_usd) AS contribution_net_of_recallable_usd 		,contribution_net_of_recallable_usd AS contribution_net_of_recallable_usd_um 		,coalesce(revised_remaining_commitment_usd, remaining_commitment_usd) AS remaining_commitment_usd 		,remaining_commitment_usd AS remaining_commitment_usd_um 		,coalesce(revised_perc_drawn_usd, perc_drawn_usd) AS perc_drawn_usd 		,perc_drawn_usd AS perc_drawn_usd_um 		,coalesce(revised_perc_drawn_lcy, perc_drawn_lcy) AS perc_drawn_lcy 		,perc_drawn_lcy AS perc_drawn_lcy_um 		,coalesce(revised_nav_usd, nav_usd) AS nav_usd 		,nav_usd AS nav_usd_um 		,nav_lcy 		,coalesce(revised_total_value_usd, total_value_usd) AS total_value_usd 		,total_value_usd AS total_value_usd_um 		,coalesce(revised_gn_irr_usd, gn_irr_usd) AS gn_irr_usd 		,gn_irr_usd AS gn_irr_usd_um 		,coalesce(revised_gn_moc_usd, gn_moc_usd) AS gn_moc_usd 		,gn_moc_usd AS gn_moc_usd_um 		,gn_moc_lcy AS gn_moc_lcy_um 		,coalesce(revised_gn_irr_lcy, gn_irr_lcy) AS gn_irr_lcy 		,gn_irr_lcy AS gn_irr_lcy_um 		,coalesce(revised_gn_moc_lcy, gn_moc_lcy) AS gn_moc_lcy 		,gn_moc_lcy_um 		,coalesce(revised_dpi_usd, dpi_usd) AS dpi_usd 		,dpi_usd AS dpi_usd_um 		,coalesce(revised_fund_size_sec, fund_size_sec) AS fund_size_sec 		,coalesce(revised_perc_drawn_Sec, perc_drawn_sec) AS perc_drawn_sec 		,purchase_price_sec 		,coalesce(revised_prior_year_gn_moc_usd, prior_year_gn_moc_usd) AS prior_year_gn_moc_usd 		,prior_year_gn_moc_usd AS prior_year_gn_moc_usd_um 		,af_inv_per_as_of_date 		,af_inv_per_report_type 		,gn_irr_usd_dyn 		,gn_irr_usd_dyn / 100 AS gn_irr_usd_dyn_um 		,gn_irr_lcy_dyn 		,gn_irr_lcy_dyn / 100 AS gn_irr_lcy_dyn_um 		,commitment_usd_dyn 		,remaining_commitment_usd_dyn 		,remaining_commitment_usd_dyn AS remaining_commitment_usd_dyn_um 		,contribution_usd_dyn 		,distribution_usd_dyn 		,recallable_usd_dyn 		,contribution_net_of_recallable_usd_dyn 		,contribution_net_of_recallable_usd_dyn AS contribution_net_of_recallable_usd_dyn_um 		,distribution_net_of_recallable_usd_dyn 		,distribution_net_of_recallable_usd_dyn AS distribution_net_of_recallable_usd_dyn_um 		,perc_drawn_usd_dyn 		,perc_drawn_usd_dyn / 100 AS perc_drawn_usd_dyn_um 		,perc_drawn_lcy_dyn 		,perc_drawn_lcy_dyn / 100 AS perc_drawn_lcy_dyn_um 		,nav_usd_dyn 		,nav_usd_dyn AS nav_usd_dyn_um 		,nav_lcy_dyn 		,total_value_usd_dyn 		,total_value_usd_dyn AS total_value_usd_dyn_um 		,dpi_usd_dyn 		,dpi_usd_dyn AS dpi_usd_dyn_um 		,gn_moc_usd_dyn 		,gn_moc_usd_dyn AS gn_moc_usd_dyn_um 		,prior_year_gn_moc_usd_dyn 		,prior_year_gn_moc_usd_dyn AS prior_year_gn_moc_usd_dyn_um 		,gn_moc_lcy_dyn 		,gn_moc_lcy_dyn AS gn_moc_lcy_dyn_um 		,dim_blocker_vehicle_id 		,perc_drawn_sec_dyn 		,perc_drawn_sec_dyn / 100 AS perc_drawn_sec_dyn_um 		,remaining_commitment_lcy_dyn 		,commitment_lcy_dyn 		,total_value_lcy_dyn 		,contribution_net_of_recallable_lcy_dyn 		,prior_year_total_value_usd_dyn 		,prior_year_contribution_net_of_recallable_usd_dyn 		,gn_investee_fund_size_lcy 		,gn_investee_fund_size 		,coalesce(revised_original_commitment_lcy, original_commitment_lcy) AS original_commitment_lcy 		,original_commitment_lcy AS original_commitment_lcy_um 	,coalesce(investments_um, investments) AS investments_um1 	,valuation_type_gg 		,valuation_type_gn 		,gg_nav_dyn 		,gg_cash_returned_dyn 		,gg_cash_invested_dyn 		,gg_moc_dyn 		,gg_moc_dyn AS gg_moc_dyn_um 		,nn_distribution_dyn 		,nn_nav_dyn 		,nn_contribution_dyn 		,nn_moc_dyn 		,nn_moc_dyn AS nn_moc_dyn_um 		,nn_dpi_dyn 		,nn_dpi_dyn AS nn_dpi_dyn_um 		,coalesce(revised_re_up, re_up) AS re_up 		,re_up AS re_up_um 		,investments_sp 		,investment_type_sp 
,gn_distribution_lcy_dyn,gn_recallable_lcy_dyn,gn_contribution_lcy_dyn


		FROM CTE_final base 	LEFT JOIN ( 		SELECT * 		FROM axiom_datalake.cl_snapshot.master_ddq_sh 		WHERE condition_investments IS NULL 		) sh 		ON coalesce(base.investment_type_sp, base.investment_type) = sh.condition_investment_type 	) base LEFT JOIN ( 	SELECT * 		,CONCAT ( 			'%' 			,condition_investments 			,'%' 			) AS investment_pattern 	FROM axiom_datalake.cl_snapshot.master_ddq_sh 	WHERE condition_investment_type IS NULL 	) sh1 ON base.investments = sh1.condition_investments ) base
		LEFT JOIN ( SELECT * 	,CONCAT ( 		'%' 		,condition_investments 		,'%' 		) AS investment_pattern FROM axiom_datalake.cl_snapshot.master_ddq_sh WHERE condition_investment_type IS NOT NULL 	AND condition_investments IS NOT NULL ) sh2 ON base.investments = sh2.condition_investments AND coalesce(base.investment_type_sp, base.investment_type) = sh2.condition_investment_type
		WHERE base.af_inv_per_as_of_date IS NOT NULL
		)
	,CTE_query_2 AS (
		SELECT fof ,MIN(axiom_fund_name) AS axiom_fund_name ,MIN(axiomfund_investment_iqid) AS axiomfund_investment_iqid ,MIN(vintage_year_of_fund) AS vintage_year_of_fund ,MIN(investments) AS investments ,qr_name AS qr_name ,MIN(soi_name) AS soi_name ,MIN(gp) AS gp ,MIN(gp_um) AS gp_um ,investment_type ,MIN(pool) AS pool ,MIN(investment_currency) AS investment_currency 
		
		,COALESCE(SUM(total_fund_size_lcy), MIN(total_fund_size_lcy)) AS total_fund_size_lcy 
		
		,COALESCE(SUM(total_fund_size_lcy_um), MIN(total_fund_size_lcy_um)) AS total_fund_size_lcy_um ,COALESCE(SUM(total_fund_size_usd), MIN(total_fund_size_usd)) AS total_fund_size_usd ,COALESCE(SUM(total_fund_size_usd_um), MIN(total_fund_size_usd_um)) AS total_fund_size_usd_um ,coalesce(sum(original_vehicle_size_usd), min(original_vehicle_size_usd)) AS original_vehicle_size_usd ,coalesce(sum(original_vehicle_size_usd_um), min(original_vehicle_size_usd_um)) AS original_vehicle_size_usd_um ,coalesce(sum(original_vehicle_size_lcy), min(original_vehicle_size_lcy)) AS original_vehicle_size_lcy ,coalesce(sum(original_vehicle_size_lcy_um), min(original_vehicle_size_lcy_um)) AS original_vehicle_size_lcy_um ,MIN(geography_one_pager) AS geography_one_pager ,MIN(geography_gn_analysis) AS geography_gn_analysis ,MIN(strategy_one_pager) AS strategy_one_pager ,MIN(strategy_for_gn_analysis) AS strategy_for_gn_analysis ,MIN(vintage_year) AS vintage_year ,MIN(vintage_cambridge) AS vintage_cambridge ,MIN(initial_investment_date) AS initial_investment_date ,MIN(liquidation_exit_date) AS liquidation_exit_date ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(investment_status) 	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL 		AND MIN(liquidation_exit_date) = '' 		OR MIN(liquidation_exit_date) = 'NULL' 		OR MIN(liquidation_exit_date) IS NULL 		THEN 'Unrealized' 	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL 		AND MIN(liquidation_exit_date) IS NOT NULL 		THEN 'realized' 	END AS investment_status ,CASE  	WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Fully Realized,Fully Realized' 		THEN 'Fully Realized' 	WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Unrealized,Unrealized' 		THEN 'Unrealized' 	WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) = 'Partially Realized,Partially Realized' 		THEN 'Partially Realized' 	WHEN CONCAT_WS(',', COLLECT_LIST(investment_status_fund_derived)) IN ( 			'Fully Realized,Partially Realized' 			,'Partially Realized,Fully Realized' 			,'Fully Realized,Unrealized' 			,'Unrealized,Fully Realized' 			,'Partially Realized,Unrealized' 			,'Unrealized,Partially Realized' 			) 		THEN 'Partially Realized' 	ELSE MIN(investment_status_fund_derived) 	END AS investment_status_fund_derived ,MIN(no_of_prior_manager_funds_committed_to) AS no_of_prior_manager_funds_committed_to ,min(no_of_prior_manager_funds_committed_to_um) AS no_of_prior_manager_funds_committed_to_um ,MIN(names_of_lp_advisory) AS names_of_lp_advisory ,MIN(names_of_investment_lead) AS names_of_investment_lead ,MIN(axiom_was_a_sponsor) AS axiom_was_a_sponsor ,MIN(board_seat_fund_only) AS board_seat_fund_only ,min(board_seat_fund_only_um) AS board_seat_fund_only_um ,MIN(next_gen) AS next_gen ,min(next_gen_um) AS next_gen_um ,MIN(over_subscribed) AS over_subscribed ,MIN(co_investment_offered) AS co_investment_offered ,MIN(discount_perc_to_nav) AS discount_perc_to_nav ,sum(commitment_usd) AS commitment_usd ,coalesce(sum(remaining_commitment_usd), MIN(remaining_commitment_usd)) AS remaining_commitment_usd ,coalesce(((sum(contribution_net_of_recallable_usd) / sum(commitment_usd))) * 100, MIN(perc_drawn_usd)) AS perc_drawn_usd 		,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL AND MIN(investments_sp)	IS NULL		THEN MIN(perc_drawn_lcy) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					perc_drawn_lcy 					,'%' 					))) 	END AS perc_drawn_lcy ,coalesce(sum(distribution_net_of_recallable_usd), MIN(distribution_net_of_recallable_usd)) AS distribution_net_of_recallable_usd ,coalesce(sum(distribution_net_of_recallable_usd_um), MIN(distribution_net_of_recallable_usd_um)) AS distribution_net_of_recallable_usd_um ,coalesce(sum(contribution_net_of_recallable_usd), MIN(contribution_net_of_recallable_usd)) AS contribution_net_of_recallable_usd ,coalesce(sum(contribution_net_of_recallable_usd_um), MIN(contribution_net_of_recallable_usd_um)) AS contribution_net_of_recallable_usd_um ,coalesce(sum(cast(nav_usd AS DOUBLE)), MIN(nav_usd)) AS nav_usd ,coalesce(sum(cast(nav_usd_um AS DOUBLE)), MIN(nav_usd_um)) AS nav_usd_um ,sum(nav_lcy) AS nav_lcy ,coalesce(sum(total_value_usd), MIN(total_value_usd)) AS total_value_usd ,sum(total_value_usd_um) AS total_value_usd_um ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL AND MIN(investments_sp)	IS NULL	THEN MIN(gn_irr_usd) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					gn_irr_usd 					,'%' 					))) 	END AS gn_irr_usd ,MIN(gn_irr_usd_um) AS gn_irr_usd_um 	,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL AND MIN(investments_sp)	IS NULL		THEN MIN(gn_irr_lcy) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					gn_irr_lcy 					,'%' 					))) 	END AS gn_irr_lcy ,MIN(gn_irr_lcy_um) AS gn_irr_lcy_um ,coalesce((sum(total_value_usd) / sum(contribution_net_of_recallable_usd)), MIN(gn_moc_usd)) AS gn_moc_usd ,coalesce((sum(total_value_usd_um) / sum(contribution_net_of_recallable_usd_um)), MIN(gn_moc_usd)) AS gn_moc_usd_um ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL AND MIN(investments_sp)	IS NULL	THEN MIN(gn_moc_lcy) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					gn_moc_lcy 					,'x' 					))) 	END AS gn_moc_lcy ,MIN(gn_moc_lcy_um) AS gn_moc_lcy_um ,coalesce((sum(distribution_net_of_recallable_usd) / sum(contribution_net_of_recallable_usd)), MIN(dpi_usd)) AS dpi_usd ,coalesce((sum(distribution_net_of_recallable_usd_um) / sum(contribution_net_of_recallable_usd_um)), MIN(dpi_usd_um)) AS dpi_usd_um ,coalesce(sum(original_commitment_lcy), min(original_commitment_lcy)) AS original_commitment_lcy ,coalesce(sum(original_commitment_lcy_um), min(original_commitment_lcy_um)) AS original_commitment_lcy_um

		,coalesce(SUM(fund_size_sec), MIN(fund_size_sec)) AS fund_size_sec 
		
		,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL AND MIN(investments_sp)	IS NULL	THEN MIN(perc_drawn_sec) 	WHEN ARRAY_CONTAINS(COLLECT_LIST(CAST(perc_drawn_sec AS STRING)), 'N/A') 		THEN 'N/A' 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 					perc_drawn_sec 					,'%' 					))) 	END AS perc_drawn_sec ,sum(purchase_price_sec) AS purchase_price_sec,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL AND MIN(investments_sp)	IS NULL		THEN MIN(prior_year_gn_moc_usd) 	ELSE CONCAT_WS(', ', COLLECT_LIST(CONCAT ( 	prior_year_gn_moc_usd 	,'x' 	))) 	END prior_year_gn_moc_usd ,MIN(prior_year_gn_moc_usd_um) AS prior_year_gn_moc_usd_um ,SUM(contribution_usd) AS contribution_usd ,SUM(distribution_usd) AS distribution_usd ,SUM(recallable_usd) AS recallable_usd ,coalesce(sum(remaining_commitment_usd_um), MIN(remaining_commitment_usd_um)) AS remaining_commitment_usd_um ,MIN(perc_drawn_usd_um) AS perc_drawn_usd_um ,MIN(perc_drawn_lcy_um) AS perc_drawn_lcy_um ,MIN(af_inv_per_as_of_date) AS af_inv_per_as_of_date ,MIN(af_inv_per_report_type) AS af_inv_per_report_type ,MIN(gn_irr_usd_dyn) AS gn_irr_usd_dyn ,MIN(gn_irr_usd_dyn_um) / 100 AS gn_irr_usd_dyn_um ,MIN(gn_irr_lcy_dyn) AS gn_irr_lcy_dyn ,MIN(gn_irr_lcy_dyn_um) / 100 AS gn_irr_lcy_dyn_um ,sum(commitment_usd_dyn) AS commitment_usd_dyn ,coalesce(SUM(remaining_commitment_usd_dyn), MIN(remaining_commitment_usd_dyn)) AS remaining_commitment_usd_dyn ,SUM(remaining_commitment_usd_dyn_um) AS remaining_commitment_usd_dyn_um ,SUM(contribution_usd_dyn) AS contribution_usd_dyn ,SUM(distribution_usd_dyn) AS distribution_usd_dyn ,SUM(recallable_usd_dyn) AS recallable_usd_dyn ,coalesce(SUM(contribution_net_of_recallable_usd_dyn), MIN(contribution_net_of_recallable_usd_dyn)) AS contribution_net_of_recallable_usd_dyn ,SUM(contribution_net_of_recallable_usd_dyn_um) AS contribution_net_of_recallable_usd_dyn_um ,coalesce(SUM(distribution_net_of_recallable_usd_dyn), MIN(distribution_net_of_recallable_usd_dyn)) AS distribution_net_of_recallable_usd_dyn ,SUM(distribution_net_of_recallable_usd_dyn_um) AS distribution_net_of_recallable_usd_dyn_um ,coalesce((((SUM(contribution_net_of_recallable_usd_dyn) / SUM(commitment_usd_dyn))) * 100), MIN(perc_drawn_usd_dyn)) AS perc_drawn_usd_dyn ,(((SUM(contribution_net_of_recallable_usd_dyn_um) / SUM(commitment_usd_dyn))) * 100) / 100 AS perc_drawn_usd_dyn_um ,coalesce(SUM(nav_usd_dyn), MIN(nav_usd_dyn)) AS nav_usd_dyn ,SUM(nav_usd_dyn_um) AS nav_usd_dyn_um ,SUM(nav_lcy_dyn) AS nav_lcy_dyn ,coalesce(SUM(total_value_usd_dyn), MIN(total_value_usd_dyn)) AS total_value_usd_dyn ,SUM(total_value_usd_dyn_um) AS total_value_usd_dyn_um ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(gn_moc_usd_dyn) 	ELSE coalesce((SUM(total_value_usd_dyn) / SUM(contribution_net_of_recallable_usd_dyn)), MIN(gn_moc_usd_dyn)) 	END AS gn_moc_usd_dyn ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(gn_moc_usd_dyn_um) 	ELSE (SUM(total_value_usd_dyn_um) / SUM(contribution_net_of_recallable_usd_dyn_um)) 	END AS gn_moc_usd_dyn_um ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(perc_drawn_sec_dyn) 	ELSE COALESCE((((SUM(remaining_commitment_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100), MIN(perc_drawn_sec_dyn)) 	END AS perc_drawn_sec_dyn ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(perc_drawn_sec_dyn_um) 	ELSE COALESCE((((SUM(remaining_commitment_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100), MIN(perc_drawn_sec_dyn_um)) 	END AS perc_drawn_sec_dyn_um ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(perc_drawn_lcy_dyn) 	ELSE COALESCE((((SUM(contribution_net_of_recallable_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100), MIN(perc_drawn_lcy_dyn)) 	END AS perc_drawn_lcy_dyn ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(perc_drawn_lcy_dyn_um) 	ELSE COALESCE((((SUM(contribution_net_of_recallable_lcy_dyn) / SUM(commitment_lcy_dyn))) * 100), MIN(perc_drawn_lcy_dyn_um)) 	END AS perc_drawn_lcy_dyn_um ,coalesce((SUM(distribution_net_of_recallable_usd_dyn) / SUM(contribution_net_of_recallable_usd_dyn)), MIN(dpi_usd_dyn)) AS dpi_usd_dyn ,(SUM(distribution_net_of_recallable_usd_dyn_um) / SUM(contribution_net_of_recallable_usd_dyn_um)) AS dpi_usd_dyn_um 
		
		--,COALESCE(MIN(gn_moc_lcy_dyn), (SUM(total_value_lcy_dyn) / SUM(contribution_net_of_recallable_lcy_dyn))) AS gn_moc_lcy_dyn 
			
		--,COALESCE( MIN(gn_moc_lcy_dyn),((SUM(gn_distribution_lcy_dyn) - SUM(gn_recallable_lcy_dyn) + SUM(nav_usd_dyn) )/(SUM(gn_contribution_lcy_dyn) -  SUM(gn_recallable_lcy_dyn))))AS gn_moc_lcy_dyn
		,(SUM(gn_distribution_lcy_dyn) - SUM(gn_recallable_lcy_dyn) + SUM(nav_usd_dyn) )/(SUM(gn_contribution_lcy_dyn) -  SUM(gn_recallable_lcy_dyn)) AS gn_moc_lcy_dyn
		
		,(SUM(total_value_lcy_dyn) / SUM(contribution_net_of_recallable_lcy_dyn)) AS gn_moc_lcy_dyn_um ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(prior_year_gn_moc_usd_dyn) 	ELSE (sum(prior_year_total_value_usd_dyn) / sum(prior_year_contribution_net_of_recallable_usd_dyn)) 	END AS prior_year_gn_moc_usd_dyn ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(prior_year_gn_moc_usd_dyn_um) 	ELSE (sum(prior_year_total_value_usd_dyn) / sum(prior_year_contribution_net_of_recallable_usd_dyn)) 	END AS prior_year_gn_moc_usd_dyn_um ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN coalesce(SUM(perc_of_fund_lcy_one_pager), min(perc_of_fund_lcy_one_pager)) 	ELSE (sum(gn_investee_fund_size) / sum(gn_investee_fund_size_lcy)) 	END AS perc_of_fund_lcy_one_pager ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN coalesce(sum(perc_of_fund_lcy_alloc_rule), MIN(perc_of_fund_lcy_alloc_rule)) 	ELSE (sum(gn_investee_fund_size) / sum(gn_investee_fund_size_lcy)) 	END AS perc_of_fund_lcy_alloc_rule ,MIN(investments_um3) AS investments_um ,min(valuation_type_gg) AS valuation_type_gg ,min(valuation_type_gn) AS valuation_type_gn ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(gg_moc_dyn) 	ELSE coalesce(((SUM(gg_nav_dyn) + SUM(gg_cash_returned_dyn)) / SUM(gg_cash_invested_dyn)), MIN(gg_moc_dyn)) 	END AS gg_moc_dyn ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(gg_moc_dyn) 	ELSE coalesce(((SUM(gg_nav_dyn) + SUM(gg_cash_returned_dyn)) / SUM(gg_cash_invested_dyn)), MIN(gg_moc_dyn)) 	END AS gg_moc_dyn_um ,SUM(gg_nav_dyn) AS gg_nav_dyn ,SUM(gg_cash_returned_dyn) AS gg_cash_returned_dyn ,SUM(gg_cash_invested_dyn) AS gg_cash_invested_dyn ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(nn_moc_dyn) 	ELSE coalesce(((SUM(nn_distribution_dyn) + SUM(nn_nav_dyn)) / SUM(nn_contribution_dyn)), MIN(nn_moc_dyn)) 	END AS nn_moc_dyn,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(nn_moc_dyn) 	ELSE coalesce(((SUM(nn_distribution_dyn) + SUM(nn_nav_dyn)) / SUM(nn_contribution_dyn)), MIN(nn_moc_dyn)) 	END AS nn_moc_dyn_um ,CASE WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL THEN MIN(nn_distribution_dyn) ELSE SUM(nn_distribution_dyn) END AS nn_distribution_dyn,CASE WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL THEN MIN(nn_nav_dyn) ELSE SUM(nn_nav_dyn) END AS nn_nav_dyn,CASE WHEN MIN(dim_blocker_vehicle_id) IS NOT NULL OR MIN(investments_sp) IS NOT NULL THEN MIN(nn_contribution_dyn) ELSE SUM(nn_contribution_dyn) END AS nn_contribution_dyn,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(nn_dpi_dyn) 	ELSE coalesce(SUM(nn_distribution_dyn) / SUM(nn_contribution_dyn), MIN(nn_dpi_dyn)) 	END AS nn_dpi_dyn ,CASE  	WHEN MIN(dim_blocker_vehicle_id) IS NULL 		THEN MIN(nn_dpi_dyn) 	ELSE coalesce(SUM(nn_distribution_dyn) / SUM(nn_contribution_dyn), MIN(nn_dpi_dyn)) 	END AS nn_dpi_dyn_um	
		,min(re_up) AS re_up ,min(re_up_um) AS re_up_um ,
		SUM(gn_distribution_lcy_dyn) AS gn_distribution_lcy_dyn
		,SUM(gn_recallable_lcy_dyn) AS gn_recallable_lcy_dyn
		,SUM(gn_contribution_lcy_dyn) AS gn_contribution_lcy_dyn
		FROM CTE_query_1 a
		GROUP BY fof ,qr_name ,investment_type
		)
	,CTE_query_3 AS (
		SELECT fof ,axiom_fund_name ,axiomfund_investment_iqid ,coalesce(vintage_year_of_fund, 'NULL') AS vintage_year_of_fund ,b.investments AS investments ,coalesce(qr_name, 'NULL') AS qr_name ,coalesce(soi_name, 'NULL') AS soi_name ,coalesce(gp, 'NULL') AS gp ,gp_um ,coalesce(b.investment_type, 'NULL') AS investment_type ,coalesce(pool, 'NULL') AS pool ,coalesce(investment_currency, 'NULL') AS investment_currency ,coalesce(cast(cast(total_fund_size_lcy AS DECIMAL(18, 6)) AS string), total_fund_size_lcy, 'NULL') AS total_fund_size_lcy ,coalesce(cast(cast(total_fund_size_lcy_um AS DECIMAL(18, 6)) AS string), total_fund_size_lcy_um, 'NULL') AS total_fund_size_lcy_um ,coalesce(cast(cast(total_fund_size_usd AS DECIMAL(18, 6)) AS string), total_fund_size_usd, 'NULL') AS total_fund_size_usd ,coalesce(cast(cast(total_fund_size_usd_um AS DECIMAL(18, 6)) AS string), total_fund_size_usd_um, 'NULL') AS total_fund_size_usd_um ,coalesce(cast(cast(original_vehicle_size_usd AS DECIMAL(18, 6)) AS string), original_vehicle_size_usd, 'NULL') AS original_vehicle_size_usd ,coalesce(cast(cast(original_vehicle_size_usd_um AS DECIMAL(18, 6)) AS string), original_vehicle_size_usd_um, 'NULL') AS original_vehicle_size_usd_um ,coalesce(cast(cast(original_vehicle_size_lcy AS DECIMAL(18, 6)) AS string), original_vehicle_size_lcy) AS original_vehicle_size_lcy ,coalesce(cast(cast(original_vehicle_size_lcy_um AS DECIMAL(18, 6)) AS string), original_vehicle_size_lcy_um) AS original_vehicle_size_lcy_um ,coalesce(cast(cast(perc_of_fund_lcy_one_pager AS DECIMAL(18, 6)) AS string), perc_of_fund_lcy_one_pager, 'NULL') AS perc_of_fund_lcy_one_pager ,coalesce(cast(cast(perc_of_fund_lcy_alloc_rule AS DECIMAL(18, 6)) AS string), perc_of_fund_lcy_alloc_rule, 'NULL') AS perc_of_fund_lcy_alloc_rule ,coalesce(geography_one_pager, 'NULL') AS geography_one_pager ,coalesce(geography_gn_analysis, 'NULL') AS geography_gn_analysis ,coalesce(strategy_one_pager, 'NULL') AS strategy_one_pager ,coalesce(strategy_for_gn_analysis, 'NULL') AS strategy_for_gn_analysis ,coalesce(vintage_year, 'NULL') AS vintage_year ,coalesce(vintage_cambridge, 'NULL') AS vintage_cambridge ,coalesce(initial_investment_date, 'NULL') AS initial_investment_date ,CASE  	WHEN liquidation_exit_date = '' 		THEN 'NULL' 	ELSE coalesce(liquidation_exit_date, 'NULL') 	END AS liquidation_exit_date ,coalesce(investment_status, 'NULL') AS ips_investment_status ,coalesce(investment_status_fund_derived, 'NULL') AS realisation_status_derived ,coalesce(no_of_prior_manager_funds_committed_to, 'NULL') AS no_of_prior_manager_funds_committed_to ,no_of_prior_manager_funds_committed_to_um ,coalesce(names_of_lp_advisory, 'NULL') AS names_of_lp_advisory ,coalesce(names_of_investment_lead, 'NULL') AS names_of_investment_lead ,coalesce(axiom_was_a_sponsor, 'NULL') AS axiom_was_a_sponsor ,coalesce(board_seat_fund_only, 'NULL') AS board_seat_fund_only ,board_seat_fund_only_um ,coalesce(next_gen, 'NULL') AS next_gen ,next_gen_um ,coalesce(over_subscribed, 'NULL') AS over_subscribed ,coalesce(cast(co_investment_offered AS string), 'NULL') AS co_investment_offered ,coalesce(discount_perc_to_nav, 'NULL') AS discount_perc_to_nav ,coalesce(cast(cast(commitment_usd AS DECIMAL(18, 6)) AS string), commitment_usd, 'NULL') AS commitment_usd,coalesce(cast(cast(contribution_usd AS DECIMAL(18, 6)) AS string), contribution_usd, 'NULL') AS contribution_usd ,coalesce(cast(cast(distribution_usd AS DECIMAL(18, 6)) AS string), distribution_usd, 'NULL') AS distribution_usd ,coalesce(cast(cast(recallable_usd AS DECIMAL(18, 6)) AS string), recallable_usd, 'NULL') AS recallable_usd ,coalesce(cast(cast(distribution_net_of_recallable_usd AS DECIMAL(18, 6)) AS string), distribution_net_of_recallable_usd, 'NULL') AS distribution_net_of_recallable_usd ,coalesce(cast(cast(distribution_net_of_recallable_usd_um AS DECIMAL(18, 6)) AS string), distribution_net_of_recallable_usd_um, 'NULL') AS distribution_net_of_recallable_usd_um ,coalesce(cast(cast(contribution_net_of_recallable_usd AS DECIMAL(18, 6)) AS string), contribution_net_of_recallable_usd, 'NULL') AS contribution_net_of_recallable_usd ,coalesce(cast(cast(contribution_net_of_recallable_usd_um AS DECIMAL(18, 6)) AS string), contribution_net_of_recallable_usd_um, 'NULL') AS contribution_net_of_recallable_usd_um ,coalesce(cast(cast(remaining_commitment_usd AS DECIMAL(18, 6)) AS string), remaining_commitment_usd, 'NULL') AS remaining_commitment_usd ,coalesce(cast(cast(remaining_commitment_usd_um AS DECIMAL(18, 6)) AS string), remaining_commitment_usd_um, 'NULL') AS remaining_commitment_usd_um ,CASE  	WHEN b.Investments = 'Adamantem Capital Fund II' 		THEN CONCAT ( 				replace(perc_drawn_usd, '%%', '%') 				,'(To Be Manually Updated)' 				) 	ELSE ( 			CASE  				WHEN perc_drawn_usd LIKE '%,%' 					THEN coalesce(replace(perc_drawn_usd, '%%', '%'), 'NULL') 				ELSE coalesce(replace(perc_drawn_usd, '%', ''), 'NULL') 				END 			) 	END AS perc_drawn_usd ,perc_drawn_usd_um AS perc_drawn_usd_um ,CASE  	WHEN b.Investments = 'Adamantem Capital Fund II' 		THEN CONCAT ( 				replace(perc_drawn_lcy, '%%', '%') 				,'(To Be Manually Updated)' 				) 	ELSE ( 			CASE  				WHEN perc_drawn_lcy LIKE '%,%' 					THEN coalesce(replace(perc_drawn_lcy, '%%', '%'), 'NULL') 				ELSE coalesce(replace(perc_drawn_lcy, '%', ''), 'NULL') 				END 			) 	END AS perc_drawn_lcy ,perc_drawn_lcy_um AS perc_drawn_lcy_um ,coalesce(cast(cast(nav_usd AS DECIMAL(18, 6)) AS string), nav_usd, 'NULL') AS nav_usd ,coalesce(cast(cast(nav_usd_um AS DECIMAL(18, 6)) AS string), nav_usd_um, 'NULL') AS nav_usd_um ,coalesce(cast(cast(nav_lcy AS DECIMAL(18, 6)) AS string), nav_lcy, 'NULL') AS nav_lcy ,coalesce(cast(cast(total_value_usd AS DECIMAL(18, 6)) AS string), total_value_usd, 'NULL') AS total_value_usd ,coalesce(cast(cast(total_value_usd_um AS DECIMAL(18, 6)) AS string), total_value_usd_um, 'NULL') AS total_value_usd_um ,CASE  	WHEN gn_irr_usd LIKE '%,%' 		THEN coalesce(replace(gn_irr_usd, '%%', '%'), 'NULL') 	ELSE coalesce(replace(gn_irr_usd, '%', ''), 'NULL') 	END AS gn_irr_usd ,gn_irr_usd_um AS gn_irr_usd_um ,CASE  	WHEN gn_moc_usd LIKE '%,%' 		THEN coalesce(replace(gn_moc_usd, 'xx', 'x'), 'NULL') 	ELSE coalesce(replace(gn_moc_usd, 'x', ''), 'NULL') 	END AS gn_moc_usd ,gn_moc_usd_um AS gn_moc_usd_um ,CASE  	WHEN gn_irr_lcy LIKE '%,%' 		THEN coalesce(replace(gn_irr_lcy, '%%', '%'), 'NULL') 	ELSE coalesce(replace(gn_irr_lcy, '%', ''), 'NULL') 	END AS gn_irr_lcy ,gn_irr_lcy_um ,CASE  	WHEN gn_moc_lcy LIKE '%,%' 		THEN coalesce(replace(gn_moc_lcy, 'xx', 'x'), 'NULL') 	ELSE coalesce(replace(gn_moc_lcy, 'x', ''), 'NULL') 	END AS gn_moc_lcy ,gn_moc_lcy_um ,CASE  	WHEN dpi_usd LIKE '%,%' 		THEN coalesce(replace(dpi_usd, 'xx', 'x'), 'NULL') 	ELSE coalesce(replace(dpi_usd, 'x', ''), 'NULL') 	END AS dpi_usd ,dpi_usd_um ,coalesce(cast(cast(original_commitment_lcy AS DECIMAL(18, 6)) AS string), original_commitment_lcy, 'NULL') AS original_commitment_lcy,cast(original_commitment_lcy AS DECIMAL(18, 6)) AS original_commitment_lcy_um ,coalesce(cast(cast(fund_size_sec AS DECIMAL(18, 6)) AS string), fund_size_sec, 'NULL') AS fund_size_sec ,CASE  	WHEN perc_drawn_sec LIKE '%,%' 		THEN coalesce(replace(perc_drawn_sec, '%%', '%'), 'NULL') 	ELSE coalesce(replace(perc_drawn_sec, '%', ''), 'NULL') 	END AS perc_drawn_sec ,coalesce(cast(cast(purchase_price_sec AS DECIMAL(18, 6)) AS string), purchase_price_sec, 'NULL') AS purchase_price_sec ,CASE  	WHEN prior_year_gn_moc_usd LIKE '%x, %' 		THEN coalesce(replace(prior_year_gn_moc_usd, 'xx', 'x'), 'NULL') 	ELSE coalesce(replace(prior_year_gn_moc_usd, 'x', ''), 'NULL') 	END AS prior_year_gn_moc_usd ,prior_year_gn_moc_usd_um ,af_inv_per_as_of_date AS as_of_date ,af_inv_per_report_type AS report_type ,coalesce(sh1.revised_gn_irr_usd, sh.revised_gn_irr_usd, cast(cast(c.gn_irr_rpt * 100 AS DECIMAL(18, 6)) AS string), gn_irr_usd_dyn, 'NULL') AS gn_irr_usd_dyn ,coalesce(cast(cast((c.gn_irr_rpt * 100 / 100) AS DECIMAL(18, 6)) AS string), gn_irr_usd_dyn_um, 'NULL') AS gn_irr_usd_dyn_um ,coalesce(sh1.revised_gn_irr_lcy, sh.revised_gn_irr_lcy, cast(cast(c.gn_irr_lcy_rpt * 100 AS DECIMAL(18, 6)) AS string), gn_irr_lcy_dyn, 'NULL') AS gn_irr_lcy_dyn ,coalesce(cast(cast((c.gn_irr_lcy_rpt * 100 / 100) AS DECIMAL(18, 6)) AS string), gn_irr_lcy_dyn_um, 'NULL') AS gn_irr_lcy_dyn_um ,coalesce(cast(cast(commitment_usd_dyn AS DECIMAL(18, 6)) AS string), commitment_usd_dyn, 'NULL') AS commitment_usd_dyn ,coalesce(cast(cast(remaining_commitment_usd_dyn AS DECIMAL(18, 6)) AS string), remaining_commitment_usd_dyn, 'NULL') AS remaining_commitment_usd_dyn ,coalesce(cast(cast(remaining_commitment_usd_dyn_um AS DECIMAL(18, 6)) AS string), remaining_commitment_usd_dyn_um, 'NULL') AS remaining_commitment_usd_dyn_um ,coalesce(cast(cast(contribution_usd_dyn AS DECIMAL(18, 6)) AS string), contribution_usd_dyn, 'NULL') AS contribution_usd_dyn ,coalesce(cast(cast(distribution_usd_dyn AS DECIMAL(18, 6)) AS string), distribution_usd_dyn, 'NULL') AS distribution_usd_dyn ,coalesce(cast(cast(recallable_usd_dyn AS DECIMAL(18, 6)) AS string), recallable_usd_dyn, 'NULL') AS recallable_usd_dyn ,coalesce(cast(cast(contribution_net_of_recallable_usd_dyn AS DECIMAL(18, 6)) AS string), contribution_net_of_recallable_usd_dyn, 'NULL') AS contribution_net_of_recallable_usd_dyn ,coalesce(cast(cast(contribution_net_of_recallable_usd_dyn_um AS DECIMAL(18, 6)) AS string), contribution_net_of_recallable_usd_dyn_um, 'NULL') AS contribution_net_of_recallable_usd_dyn_um ,coalesce(cast(cast(distribution_net_of_recallable_usd_dyn AS DECIMAL(18, 6)) AS string), distribution_net_of_recallable_usd_dyn, 'NULL') AS distribution_net_of_recallable_usd_dyn ,coalesce(cast(cast(distribution_net_of_recallable_usd_dyn_um AS DECIMAL(18, 6)) AS string), distribution_net_of_recallable_usd_dyn_um, 'NULL') AS distribution_net_of_recallable_usd_dyn_um ,coalesce(cast(cast(perc_drawn_usd_dyn AS DECIMAL(18, 6)) AS string), perc_drawn_usd_dyn, 'NULL') AS perc_drawn_usd_dyn ,coalesce(cast(cast(perc_drawn_usd_dyn_um AS DECIMAL(18, 6)) AS string), perc_drawn_usd_dyn_um, 'NULL') AS perc_drawn_usd_dyn_um ,coalesce(cast(cast(perc_drawn_lcy_dyn AS DECIMAL(18, 6)) AS string), perc_drawn_lcy_dyn, 'NULL') AS perc_drawn_lcy_dyn ,coalesce(cast(cast(perc_drawn_lcy_dyn_um AS DECIMAL(18, 6)) AS string), perc_drawn_lcy_dyn_um, 'NULL') AS perc_drawn_lcy_dyn_um ,coalesce(cast(cast(nav_usd_dyn AS DECIMAL(18, 6)) AS string), nav_usd_dyn, 'NULL') AS nav_usd_dyn ,coalesce(cast(cast(nav_usd_dyn_um AS DECIMAL(18, 6)) AS string), nav_usd_dyn_um, 'NULL') AS nav_usd_dyn_um ,coalesce(cast(cast(nav_lcy_dyn AS DECIMAL(18, 6)) AS string), nav_lcy_dyn, 'NULL') AS nav_lcy_dyn ,coalesce(cast(cast(total_value_usd_dyn AS DECIMAL(18, 6)) AS string), total_value_usd_dyn, 'NULL') AS total_value_usd_dyn ,coalesce(cast(cast(total_value_usd_dyn_um AS DECIMAL(18, 6)) AS string), total_value_usd_dyn_um, 'NULL') AS total_value_usd_dyn_um ,coalesce(cast(cast(dpi_usd_dyn AS DECIMAL(18, 6)) AS string), dpi_usd_dyn, 'NULL') AS dpi_usd_dyn ,coalesce(cast(cast(dpi_usd_dyn_um AS DECIMAL(18, 6)) AS string), dpi_usd_dyn_um, 'NULL') AS dpi_usd_dyn_um ,coalesce(cast(cast(gn_moc_usd_dyn AS DECIMAL(18, 6)) AS string), gn_moc_usd_dyn, 'NULL') AS gn_moc_usd_dyn ,coalesce(cast(cast(gn_moc_usd_dyn_um AS DECIMAL(18, 6)) AS string), gn_moc_usd_dyn_um, 'NULL') AS gn_moc_usd_dyn_um ,coalesce(cast(cast(gn_moc_lcy_dyn AS DECIMAL(18, 6)) AS string), gn_moc_lcy_dyn, 'NULL') AS gn_moc_lcy_dyn ,coalesce(cast(cast(gn_moc_lcy_dyn_um AS DECIMAL(18, 6)) AS string), gn_moc_lcy_dyn_um) AS gn_moc_lcy_dyn_um ,coalesce(cast(cast(prior_year_gn_moc_usd_dyn AS DECIMAL(18, 6)) AS string), prior_year_gn_moc_usd_dyn, 'NULL') AS prior_year_gn_moc_usd_dyn ,coalesce(cast(cast(prior_year_gn_moc_usd_dyn_um AS DECIMAL(18, 6)) AS string), prior_year_gn_moc_usd_dyn_um, 'NULL') AS prior_year_gn_moc_usd_dyn_um ,CURRENT_DATE () AS reporting_date ,current_timestamp() AS etl_created_datetime ,current_timestamp() AS etl_updated_datetime ,coalesce(cast(cast(replace(perc_drawn_sec_dyn, '%', '') AS DECIMAL(18, 6)) AS string), perc_drawn_sec_dyn, 'NULL') AS perc_drawn_sec_dyn ,coalesce(cast(cast(replace(perc_drawn_sec_dyn_um, '%', '') AS DECIMAL(18, 6)) AS string), perc_drawn_sec_dyn_um, 'NULL') AS perc_drawn_sec_dyn_um ,b.investments_um ,valuation_type_gg ,valuation_type_gn ,coalesce(cast(cast(gg_moc_dyn AS DECIMAL(18, 6)) AS string), gg_moc_dyn, 'NULL') AS gg_moc_dyn ,coalesce(cast(cast(gg_moc_dyn_um AS DECIMAL(18, 6)) AS string), gg_moc_dyn_um, 'NULL') AS gg_moc_dyn_um ,coalesce(cast(cast(gg_nav_dyn AS DECIMAL(18, 6)) AS string), gg_nav_dyn, 'NULL') AS gg_nav_dyn ,coalesce(cast(cast(gg_cash_returned_dyn AS DECIMAL(18, 6)) AS string), gg_cash_returned_dyn, 'NULL') AS gg_cash_returned_dyn ,coalesce(cast(cast(gg_cash_invested_dyn AS DECIMAL(18, 6)) AS string), gg_cash_invested_dyn, 'NULL') AS gg_cash_invested_dyn ,coalesce(cast(cast(nn_distribution_dyn AS DECIMAL(18, 6)) AS string), nn_distribution_dyn, 'NULL') AS nn_distribution_dyn ,coalesce(cast(cast(nn_nav_dyn AS DECIMAL(18, 6)) AS string), nn_nav_dyn, 'NULL') AS nn_nav_dyn ,coalesce(cast(cast(nn_contribution_dyn AS DECIMAL(18, 6)) AS string), nn_contribution_dyn, 'NULL') AS nn_contribution_dyn ,coalesce(cast(cast(nn_moc_dyn AS DECIMAL(18, 6)) AS string), nn_moc_dyn, 'NULL') AS nn_moc_dyn ,coalesce(cast(cast(nn_moc_dyn_um AS DECIMAL(18, 6)) AS string), nn_moc_dyn_um, 'NULL') AS nn_moc_dyn_um ,coalesce(cast(cast(nn_dpi_dyn AS DECIMAL(18, 6)) AS string), nn_dpi_dyn, 'NULL') AS nn_dpi_dyn ,coalesce(cast(cast(nn_dpi_dyn_um AS DECIMAL(18, 6)) AS string), nn_dpi_dyn_um, 'NULL') AS nn_dpi_dyn_um ,re_up AS re_up ,re_up_um AS re_up_um ,CAST(ROUND(nn_irr * 100, 6) AS STRING) AS nn_irr_dyn
						
		FROM CTE_query_2 b
		LEFT JOIN axiom_datalake.cl_snapshot.rpt_invesment_irr_table c ON b.fof = c.fof_short_name AND lower(b.investment_type) = lower(c.investment_type) AND b.investments = c.investments AND b.af_inv_per_as_of_date = c.as_of_date AND b.af_inv_per_report_type = c.p_report_type
		LEFT JOIN axiom_datalake.cl_snapshot.nn_irr_table nn_irr ON b.fof = nn_irr.fof_short_name AND b.af_inv_per_as_of_date = nn_irr.as_of_date AND b.af_inv_per_report_type = nn_irr.p_report_type
		LEFT JOIN ( SELECT * FROM axiom_datalake.cl_snapshot.master_ddq_sh WHERE condition_investment_type IS NULL ) sh ON b.investments = sh.condition_investments
		LEFT JOIN ( SELECT * FROM axiom_datalake.cl_snapshot.master_ddq_sh WHERE condition_investment_type IS NOT NULL 	AND condition_investments IS NOT NULL ) sh1 ON b.investments = sh1.condition_investments AND b.investment_type = sh1.condition_investment_type
		)

-- INSERT INTO axiom_datalake.cl_snapshot.rpt_investment_level (
-- 	fof
-- 	,axiom_fund_name
-- 	,axiomfund_investment_iqid
-- 	,vintage_year_of_fund
-- 	,investments
-- 	,qr_name
-- 	,soi_name
-- 	,gp
-- 	,gp_um
-- 	,investment_type
-- 	,pool
-- 	,investment_currency
-- 	,total_fund_size_lcy
-- 	,total_fund_size_lcy_um
-- 	,total_fund_size_usd
-- 	,total_fund_size_usd_um
-- 	,original_vehicle_size_usd
-- 	,original_vehicle_size_usd_um
-- 	,original_vehicle_size_lcy
-- 	,original_vehicle_size_lcy_um
-- 	,perc_of_fund_lcy_one_pager
-- 	,perc_of_fund_lcy_alloc_rule
-- 	,geography_one_pager
-- 	,geography_gn_analysis
-- 	,strategy_one_pager
-- 	,strategy_for_gn_analysis
-- 	,vintage_year
-- 	,vintage_cambridge
-- 	,initial_investment_date
-- 	,liquidation_exit_date
-- 	,ips_investment_status
-- 	,realization_status_derived
-- 	,no_of_prior_manager_funds_committed_to
-- 	,no_of_prior_manager_funds_committed_to_um
-- 	,names_of_lp_advisory
-- 	,names_of_investment_lead
-- 	,axiom_was_a_sponsor
-- 	,board_seat_fund_only
-- 	,board_seat_fund_only_um
-- 	,next_gen
-- 	,next_gen_um
-- 	,over_subscribed
-- 	,co_investment_offered
-- 	,discount_perc_to_nav
-- 	,commitment_usd
-- 	,contribution_usd
-- 	,distribution_usd
-- 	,recallable_usd
-- 	,distribution_net_of_recallable_usd
-- 	,distribution_net_of_recallable_usd_um
-- 	,contribution_net_of_recallable_usd
-- 	,contribution_net_of_recallable_usd_um
-- 	,remaining_commitment_usd
-- 	,remaining_commitment_usd_um
-- 	,perc_drawn_usd
-- 	,perc_drawn_usd_um
-- 	,perc_drawn_lcy
-- 	,perc_drawn_lcy_um
-- 	,nav_usd
-- 	,nav_usd_um
-- 	,nav_lcy
-- 	,total_value_usd
-- 	,total_value_usd_um
-- 	,gn_irr_usd
-- 	,gn_irr_usd_um
-- 	,gn_moc_usd
-- 	,gn_moc_usd_um
-- 	,gn_irr_lcy
-- 	,gn_irr_lcy_um
-- 	,gn_moc_lcy
-- 	,gn_moc_lcy_um
-- 	,dpi_usd
-- 	,dpi_usd_um
-- 	,original_commitment_lcy
-- 	,original_commitment_lcy_um
-- 	,fund_size_sec
-- 	,perc_drawn_sec
-- 	,purchase_price_sec
-- 	,prior_year_gn_moc_usd
-- 	,prior_year_gn_moc_usd_um
-- 	,as_of_date
-- 	,report_type
-- 	,gn_irr_usd_dyn
-- 	,gn_irr_usd_dyn_um
-- 	,gn_irr_lcy_dyn
-- 	,gn_irr_lcy_dyn_um
-- 	,commitment_usd_dyn
-- 	,remaining_commitment_usd_dyn
-- 	,remaining_commitment_usd_dyn_um
-- 	,contribution_usd_dyn
-- 	,distribution_usd_dyn
-- 	,recallable_usd_dyn
-- 	,contribution_net_of_recallable_usd_dyn
-- 	,contribution_net_of_recallable_usd_dyn_um
-- 	,distribution_net_of_recallable_usd_dyn
-- 	,distribution_net_of_recallable_usd_dyn_um
-- 	,perc_drawn_usd_dyn
-- 	,perc_drawn_usd_dyn_um
-- 	,perc_drawn_lcy_dyn
-- 	,perc_drawn_lcy_dyn_um
-- 	,nav_usd_dyn
-- 	,nav_usd_dyn_um
-- 	,nav_lcy_dyn
-- 	,total_value_usd_dyn
-- 	,total_value_usd_dyn_um
-- 	,dpi_usd_dyn
-- 	,dpi_usd_dyn_um
-- 	,gn_moc_usd_dyn
-- 	,gn_moc_usd_dyn_um
-- 	,gn_moc_lcy_dyn
-- 	,gn_moc_lcy_dyn_um
-- 	,prior_year_gn_moc_usd_dyn
-- 	,prior_year_gn_moc_usd_dyn_um
-- 	,reporting_date
-- 	,etl_created_datetime
-- 	,etl_updated_datetime
-- 	,perc_drawn_sec_dyn
-- 	,perc_drawn_sec_dyn_um
-- 	,investments_um
-- 	,valuation_type_gg
-- 	,valuation_type_gn
-- 	,gg_moc_dyn
-- 	,gg_moc_dyn_um
-- 	,gg_nav_dyn
-- 	,gg_cash_returned_dyn
-- 	,gg_cash_invested_dyn
-- 	,nn_moc_dyn
-- 	,nn_moc_dyn_um
-- 	,nn_distribution_dyn
-- 	,nn_nav_dyn
-- 	,nn_contribution_dyn
-- 	,nn_dpi_dyn
-- 	,nn_dpi_dyn_um
-- 	,re_up
-- 	,re_up_um
-- 	,nn_irr_dyn
-- 	)
SELECT fof
	,axiom_fund_name
	,axiomfund_investment_iqid
	,vintage_year_of_fund
	,investments
	,qr_name
	,soi_name
	,gp
	,gp_um
	,investment_type
	,pool
	,investment_currency
	,total_fund_size_lcy
	,total_fund_size_lcy_um
	,total_fund_size_usd
	,total_fund_size_usd_um
	,original_vehicle_size_usd
	,original_vehicle_size_usd_um
	,original_vehicle_size_lcy
	,original_vehicle_size_lcy_um
	,perc_of_fund_lcy_one_pager
	,perc_of_fund_lcy_alloc_rule
	,geography_one_pager
	,geography_gn_analysis
	,strategy_one_pager
	,strategy_for_gn_analysis
	,vintage_year
	,vintage_cambridge
	,initial_investment_date
	,liquidation_exit_date
	,ips_investment_status
	,realisation_status_derived as realization_status_derived
	,no_of_prior_manager_funds_committed_to
	,no_of_prior_manager_funds_committed_to_um
	,names_of_lp_advisory
	,names_of_investment_lead
	,axiom_was_a_sponsor
	,board_seat_fund_only
	,board_seat_fund_only_um
	,next_gen
	,next_gen_um
	,over_subscribed
	,co_investment_offered
	,discount_perc_to_nav
	,commitment_usd
	,contribution_usd
	,distribution_usd
	,recallable_usd
	,distribution_net_of_recallable_usd
	,distribution_net_of_recallable_usd_um
	,contribution_net_of_recallable_usd
	,contribution_net_of_recallable_usd_um
	,remaining_commitment_usd
	,remaining_commitment_usd_um
	,perc_drawn_usd
	,perc_drawn_usd_um
	,perc_drawn_lcy
	,perc_drawn_lcy_um
	,nav_usd
	,nav_usd_um
	,nav_lcy
	,total_value_usd
	,total_value_usd_um
	,gn_irr_usd
	,gn_irr_usd_um
	,gn_moc_usd
	,gn_moc_usd_um
	,gn_irr_lcy
	,gn_irr_lcy_um
	,gn_moc_lcy
	,gn_moc_lcy_um
	,dpi_usd
	,dpi_usd_um
	,original_commitment_lcy
	,original_commitment_lcy_um
	,fund_size_sec
	,perc_drawn_sec
	,purchase_price_sec
	,prior_year_gn_moc_usd
	,prior_year_gn_moc_usd_um
	,as_of_date
	,report_type
	,gn_irr_usd_dyn
	,gn_irr_usd_dyn_um
	,gn_irr_lcy_dyn
	,gn_irr_lcy_dyn_um
	,commitment_usd_dyn
	,remaining_commitment_usd_dyn
	,remaining_commitment_usd_dyn_um
	,contribution_usd_dyn
	,distribution_usd_dyn
	,recallable_usd_dyn
	,contribution_net_of_recallable_usd_dyn
	,contribution_net_of_recallable_usd_dyn_um
	,distribution_net_of_recallable_usd_dyn
	,distribution_net_of_recallable_usd_dyn_um
	,perc_drawn_usd_dyn
	,perc_drawn_usd_dyn_um
	,perc_drawn_lcy_dyn
	,perc_drawn_lcy_dyn_um
	,nav_usd_dyn
	,nav_usd_dyn_um
	,nav_lcy_dyn
	,total_value_usd_dyn
	,total_value_usd_dyn_um
	,dpi_usd_dyn
	,dpi_usd_dyn_um
	,gn_moc_usd_dyn
	,gn_moc_usd_dyn_um
	,gn_moc_lcy_dyn
	,gn_moc_lcy_dyn_um
	,prior_year_gn_moc_usd_dyn
	,prior_year_gn_moc_usd_dyn_um
	,reporting_date
	,etl_created_datetime
	,etl_updated_datetime
	,perc_drawn_sec_dyn
	,perc_drawn_sec_dyn_um
	,investments_um
	,valuation_type_gg
	,valuation_type_gn
	,gg_moc_dyn
	,gg_moc_dyn_um
	,gg_nav_dyn
	,gg_cash_returned_dyn
	,gg_cash_invested_dyn
	,nn_moc_dyn
	,nn_moc_dyn_um
	,nn_distribution_dyn
	,nn_nav_dyn
	,nn_contribution_dyn
	,nn_dpi_dyn
	,nn_dpi_dyn_um
	,re_up
	,re_up_um
	,nn_irr_dyn
FROM CTE_query_3


In [0]:
%sql
select investments,report_type,as_of_date,fof,gn_irr_usd,gn_irr_usd_dyn,a.* from axiom_datalake.cl_snapshot.rpt_investment_level a